# 1. Modulos

In [ ]:
import datetime as dt
from datetime import datetime, date, timedelta

import os
import sys
from pathlib import Path
import pandas as pd
import openpyxl
from openpyxl import load_workbook
import xlsxwriter
import re  # Expresiones regulares para procesamiento de texto

# Añadir la carpeta raíz del proyecto al sys.path para importar módulos personalizados
sys.path.append(os.path.abspath("c:/Users/osmarrincon/Documents/capresoca-data-automation"))
#sys.path.append(os.path.abspath(r"C:\Users\crist\Documents\Proyectos Python\capresoca-data-automation"))  # Ruta alternativa (comentada)

# Importar función y clase personalizada del proyecto
from src.file_loader import cargar_maestros_ADRES  # Función para cargar archivos maestros ADRES
from src.data_cleaning import BduaReportProcessor      # Clase para limpiar y normalizar población Maestro ADRES
from src.data_cleaning import DataCleaner # Clase para limpiar y normalizar DataFrames de Pandas

# 2. Rutas y variables

## Oficce

In [ ]:
R_Maestro__EPSC25 = r"C:\Users\osmarrincon\OneDrive - 891856000_CAPRESOCA E P S\Capresoca\AlmostClear\Procesos BDUA\Contributivo\Maestro\2026\EPSC25MC0006012026.TXT"
R_Maestro__EPS025 = r"C:\Users\osmarrincon\OneDrive - 891856000_CAPRESOCA E P S\Capresoca\AlmostClear\Procesos BDUA\Subsidiados\Maestro\MS\2026\EPS025MS0006012026.TXT"
R_IPS = r"C:\Users\osmarrincon\OneDrive - 891856000_CAPRESOCA E P S\Capresoca\AlmostClear\Constantes\IPS_CODIGO.txt"
R_Dic_Nomenclaturas = r"C:\Users\osmarrincon\OneDrive - 891856000_CAPRESOCA E P S\Capresoca\AlmostClear\Constantes\NOMENCLATURA_DE_DIRECCIONES.xlsx"

# Crear el objeto de fecha
fecha_reporte = dt.datetime(2026, 1, 9)
fecha_archivo = "09-01-2026"

Ruta_Salida = fr"C:\Users\osmarrincon\OneDrive - 891856000_CAPRESOCA E P S\Escritorio\Yesid Rincón Z\Traslados\Procesos BDUA\2026\01_Enero\09"

S_Excel = fr"C:\Users\osmarrincon\OneDrive - 891856000_CAPRESOCA E P S\Escritorio\Yesid Rincón Z\Traslados\Procesos BDUA\2026\01_Enero\09\DataFrames_Activos 09012026.xlsx"
hoja = "Df_NS_Envio"

## Home

In [ ]:
#R_Maestro__EPSC25 = r"C:\Users\crist\OneDrive - 891856000_CAPRESOCA E P S\Capresoca\AlmostClear\Procesos BDUA\Contributivo\Maestro\2025-2\EPSC25MC0010122025.TXT"
#R_Maestro__EPS025 = r"C:\Users\crist\OneDrive - 891856000_CAPRESOCA E P S\Capresoca\AlmostClear\Procesos BDUA\Subsidiados\Maestro\MS\2025-02\EPS025MS0010122025.TXT"
#R_IPS = r"C:\Users\crist\OneDrive - 891856000_CAPRESOCA E P S\Capresoca\AlmostClear\Constantes\IPS_CODIGO.txt"

#R_Dic_Nomenclaturas = r"C:\Users\crist\OneDrive - 891856000_CAPRESOCA E P S\Capresoca\AlmostClear\Constantes\NOMENCLATURA_DE_DIRECCIONES.xlsx"
# Crear el objeto de fecha
#fecha_reporte = dt.datetime(2025, 12, 12)
#fecha_archivo = "12-12-2025"

#Ruta_Salida = fr"C:\Users\crist\OneDrive - 891856000_CAPRESOCA E P S\Escritorio\Yesid Rincón Z\Traslados\Procesos BDUA\2025\12_Diciembre\12"

#S_Excel = fr"C:\Users\crist\OneDrive - 891856000_CAPRESOCA E P S\Escritorio\Yesid Rincón Z\Traslados\Procesos BDUA\2025\12_Diciembre\12\DataFrames_Activos 12122025.xlsx"
#hoja = "Df_NS_Envio"

# 3. Cargue Dataframes

In [ ]:
# Cargar y combinar los maestros
maestro_ADRES = cargar_maestros_ADRES(R_Maestro__EPS025, R_Maestro__EPSC25)
df_NS = pd.read_excel(S_Excel, hoja, header=0, dtype=str)
df_Dic_Nomenclaturas = pd.read_excel(R_Dic_Nomenclaturas, "NOMENCLATURA", header=0, dtype=str)
df_IPS = pd.read_csv(R_IPS, sep=None, engine='python', encoding='utf-8', header=None, dtype=str)


# 4. Limpiar datos
## 4.1. Listado censal o Sisben ADRES

In [ ]:
# Duplicar la columna "MARCASISBENIV+MARCASISBENIII_2" y nombrarla "MARCASISBENIV+MARCASISBENIII"
maestro_ADRES["MARCASISBENIV+MARCASISBENIII_2"] = \
    maestro_ADRES["MARCASISBENIV+MARCASISBENIII"]

# 1. Instanciar el procesador: Se crea un objeto pasando el DataFrame.
#    La jerarquía de población ya está definida por defecto dentro de la clase.
processor = BduaReportProcessor(df=maestro_ADRES)

# 2. Ejecutar la limpieza y asignarla de vuelta.
#    El método retorna un DataFrame completamente nuevo con la columna actualizada.
maestro_ADRES = processor.prioritize_population_markers(
    col_name="MARCASISBENIV+MARCASISBENIII"
)

# ¡Listo! 'maestro_ADRES' ahora contiene los datos limpios.

In [ ]:
# Crear las dos nuevas columnas basadas en la columna MARCASISBENIV+MARCASISBENIII
def extraer_grupo_poblacional(valor):
	"""Extrae el grupo poblacional según las reglas especificadas"""
	if pd.isna(valor) or valor == '' or str(valor).strip() == '':
		return "No sisben"
	elif valor == "Sisben D":
		return "34"
	elif "LC(" in str(valor):
		# Extraer número entre paréntesis de LC(número)
		match = re.search(r'LC\((\d+)\)', str(valor))
		return match.group(1) if match else "No sisben"
	elif "SIV(" in str(valor):
		return "5"
	else:
		return "No sisben"

def extraer_nivel_sisben(valor):
	"""Extrae el nivel de sisben para casos SIV"""
	if pd.notna(valor) and "SIV(" in str(valor):
		# Extraer código entre paréntesis de SIV(código)
		match = re.search(r'SIV\(([^)]+)\)', str(valor))
		return match.group(1) if match else ""
	else:
		return ""

# Aplicar las funciones para crear las nuevas columnas
maestro_ADRES['Gr_Poblacional_Actual'] = maestro_ADRES['MARCASISBENIV+MARCASISBENIII'].apply(extraer_grupo_poblacional)
maestro_ADRES['N_Sisben_Actual'] = maestro_ADRES['MARCASISBENIV+MARCASISBENIII'].apply(extraer_nivel_sisben)

# Mostrar algunas filas para verificar el resultado
print("Verificación de las nuevas columnas:")
print(maestro_ADRES[['MARCASISBENIV+MARCASISBENIII', 'Gr_Poblacional_Actual', 'N_Sisben_Actual']].head(10))

### 4.1.1. ID Maestro ADRES 

In [ ]:
# Crear la columna ID_User en el dataframe maestro_ADRES
print("📝 CREANDO COLUMNA ID_User PARA maestro_ADRES")
print("="*60)

# Crear la columna ID_User concatenando TPS_IDN_ID + HST_IDN_NUMERO_IDENTIFICACION
maestro_ADRES['ID_User'] = maestro_ADRES['TPS_IDN_ID'].astype(str) + maestro_ADRES['HST_IDN_NUMERO_IDENTIFICACION'].astype(str)

# Verificación de resultados
print(f"✅ Columna ID_User creada exitosamente")
print(f"📊 Total de registros: {len(maestro_ADRES):,}")
print(f"📋 Ejemplos de ID_User:")
print(f"   {maestro_ADRES['ID_User'].head(10).tolist()}")

# Verificar valores únicos
ids_unicos = maestro_ADRES['ID_User'].nunique()
total_registros = len(maestro_ADRES)
print(f"\n📈 Estadísticas de ID_User:")
print(f"   IDs únicos: {ids_unicos:,}")
print(f"   Total registros: {total_registros:,}")
print(f"   IDs duplicados: {total_registros - ids_unicos:,}")

if ids_unicos < total_registros:
    print(f"⚠️ Se encontraron {total_registros - ids_unicos} IDs duplicados")
else:
    print(f"✅ Todos los IDs son únicos")

### 4.1.2. MAestro EDAD

In [ ]:
# Crear la columna EDAD en maestro_ADRES
print("📝 CREANDO COLUMNA EDAD EN maestro_ADRES")
print("="*60)

# Obtener la fecha actual
fecha_actual = datetime.now()
print(f"📅 Fecha actual: {fecha_actual.strftime('%d/%m/%Y')}")

def calcular_edad_exacta(fecha_nacimiento_str):
    """
    Calcula la edad exacta en años sin aproximaciones
    
    Args:
        fecha_nacimiento_str: Fecha en formato DD/MM/YYYY como string
    
    Returns:
        int: Edad en años completos
    """
    if pd.isna(fecha_nacimiento_str) or str(fecha_nacimiento_str).strip() == '':
        return None
    
    try:
        # Convertir string a datetime
        fecha_nacimiento = datetime.strptime(str(fecha_nacimiento_str), '%d/%m/%Y')
        
        # Calcular edad
        edad = fecha_actual.year - fecha_nacimiento.year
        
        # Verificar si ya cumplió años este año
        # Si el mes y día actual son menores al de nacimiento, restar 1 año
        if (fecha_actual.month, fecha_actual.day) < (fecha_nacimiento.month, fecha_nacimiento.day):
            edad -= 1
            
        return edad
        
    except Exception as e:
        print(f"   ⚠️ Error procesando fecha '{fecha_nacimiento_str}': {e}")
        return None

# Aplicar el cálculo de edad
print(f"📊 Procesando {len(maestro_ADRES)} registros...")

maestro_ADRES['EDAD'] = maestro_ADRES['AFL_FECHA_NACIMIENTO'].apply(calcular_edad_exacta)

# Verificación de resultados
edad_validas = maestro_ADRES['EDAD'].notna().sum()
edad_invalidas = maestro_ADRES['EDAD'].isna().sum()

print(f"\n📈 RESULTADOS:")
print(f"   Total de registros: {len(maestro_ADRES):,}")
print(f"   Edades calculadas: {edad_validas:,}")
print(f"   Edades no calculadas: {edad_invalidas:,}")
print(f"   Porcentaje exitoso: {(edad_validas/len(maestro_ADRES))*100:.2f}%")

# Estadísticas de edades
if edad_validas > 0:
    edad_min = maestro_ADRES['EDAD'].min()
    edad_max = maestro_ADRES['EDAD'].max()
    edad_promedio = maestro_ADRES['EDAD'].mean()
    
    print(f"\n📊 ESTADÍSTICAS DE EDAD:")
    print(f"   Edad mínima: {edad_min} años")
    print(f"   Edad máxima: {edad_max} años")
    print(f"   Edad promedio: {edad_promedio:.1f} años")
    
    # Mostrar ejemplos
    print(f"\n📋 EJEMPLOS:")
    ejemplos = maestro_ADRES[['AFL_FECHA_NACIMIENTO', 'EDAD']].dropna().head(10)
    for idx, row in ejemplos.iterrows():
        print(f"   Fecha nacimiento: {row['AFL_FECHA_NACIMIENTO']} → Edad: {row['EDAD']} años")

# Verificar casos problemáticos si los hay
if edad_invalidas > 0:
    print(f"\n⚠️ REGISTROS PROBLEMÁTICOS:")
    problematicos = maestro_ADRES[maestro_ADRES['EDAD'].isna()]['AFL_FECHA_NACIMIENTO'].head(5)
    print(f"   Ejemplos de fechas problemáticas: {problematicos.tolist()}")

print(f"\n✅ Columna EDAD creada exitosamente")

## 4.2. Estructura general de NS

In [ ]:
# 1. Ordenar por la columna NOVEDAD (de menor a mayor)
df_NS = df_NS.sort_values('NOVEDAD').reset_index(drop=True)

# 2. Función mejorada para formatear fechas con detección automática de formato
def format_date_column_with_diagnosis(df, column_name, target_format='%d/%m/%Y'):
    """Estandariza formato de fecha con diagnóstico completo y detección de múltiples formatos"""
    print(f"\n=== PROCESANDO COLUMNA: {column_name} ===")
    
    # DIAGNÓSTICO ANTES
    original_total = len(df)
    original_nulls = df[column_name].isna().sum()
    original_not_nulls = df[column_name].notna().sum()
    
    print(f"📊 ANTES DEL PROCESAMIENTO:")
    print(f"   Total registros: {original_total}")
    print(f"   Fechas vacías/nulas: {original_nulls}")
    print(f"   Fechas no vacías: {original_not_nulls}")
    print(f"   Porcentaje de vacías: {(original_nulls/original_total)*100:.2f}%")
    
    # Mostrar ejemplos de fechas originales
    if original_not_nulls > 0:
        print(f"   Ejemplos de fechas originales: {df[column_name].dropna().head(5).tolist()}")
    
    # Guardar valores originales para debugging
    original_values = df[column_name].copy()
    
    try:
        # ✅ ANÁLISIS DE MÚLTIPLES FORMATOS EN LOS DATOS
        sample_dates = df[column_name].dropna().astype(str).head(10).tolist()
        
        formato_iso_tiempo = any(":" in date and len(date) > 10 for date in sample_dates)
        formato_iso_simple = any("-" in date and date.count("-") == 2 and ":" not in date for date in sample_dates)
        formato_barras = any("/" in date for date in sample_dates)
        
        print(f"   🔍 FORMATOS DETECTADOS:")
        print(f"     ISO con tiempo (YYYY-MM-DD HH:MM:SS): {formato_iso_tiempo}")
        print(f"     ISO simple (YYYY-MM-DD): {formato_iso_simple}")
        print(f"     Formato con barras (DD/MM/YYYY): {formato_barras}")
        
        # ✅ PROCESAMIENTO SECUENCIAL POR FORMATO
        converted_dates = pd.Series([pd.NaT] * len(df), index=df.index)
        
        for idx, valor in df[column_name].items():
            if pd.isna(valor) or str(valor).strip() == '':
                continue
                
            valor_str = str(valor).strip()
            
            try:
                # Intentar ISO con tiempo primero
                if ":" in valor_str and len(valor_str) > 10:
                    converted_dates.loc[idx] = pd.to_datetime(valor_str)
                # Intentar ISO simple
                elif "-" in valor_str and valor_str.count("-") == 2 and ":" not in valor_str:
                    converted_dates.loc[idx] = pd.to_datetime(valor_str)
                # Intentar formato con barras (europeo)
                elif "/" in valor_str:
                    converted_dates.loc[idx] = pd.to_datetime(valor_str, dayfirst=True)
                # Último intento con detección automática
                else:
                    converted_dates.loc[idx] = pd.to_datetime(valor_str)
                    
            except Exception:
                # Si falla todo, intentar una vez más con detección automática sin dayfirst
                try:
                    converted_dates.loc[idx] = pd.to_datetime(valor_str)
                except Exception:
                    # Si sigue fallando, dejar como NaT
                    pass
        
        # Aplicar formato de salida solo a fechas válidas
        mask_validas = converted_dates.notna()
        df[column_name] = pd.Series([None] * len(df), index=df.index)
        df.loc[mask_validas, column_name] = converted_dates.loc[mask_validas].dt.strftime(target_format)
        
        # DIAGNÓSTICO DESPUÉS
        final_nulls = df[column_name].isna().sum()
        final_not_nulls = df[column_name].notna().sum()
        dates_lost = original_not_nulls - final_not_nulls
        
        print(f"📈 DESPUÉS DEL PROCESAMIENTO:")
        print(f"   Fechas vacías/nulas: {final_nulls}")
        print(f"   Fechas no vacías: {final_not_nulls}")
        print(f"   Porcentaje de vacías: {(final_nulls/original_total)*100:.2f}%")
        print(f"   Fechas perdidas en el proceso: {dates_lost}")
        
        if dates_lost > 0:
            print(f"   ⚠️  SE PERDIERON {dates_lost} FECHAS EN EL PROCESO")
            # Mostrar ejemplos de fechas perdidas
            lost_mask = converted_dates.isna() & original_values.notna()
            if lost_mask.any():
                print(f"   Ejemplos de fechas perdidas: {original_values[lost_mask].head().tolist()}")
        else:
            print(f"   ✅ NO SE PERDIERON FECHAS")
        
        # Mostrar ejemplos de fechas finales
        if final_not_nulls > 0:
            print(f"   Ejemplos de fechas finales: {df[column_name].dropna().head(5).tolist()}")
            
    except Exception as e:
        print(f"❌ Error al formatear la columna {column_name}: {e}")
    
    return df

# Validación inicial de fechas vacías
print("🔍 VALIDACIÓN INICIAL DE FECHAS VACÍAS:")
print(f"AFL_FECHA_NACIMIENTO vacías: {df_NS['AFL_FECHA_NACIMIENTO'].isna().sum()}")
print(f"FECHA_NOVEDAD vacías: {df_NS['FECHA_NOVEDAD'].isna().sum()}")

# Aplicar formateo de fechas con diagnóstico
df_NS = format_date_column_with_diagnosis(df_NS, 'AFL_FECHA_NACIMIENTO')
df_NS = format_date_column_with_diagnosis(df_NS, 'FECHA_NOVEDAD')

# 3. Estandarizar DPR_ID a 2 dígitos con ceros a la izquierda
df_NS['DPR_ID'] = df_NS['DPR_ID'].astype(str).str.zfill(2)

# 4. Estandarizar MNS_ID a 3 dígitos con ceros a la izquierda
df_NS['MNS_ID'] = df_NS['MNS_ID'].astype(str).str.zfill(3)

# VERIFICACIÓN FINAL COMPLETA
print("\n" + "="*60)
print("📋 VERIFICACIÓN FINAL DE ESTANDARIZACIÓN:")
print("="*60)
print(f"NOVEDAD ordenada - Primeros 5 valores: {df_NS['NOVEDAD'].head().tolist()}")
print(f"AFL_FECHA_NACIMIENTO - Muestra: {df_NS['AFL_FECHA_NACIMIENTO'].head().tolist()}")
print(f"FECHA_NOVEDAD - Muestra: {df_NS['FECHA_NOVEDAD'].head().tolist()}")
print(f"DPR_ID - Valores únicos: {sorted(df_NS['DPR_ID'].unique())}")
print(f"MNS_ID - Valores únicos: {sorted(df_NS['MNS_ID'].unique())}")

# RESUMEN FINAL DE FECHAS VACÍAS
print(f"\n🎯 RESUMEN FINAL DE FECHAS VACÍAS:")
print(f"AFL_FECHA_NACIMIENTO vacías: {df_NS['AFL_FECHA_NACIMIENTO'].isna().sum()}")
print(f"FECHA_NOVEDAD vacías: {df_NS['FECHA_NOVEDAD'].isna().sum()}")
print(f"Total de registros: {len(df_NS)}")

## 4.3. Ñ

In [ ]:
# Función mejorada para corregir caracteres especiales con validaciones adicionales
def corregir_caracteres_especiales(df, caracteres_problematicos=['¥', '?', 'Ã‘'], caracter_correcto='Ñ'):
    """
    Corrige caracteres especiales mal codificados en todas las columnas de texto del DataFrame
    
    Args:
        df: DataFrame a procesar
        caracteres_problematicos: Lista de caracteres que se quieren reemplazar
        caracter_correcto: Caracter por el cual se van a reemplazar
    
    Returns:
        DataFrame con caracteres corregidos
    """
    print(f"🔧 INICIANDO CORRECCIÓN DE CARACTERES ESPECIALES")
    print(f"   Caracteres a corregir: {caracteres_problematicos} → {caracter_correcto}")
    
    # Validación inicial
    if df is None or df.empty:
        print("   ⚠️ DataFrame vacío o None")
        return df
    
    # Contadores
    total_cambios = 0
    cambios_por_columna = {}
    filas_modificadas = set()  # Usar set para evitar duplicados
    
    # Obtener solo columnas de tipo object (texto)
    columnas_texto = df.select_dtypes(include=['object']).columns.tolist()
    total_filas = len(df)
    
    print(f"   DataFrame shape: {df.shape}")
    print(f"   Procesando {len(columnas_texto)} columnas de texto en {total_filas} filas...")
    print(f"   Columnas de texto: {columnas_texto[:5]}{'...' if len(columnas_texto) > 5 else ''}")
    
    # Procesar cada columna de texto
    for columna in columnas_texto:
        # Validar que la columna existe
        if columna not in df.columns:
            print(f"   ⚠️ Columna '{columna}' no encontrada, saltando...")
            continue
            
        cambios_columna = 0
        
        try:
            # Para cada caracter problemático
            for char_problema in caracteres_problematicos:
                # Obtener la serie de la columna y convertir a string
                serie_columna = df[columna].astype(str)
                
                # Identificar filas con problemas ANTES del cambio
                mask_problemas = serie_columna.str.contains(char_problema, na=False, regex=False)
                ocurrencias_antes = mask_problemas.sum()
                
                if ocurrencias_antes > 0:
                    # Agregar índices de filas modificadas al set
                    indices_modificados = df[mask_problemas].index.tolist()
                    filas_modificadas.update(indices_modificados)
                    
                    # Realizar el reemplazo
                    df[columna] = serie_columna.str.replace(char_problema, caracter_correcto, regex=False)
                    cambios_columna += ocurrencias_antes
                    
                    # Mostrar ejemplos de cambios
                    if ocurrencias_antes > 0:
                        ejemplos_antes = df.loc[indices_modificados[:3], columna].tolist()
                        print(f"     📝 {columna}: {ocurrencias_antes} ocurrencias de '{char_problema}' corregidas")
                        print(f"        Ejemplos corregidos: {ejemplos_antes}")
        
        except Exception as e:
            print(f"   ❌ Error procesando columna '{columna}': {e}")
            continue
        
        if cambios_columna > 0:
            cambios_por_columna[columna] = cambios_columna
            total_cambios += cambios_columna
    
    # Estadísticas finales
    total_filas_modificadas = len(filas_modificadas)
    porcentaje_filas_modificadas = (total_filas_modificadas / total_filas) * 100 if total_filas > 0 else 0
    
    # Resumen final
    print(f"\n📊 RESUMEN DE CORRECCIONES:")
    print(f"   Total de cambios realizados: {total_cambios}")
    print(f"   Total de filas modificadas: {total_filas_modificadas}")
    print(f"   Porcentaje de filas modificadas: {porcentaje_filas_modificadas:.2f}%")
    print(f"   Filas sin modificar: {total_filas - total_filas_modificadas}")
    
    if cambios_por_columna:
        print(f"   📈 Columnas afectadas:")
        for col, cambios in cambios_por_columna.items():
            print(f"     - {col}: {cambios} cambios")
        
        # Mostrar algunos índices de filas modificadas
        if total_filas_modificadas > 0:
            indices_muestra = sorted(list(filas_modificadas))[:10]
            print(f"   📍 Ejemplos de filas modificadas (índices): {indices_muestra}")
            if total_filas_modificadas > 10:
                print(f"      ... y {total_filas_modificadas - 10} filas más")
    else:
        print(f"   ✅ No se encontraron caracteres problemáticos")
    
    return df

# Aplicar corrección a df_NS con conteo de filas
print("="*60)
print("CORRIGIENDO CARACTERES EN df_NS")
print("="*60)
df_NS = corregir_caracteres_especiales(df_NS)

print("\n" + "="*60)
print("CORRIGIENDO CARACTERES EN maestro_ADRES")
print("="*60)
maestro_ADRES = corregir_caracteres_especiales(maestro_ADRES)

# Verificación adicional mejorada con conteo
print("\n🔍 VERIFICACIÓN ESPECÍFICA PARA PALABRAS COMUNES:")
palabras_verificar = ['NIÑO', 'NIÑA', 'AÑO', 'PEÑA', 'MONTAÑA', 'ESPAÑA']

for df_name, df in [('df_NS', df_NS), ('maestro_ADRES', maestro_ADRES)]:
    print(f"\n   📋 Verificando en {df_name} ({len(df)} filas):")
    columnas_texto = df.select_dtypes(include=['object']).columns
    
    for palabra in palabras_verificar:
        total_encontradas = 0
        filas_con_palabra = set()
        
        for col in columnas_texto:
            try:
                # CORREGIDO: agregado regex=False también en la verificación
                mask_palabra = df[col].astype(str).str.contains(palabra, case=False, na=False, regex=False)
                count = mask_palabra.sum()
                total_encontradas += count
                
                if count > 0:
                    filas_con_palabra.update(df[mask_palabra].index.tolist())
            except Exception as e:
                print(f"     ⚠️ Error verificando '{palabra}' en columna '{col}': {e}")
                continue
        
        if total_encontradas > 0:
            print(f"     ✅ '{palabra}': {total_encontradas} ocurrencias en {len(filas_con_palabra)} filas")
        else:
            print(f"     ❌ '{palabra}': No encontrada")

# Resumen global final
print(f"\n🎯 RESUMEN GLOBAL DE CORRECCIONES:")
print(f"   df_NS: {len(df_NS)} registros procesados")
print(f"   maestro_ADRES: {len(maestro_ADRES)} registros procesados")
print(f"   ✅ Proceso de corrección de caracteres completado")


## 4.4. Estado y EPS

In [ ]:
# Vaciar las columnas ENT_ID_ADRES y TPS_EST_AFL_ID_from_adres
print("🔧 VACIANDO COLUMNAS ENT_ID_ADRES Y TPS_EST_AFL_ID_from_adres")
df_NS['ENT_ID_ADRES'] = ''
df_NS['TPS_EST_AFL_ID_from_adres'] = ''

# Crear la columna ID_User (concatenación de TPS_IDN_ID + HST_IDN_NUMERO_IDENTIFICACION)
print("📝 CREANDO COLUMNA ID_User")
df_NS['ID_User'] = df_NS['TPS_IDN_ID'].astype(str) + df_NS['HST_IDN_NUMERO_IDENTIFICACION'].astype(str)

# Crear la columna ID_Register con lógica condicional
print("📝 CREANDO COLUMNA ID_Register")

# Para registros N39: TPS_IDN_ID + HST_IDN_NUMERO_IDENTIFICACION + NOVEDAD + COD_1_NOVEDAD
mask_n39 = df_NS['NOVEDAD'] == 'N39'
df_NS.loc[mask_n39, 'ID_Register'] = (
    df_NS.loc[mask_n39, 'TPS_IDN_ID'].astype(str) + 
    df_NS.loc[mask_n39, 'HST_IDN_NUMERO_IDENTIFICACION'].astype(str) + 
    df_NS.loc[mask_n39, 'NOVEDAD'].astype(str) + 
    df_NS.loc[mask_n39, 'COD_1_NOVEDAD'].astype(str)
)

# Para todos los demás registros: TPS_IDN_ID + HST_IDN_NUMERO_IDENTIFICACION + NOVEDAD
mask_no_n39 = df_NS['NOVEDAD'] != 'N39'
df_NS.loc[mask_no_n39, 'ID_Register'] = (
    df_NS.loc[mask_no_n39, 'TPS_IDN_ID'].astype(str) + 
    df_NS.loc[mask_no_n39, 'HST_IDN_NUMERO_IDENTIFICACION'].astype(str) + 
    df_NS.loc[mask_no_n39, 'NOVEDAD'].astype(str)
)

# Verificación de resultados
print("\n📊 VERIFICACIÓN DE RESULTADOS:")
print(f"Total de registros: {len(df_NS)}")
print(f"Registros N39: {mask_n39.sum()}")
print(f"Registros no N39: {mask_no_n39.sum()}")

print(f"\nColumnas vaciadas:")
print(f"ENT_ID_ADRES valores únicos: {df_NS['ENT_ID_ADRES'].unique()}")
print(f"TPS_EST_AFL_ID_from_adres valores únicos: {df_NS['TPS_EST_AFL_ID_from_adres'].unique()}")

print(f"\nEjemplos de ID_User: {df_NS['ID_User'].head().tolist()}")
print(f"Ejemplos de ID_Register (N39): {df_NS[mask_n39]['ID_Register'].head().tolist()}")
print(f"Ejemplos de ID_Register (no N39): {df_NS[mask_no_n39]['ID_Register'].head().tolist()}")

print(f"\n✅ Proceso completado exitosamente")

In [ ]:
# Realizar el cruce con maestro_ADRES para traer ENT_ID y TPS_EST_AFL_ID
print("🔄 REALIZANDO CRUCE CON maestro_ADRES")
print("="*60)

# Crear un DataFrame con solo las columnas necesarias del maestro_ADRES
maestro_subset = maestro_ADRES[['ID_User', 'ENT_ID', 'TPS_EST_AFL_ID']].copy()

# Renombrar las columnas para evitar conflictos
maestro_subset = maestro_subset.rename(columns={
    'ENT_ID': 'ENT_ID_ADRES_NEW',
    'TPS_EST_AFL_ID': 'TPS_EST_AFL_ID_from_adres_NEW'
})

print(f"📊 maestro_ADRES: {len(maestro_ADRES):,} registros")
print(f"📊 df_NS: {len(df_NS):,} registros")
print(f"📊 IDs únicos en maestro_ADRES: {maestro_subset['ID_User'].nunique():,}")
print(f"📊 IDs únicos en df_NS: {df_NS['ID_User'].nunique():,}")

# Realizar el merge LEFT para mantener todos los registros de df_NS
df_NS_merged = df_NS.merge(maestro_subset, on='ID_User', how='left')

# Actualizar las columnas originales con los nuevos valores
df_NS_merged['ENT_ID_ADRES'] = df_NS_merged['ENT_ID_ADRES_NEW']
df_NS_merged['TPS_EST_AFL_ID_from_adres'] = df_NS_merged['TPS_EST_AFL_ID_from_adres_NEW']

# Eliminar las columnas temporales
df_NS = df_NS_merged.drop(columns=['ENT_ID_ADRES_NEW', 'TPS_EST_AFL_ID_from_adres_NEW'])

print(f"✅ Cruce completado")

# ANÁLISIS DE RESULTADOS
print(f"\n📈 ANÁLISIS DE RESULTADOS DEL CRUCE:")
print("="*60)

# Calcular estadísticas generales
total_registros = len(df_NS)
print(f"📊 Total de registros en df_NS: {total_registros:,}")

# ENT_ID_ADRES
ent_id_vacios = df_NS['ENT_ID_ADRES'].isna().sum()
ent_id_llenos = df_NS['ENT_ID_ADRES'].notna().sum()
ent_id_porcentaje_vacios = (ent_id_vacios / total_registros) * 100

print(f"\n🏢 ENT_ID_ADRES:")
print(f"   Registros con datos: {ent_id_llenos:,}")
print(f"   Registros vacíos: {ent_id_vacios:,}")
print(f"   Porcentaje vacíos: {ent_id_porcentaje_vacios:.2f}%")

# TPS_EST_AFL_ID_from_adres
tps_est_vacios = df_NS['TPS_EST_AFL_ID_from_adres'].isna().sum()
tps_est_llenos = df_NS['TPS_EST_AFL_ID_from_adres'].notna().sum()
tps_est_porcentaje_vacios = (tps_est_vacios / total_registros) * 100

print(f"\n🔑 TPS_EST_AFL_ID_from_adres:")
print(f"   Registros con datos: {tps_est_llenos:,}")
print(f"   Registros vacíos: {tps_est_vacios:,}")
print(f"   Porcentaje vacíos: {tps_est_porcentaje_vacios:.2f}%")

# ANÁLISIS POR TIPO DE NOVEDAD
print(f"\n📋 ANÁLISIS POR TIPO DE NOVEDAD:")
print("="*60)

# Obtener todos los tipos de novedad
tipos_novedad = df_NS['NOVEDAD'].value_counts()
print(f"📊 Tipos de novedad encontrados: {len(tipos_novedad)}")

print(f"\n🔍 DETALLE POR NOVEDAD - ENT_ID_ADRES:")
for novedad, total in tipos_novedad.items():
    mask_novedad = df_NS['NOVEDAD'] == novedad
    vacios_novedad = df_NS.loc[mask_novedad, 'ENT_ID_ADRES'].isna().sum()
    llenos_novedad = df_NS.loc[mask_novedad, 'ENT_ID_ADRES'].notna().sum()
    porcentaje_vacios = (vacios_novedad / total) * 100
    
    print(f"   {novedad}: {total:,} registros")
    print(f"     Con datos: {llenos_novedad:,} ({((llenos_novedad/total)*100):.1f}%)")
    print(f"     Vacíos: {vacios_novedad:,} ({porcentaje_vacios:.1f}%)")

print(f"\n🔍 DETALLE POR NOVEDAD - TPS_EST_AFL_ID_from_adres:")
for novedad, total in tipos_novedad.items():
    mask_novedad = df_NS['NOVEDAD'] == novedad
    vacios_novedad = df_NS.loc[mask_novedad, 'TPS_EST_AFL_ID_from_adres'].isna().sum()
    llenos_novedad = df_NS.loc[mask_novedad, 'TPS_EST_AFL_ID_from_adres'].notna().sum()
    porcentaje_vacios = (vacios_novedad / total) * 100
    
    print(f"   {novedad}: {total:,} registros")
    print(f"     Con datos: {llenos_novedad:,} ({((llenos_novedad/total)*100):.1f}%)")
    print(f"     Vacíos: {vacios_novedad:,} ({porcentaje_vacios:.1f}%)")

# MOSTRAR EJEMPLOS DE DATOS OBTENIDOS
print(f"\n📋 EJEMPLOS DE DATOS OBTENIDOS:")
print("="*60)
ejemplos_con_datos = df_NS[df_NS['ENT_ID_ADRES'].notna()][['ID_User', 'NOVEDAD', 'ENT_ID_ADRES', 'TPS_EST_AFL_ID_from_adres']].head(10)
if len(ejemplos_con_datos) > 0:
    print("Ejemplos de registros con datos:")
    print(ejemplos_con_datos.to_string(index=False))

# IDENTIFICAR REGISTROS SIN MATCH
print(f"\n⚠️ REGISTROS SIN MATCH EN MAESTRO_ADRES:")
ids_sin_match = df_NS[df_NS['ENT_ID_ADRES'].isna()]['ID_User'].unique()
print(f"   IDs únicos sin match: {len(ids_sin_match):,}")
if len(ids_sin_match) > 0:
    print(f"   Ejemplos de IDs sin match: {ids_sin_match[:10].tolist()}")

print(f"\n✅ PROCESO DE CRUCE COMPLETADO")

In [ ]:
# Crear un nuevo DataFrame con los registros donde ENT_ID_ADRES está vacío
df_NS_vacios = df_NS[df_NS['ENT_ID_ADRES'].isna()].copy()

# Verificación
print("📊 NUEVO DATAFRAME: df_NS_vacios")
print("="*60)
print(f"Total de registros con ENT_ID_ADRES vacío: {len(df_NS_vacios):,}")
print(f"Total de columnas: {len(df_NS_vacios.columns)}")

if len(df_NS_vacios) > 0:
    print(f"\n📋 Tipos de novedad en registros vacíos:")
    print(df_NS_vacios['NOVEDAD'].value_counts())
    
    print(f"\n📋 Primeros registros:")
    print(df_NS_vacios[['NUM_SOLICITUD_NOVEDAD', 'TPS_IDN_ID', 'HST_IDN_NUMERO_IDENTIFICACION', 
                        'NOVEDAD', 'ENT_ID_ADRES']].head())
else:
    print("✅ No hay registros con ENT_ID_ADRES vacío")

# Verificar que df_NS no fue alterado
print(f"\n✅ Verificación: df_NS mantiene {len(df_NS):,} registros")

In [ ]:
del df_NS_vacios

In [ ]:
print("🔧 PROCESANDO REGISTROS VACÍOS EN ENT_ID_ADRES Y TPS_EST_AFL_ID_from_adres")
print("="*80)
print("📋 OBJETIVO: Completar datos faltantes con merge de ADRES y lógica de evoluciones")

# DIAGNÓSTICO INICIAL
registros_vacios_inicial = df_NS[
    (df_NS['ENT_ID_ADRES'].isna()) | 
    (df_NS['TPS_EST_AFL_ID_from_adres'].isna())
]
total_vacios_inicial = len(registros_vacios_inicial)

print(f"\n📊 DIAGNÓSTICO INICIAL:")
print(f"   Total registros en df_NS: {len(df_NS):,}")
print(f"   Registros con datos vacíos: {total_vacios_inicial:,}")
print(f"   Porcentaje vacíos: {(total_vacios_inicial/len(df_NS))*100:.2f}%")

if total_vacios_inicial == 0:
    print("✅ No hay registros vacíos para procesar")
else:
    # Analizar tipos de novedad en registros vacíos
    print(f"\n🔍 ANÁLISIS DE REGISTROS VACÍOS POR NOVEDAD:")
    vacios_por_novedad = registros_vacios_inicial['NOVEDAD'].value_counts()
    for novedad, count in vacios_por_novedad.items():
        print(f"   {novedad}: {count} registros")
    
    # ========== PASO 1: MERGE DIRECTO CON ADRES (LÓGICA CORRECTA) ==========
    print(f"\n🔍 PASO 1: Completar desde maestro_ADRES")
    
    # Crear DataFrame mini de ADRES con datos únicos
    df_adres_mini = (
        maestro_ADRES[[
            "TPS_IDN_ID", 
            "HST_IDN_NUMERO_IDENTIFICACION", 
            "ENT_ID", 
            "TPS_EST_AFL_ID"
        ]]
        .rename(columns={
            "ENT_ID": "ENT_ID_ADRES_from_maestro",
            "TPS_EST_AFL_ID": "TPS_EST_AFL_ID_from_maestro"
        })
        .drop_duplicates(subset=["TPS_IDN_ID", "HST_IDN_NUMERO_IDENTIFICACION"])
    )
    
    print(f"   📊 Registros únicos en maestro_ADRES: {len(df_adres_mini):,}")
    
    # Hacer merge LEFT para mantener todos los registros de df_NS
    df_NS_actualizado = df_NS.merge(
        df_adres_mini,
        on=["TPS_IDN_ID", "HST_IDN_NUMERO_IDENTIFICACION"],
        how="left"
    )
    
    # Rellenar solo donde faltaba
    df_NS_actualizado["ENT_ID_ADRES"] = df_NS_actualizado["ENT_ID_ADRES"].fillna(
        df_NS_actualizado["ENT_ID_ADRES_from_maestro"]
    )
    df_NS_actualizado["TPS_EST_AFL_ID_from_adres"] = df_NS_actualizado["TPS_EST_AFL_ID_from_adres"].fillna(
        df_NS_actualizado["TPS_EST_AFL_ID_from_maestro"]
    )
    
    # Contar actualizaciones del PASO 1
    registros_vacios_despues_paso1 = df_NS_actualizado[
        (df_NS_actualizado['ENT_ID_ADRES'].isna()) | 
        (df_NS_actualizado['TPS_EST_AFL_ID_from_adres'].isna())
    ]
    actualizados_por_adres = total_vacios_inicial - len(registros_vacios_despues_paso1)
    
    print(f"   ✅ Registros completados desde maestro_ADRES: {actualizados_por_adres:,}")
    print(f"   📊 Registros aún vacíos después del PASO 1: {len(registros_vacios_despues_paso1):,}")
    
    # Eliminar columnas temporales
    df_NS_actualizado.drop(columns=[
        "ENT_ID_ADRES_from_maestro", 
        "TPS_EST_AFL_ID_from_maestro"
    ], inplace=True)
    
    # ========== PASO 2: LÓGICA CORRECTA DE EVOLUCIONES ==========
    print(f"\n🔍 PASO 2: Completar por evoluciones (LÓGICA CORREGIDA)")
    
    # ✅ CREAR MAPA CORRECTO: N01 con datos completos → beneficiarios
    mapa_evoluciones = (
        df_NS_actualizado.loc[
            (df_NS_actualizado["NOVEDAD"] == "N01") & 
            (df_NS_actualizado["ENT_ID_ADRES"].notna()) &
            (df_NS_actualizado["TPS_EST_AFL_ID_from_adres"].notna()),
            ["COD_1_NOVEDAD", "COD_2_NOVEDAD", "ENT_ID_ADRES", "TPS_EST_AFL_ID_from_adres"]
        ]
        .dropna(subset=["COD_1_NOVEDAD", "COD_2_NOVEDAD"])  # Solo N01 con códigos válidos
        .rename(columns={
            "COD_1_NOVEDAD": "TPS_IDN_ID",  # ← CLAVE: Los códigos de N01 se vuelven IDs principales
            "COD_2_NOVEDAD": "HST_IDN_NUMERO_IDENTIFICACION",
            "ENT_ID_ADRES": "ENT_ID_from_evol",
            "TPS_EST_AFL_ID_from_adres": "TPS_EST_AFL_ID_from_evol"
        })
        .drop_duplicates(subset=["TPS_IDN_ID", "HST_IDN_NUMERO_IDENTIFICACION"])
    )
    
    print(f"   📋 Registros N01 válidos para mapeo: {len(mapa_evoluciones):,}")
    
    if len(mapa_evoluciones) > 0:
        # Hacer merge para traer datos de evoluciones
        df_NS_actualizado = df_NS_actualizado.merge(
            mapa_evoluciones,
            on=["TPS_IDN_ID", "HST_IDN_NUMERO_IDENTIFICACION"],
            how="left"
        )
        
        # ✅ APLICAR EVOLUCIONES SOLO A REGISTROS VACÍOS QUE NO SEAN N01
        mask_evol = (
            df_NS_actualizado["ENT_ID_ADRES"].isna() &
            df_NS_actualizado["NOVEDAD"].str.startswith("N0") &
            (df_NS_actualizado["NOVEDAD"] != "N01")  # Excluir N01
        )
        
        registros_completados_por_evol = mask_evol.sum()
        
        # Aplicar evoluciones
        df_NS_actualizado.loc[mask_evol, "ENT_ID_ADRES"] = df_NS_actualizado.loc[mask_evol, "ENT_ID_from_evol"]
        df_NS_actualizado.loc[mask_evol, "TPS_EST_AFL_ID_from_adres"] = df_NS_actualizado.loc[mask_evol, "TPS_EST_AFL_ID_from_evol"]
        
        print(f"   ✅ Registros completados por evoluciones: {registros_completados_por_evol:,}")
        
        # Eliminar columnas temporales
        df_NS_actualizado.drop(columns=[
            "ENT_ID_from_evol", 
            "TPS_EST_AFL_ID_from_evol"
        ], inplace=True, errors='ignore')
    else:
        print(f"   ⚠️ No hay registros N01 válidos para crear mapeo de evoluciones")
        registros_completados_por_evol = 0
    
    # ========== PASO 3: BÚSQUEDA INTERNA ADICIONAL ==========
    print(f"\n🔍 PASO 3: Búsqueda interna adicional por TPS_IDN_ID + HST_IDN_NUMERO_IDENTIFICACION")
    
    # Crear referencia interna de registros con datos completos
    df_referencia_interna = df_NS_actualizado[
        (df_NS_actualizado['ENT_ID_ADRES'].notna()) & 
        (df_NS_actualizado['TPS_EST_AFL_ID_from_adres'].notna())
    ][['TPS_IDN_ID', 'HST_IDN_NUMERO_IDENTIFICACION', 'ENT_ID_ADRES', 'TPS_EST_AFL_ID_from_adres']].drop_duplicates()
    
    print(f"   📊 Registros de referencia interna: {len(df_referencia_interna):,}")
    
    # Identificar registros aún vacíos
    registros_aun_vacios = df_NS_actualizado[
        (df_NS_actualizado['ENT_ID_ADRES'].isna()) | 
        (df_NS_actualizado['TPS_EST_AFL_ID_from_adres'].isna())
    ]
    
    actualizados_busqueda_interna = 0
    
    if len(registros_aun_vacios) > 0 and len(df_referencia_interna) > 0:
        # Hacer merge para buscar coincidencias internas
        merge_interno = registros_aun_vacios.merge(
            df_referencia_interna,
            on=['TPS_IDN_ID', 'HST_IDN_NUMERO_IDENTIFICACION'],
            how='left',
            suffixes=('', '_interno')
        )
        
        # Aplicar valores encontrados
        mask_encontrados = merge_interno['ENT_ID_ADRES_interno'].notna()
        indices_encontrados = merge_interno[mask_encontrados].index
        
        df_NS_actualizado.loc[indices_encontrados, 'ENT_ID_ADRES'] = merge_interno.loc[indices_encontrados, 'ENT_ID_ADRES_interno']
        df_NS_actualizado.loc[indices_encontrados, 'TPS_EST_AFL_ID_from_adres'] = merge_interno.loc[indices_encontrados, 'TPS_EST_AFL_ID_from_adres_interno']
        
        actualizados_busqueda_interna = mask_encontrados.sum()
    
    print(f"   ✅ Registros completados por búsqueda interna: {actualizados_busqueda_interna:,}")
    
    # Actualizar el DataFrame principal
    df_NS = df_NS_actualizado.copy()
    
    # ========== DIAGNÓSTICO FINAL ==========
    print(f"\n📈 DIAGNÓSTICO FINAL:")
    registros_vacios_final = df_NS[
        (df_NS['ENT_ID_ADRES'].isna()) | 
        (df_NS['TPS_EST_AFL_ID_from_adres'].isna())
    ]
    total_vacios_final = len(registros_vacios_final)
    total_completados = actualizados_por_adres + registros_completados_por_evol + actualizados_busqueda_interna
    
    print(f"   Registros vacíos antes: {total_vacios_inicial:,}")
    print(f"   Registros vacíos después: {total_vacios_final:,}")
    print(f"   Total registros completados: {total_completados:,}")
    print(f"   Mejora: {total_vacios_inicial - total_vacios_final:,} registros completados")
    print(f"   Porcentaje de mejora: {((total_vacios_inicial - total_vacios_final)/total_vacios_inicial)*100:.2f}%" if total_vacios_inicial > 0 else "0.00%")
    
    # RESUMEN POR MÉTODO
    print(f"\n📋 RESUMEN POR MÉTODO:")
    print(f"   Por merge con maestro_ADRES: {actualizados_por_adres:,}")
    print(f"   Por evoluciones (N01 → N02,N03...): {registros_completados_por_evol:,}")
    print(f"   Por búsqueda interna adicional: {actualizados_busqueda_interna:,}")
    print(f"   Total completados: {total_completados:,}")
    
    # ANÁLISIS DE REGISTROS QUE AÚN QUEDAN VACÍOS
    if total_vacios_final > 0:
        print(f"\n⚠️ REGISTROS QUE AÚN QUEDAN VACÍOS: {total_vacios_final:,}")
        print(f"   📊 Por tipo de novedad:")
        vacios_finales_por_novedad = registros_vacios_final['NOVEDAD'].value_counts()
        for novedad, count in vacios_finales_por_novedad.items():
            print(f"     {novedad}: {count} registros")
        
        # Mostrar ejemplos de registros que no se pudieron completar
        print(f"\n   📋 Ejemplos de registros no completados:")
        ejemplos_vacios = registros_vacios_final[['NUM_SOLICITUD_NOVEDAD', 'TPS_IDN_ID', 'HST_IDN_NUMERO_IDENTIFICACION', 
                                                 'NOVEDAD', 'COD_1_NOVEDAD', 'COD_2_NOVEDAD']].head(5)
        print(ejemplos_vacios.to_string(index=False))
    else:
        print(f"\n🎉 ¡ÉXITO! Todos los registros fueron completados")
    
    # VERIFICACIÓN DE INTEGRIDAD
    print(f"\n🔍 VERIFICACIÓN DE INTEGRIDAD:")
    ent_id_vacios_final = df_NS['ENT_ID_ADRES'].isna().sum()
    tps_est_vacios_final = df_NS['TPS_EST_AFL_ID_from_adres'].isna().sum()
    
    print(f"   ENT_ID_ADRES vacíos: {ent_id_vacios_final:,}")
    print(f"   TPS_EST_AFL_ID_from_adres vacíos: {tps_est_vacios_final:,}")
    print(f"   Consistencia: {'✅' if ent_id_vacios_final == tps_est_vacios_final else '❌'}")

print(f"\n✅ PROCESO DE EVOLUCIONES COMPLETADO")

## 4.5. validación inical regimen y estado

In [ ]:
# Crear DataFrame para registros que no cumplen criterios de envío
print("🔍 CREANDO DATAFRAME DF_No_Envio - FILTROS DE VALIDACIÓN")
print("="*80)

# Inicializar DF_No_Envio como DataFrame vacío con la misma estructura que df_NS más columna 'motivo'
DF_No_Envio = pd.DataFrame(columns=list(df_NS.columns) + ['motivo'])

# Contador de registros movidos
registros_movidos = 0
conteo_por_motivo = {}

# ========== VALIDACIÓN 1: ENT_ID_ADRES vacío o nulo ==========
print("🔍 VALIDACIÓN 1: ENT_ID_ADRES vacío o nulo")

# Identificar registros con ENT_ID_ADRES vacío o nulo
mask_vacios = df_NS['ENT_ID_ADRES'].isna() | (df_NS['ENT_ID_ADRES'] == '') | (df_NS['ENT_ID_ADRES'] == 'nan')
registros_vacios = df_NS[mask_vacios].copy()
count_vacios = len(registros_vacios)

if count_vacios > 0:
    # Agregar motivo
    registros_vacios['motivo'] = 'Afiliado no pertenece a la EPS'
    
    # Agregar a DF_No_Envio
    DF_No_Envio = pd.concat([DF_No_Envio, registros_vacios], ignore_index=True)
    
    # Eliminar de df_NS
    df_NS = df_NS[~mask_vacios].reset_index(drop=True)
    
    registros_movidos += count_vacios
    conteo_por_motivo['Afiliado no pertenece a la EPS'] = count_vacios

print(f"   📊 Registros con ENT_ID_ADRES vacío: {count_vacios}")
print(f"   ✅ Movidos a DF_No_Envio: {count_vacios}")

# ========== VALIDACIÓN 2: ENT_ID_ADRES igual a "EPSC25" ==========
print(f"\n🔍 VALIDACIÓN 2: ENT_ID_ADRES igual a 'EPSC25'")

# Identificar registros con ENT_ID_ADRES igual a "EPSC25"
mask_epsc25 = df_NS['ENT_ID_ADRES'] == 'EPSC25'
registros_epsc25 = df_NS[mask_epsc25].copy()
count_epsc25 = len(registros_epsc25)

if count_epsc25 > 0:
    # Agregar motivo
    registros_epsc25['motivo'] = 'Afiliado del regimen contributivo'
    
    # Agregar a DF_No_Envio
    DF_No_Envio = pd.concat([DF_No_Envio, registros_epsc25], ignore_index=True)
    
    # Eliminar de df_NS
    df_NS = df_NS[~mask_epsc25].reset_index(drop=True)
    
    registros_movidos += count_epsc25
    conteo_por_motivo['Afiliado del regimen contributivo'] = count_epsc25

print(f"   📊 Registros con ENT_ID_ADRES = 'EPSC25': {count_epsc25}")
print(f"   ✅ Movidos a DF_No_Envio: {count_epsc25}")

# ========== RESUMEN FINAL ==========
print(f"\n📈 RESUMEN DE FILTROS APLICADOS:")
print("="*60)
print(f"📊 Total de registros movidos: {registros_movidos}")
print(f"📊 Registros restantes en df_NS: {len(df_NS)}")
print(f"📊 Registros en DF_No_Envio: {len(DF_No_Envio)}")

print(f"\n📋 DETALLE POR MOTIVO:")
for motivo, cantidad in conteo_por_motivo.items():
    porcentaje = (cantidad / registros_movidos * 100) if registros_movidos > 0 else 0
    print(f"   • {motivo}: {cantidad} registros ({porcentaje:.2f}%)")

# Verificar integridad
print(f"\n🔍 VERIFICACIÓN DE INTEGRIDAD:")
print(f"   ENT_ID_ADRES únicos en df_NS: {df_NS['ENT_ID_ADRES'].unique()}")
print(f"   Motivos únicos en DF_No_Envio: {DF_No_Envio['motivo'].unique() if len(DF_No_Envio) > 0 else 'N/A'}")

# Mostrar ejemplos si hay registros
if len(DF_No_Envio) > 0:
    print(f"\n📋 EJEMPLOS DE REGISTROS EN DF_No_Envio:")
    ejemplos = DF_No_Envio[['NUM_SOLICITUD_NOVEDAD', 'HST_IDN_NUMERO_IDENTIFICACION', 
                           'NOVEDAD', 'ENT_ID_ADRES', 'motivo']].head(10)
    print(ejemplos.to_string(index=False))
else:
    print(f"\n✅ No hay registros en DF_No_Envio")

print(f"\n✅ PROCESO DE FILTRADO COMPLETADO")

# 5. Procesar novedades
## 5.1. N01 Actualización o corrección del tipo y número de identificación del afiliado y/o fecha de nacimiento

In [ ]:
import pandas as pd
import numpy as np

def validar_novedades_N01(df):
    """
    Valida, corrige y filtra los registros con NOVEDAD = 'N01',
    incluyendo la validación de no-cambio.
    """
    # Inicialización de contadores y DataFrame de No Envío
    registros_iniciales_df = len(df)
    DF_No_Envio = pd.DataFrame()
    total_movidos = 0
    
    print("🔍 INICIANDO VALIDACIÓN DE NOVEDADES N01")
    print("="*60)
    
    # 1. Filtrar solo registros N01
    mask_n01 = df['NOVEDAD'] == 'N01'
    registros_n01 = df[mask_n01].copy()
    total_n01_inicial = len(registros_n01)
    
    print(f"📊 Total de registros N01 iniciales: {total_n01_inicial}")
    
    if total_n01_inicial == 0:
        print("⚠️ No se encontraron registros N01")
        # Devolver el DF original y un DF_No_Envio vacío
        return df, DF_No_Envio
    
    # =========================================================================
    # PASO CLAVE 1: PRE-PROCESAMIENTO Y CONVERSIÓN DE COD_3_NOVEDAD
    # (Necesario para tener un formato estándar ANTES de la validación de igualdad)
    # =========================================================================

    def convertir_fecha_cod3(valor):
        """Convierte diferentes formatos de fecha a DD/MM/YYYY"""
        if pd.isna(valor) or str(valor).strip() == '':
            return None
        
        valor_str = str(valor).strip()
        
        try:
            # Si es un número (formato Excel serializado)
            if valor_str.isdigit() and len(valor_str) >= 5:
                # Convertir número Excel a fecha
                fecha_excel = pd.to_datetime('1900-01-01') + pd.Timedelta(days=int(valor_str)-2)
                return fecha_excel.strftime('%d/%m/%Y')
            
            # Si ya tiene formato de fecha
            elif '/' in valor_str or '-' in valor_str:
                # Intentar convertir con diferentes formatos
                fecha_convertida = pd.to_datetime(valor_str, dayfirst=True, errors='coerce')
                if pd.notna(fecha_convertida):
                    return fecha_convertida.strftime('%d/%m/%Y')
                else:
                    return None
            else:
                return None
                
        except Exception as e:
            return None
    
    # Aplicar conversión de fechas
    registros_n01['COD_3_CONVERTIDA'] = registros_n01['COD_3_NOVEDAD'].apply(convertir_fecha_cod3)
    
    # =========================================================================
    # PASO CLAVE 2: NUEVA VALIDACIÓN DE NO-CAMBIO
    # =========================================================================
    print("\n🔍 VALIDANDO REGISTROS SIN CAMBIO DE DATOS")
    
    # Estandarizar columnas para una comparación justa (limpiar espacios y mayúsculas/minúsculas)
    registros_n01['COD_1_NOVEDAD_STR'] = registros_n01['COD_1_NOVEDAD'].astype(str).str.strip().str.upper()
    registros_n01['TPS_IDN_ID_STR'] = registros_n01['TPS_IDN_ID'].astype(str).str.strip().str.upper()
    
    registros_n01['COD_2_NOVEDAD_STR'] = registros_n01['COD_2_NOVEDAD'].astype(str).str.strip().str.upper()
    registros_n01['HST_IDN_NUMERO_IDENTIFICACION_STR'] = registros_n01['HST_IDN_NUMERO_IDENTIFICACION'].astype(str).str.strip().str.upper()

    # Formatear AFL_FECHA_NACIMIENTO al formato DD/MM/YYYY para comparar con la columna convertida
    registros_n01['AFL_FECHA_NACIMIENTO_STR'] = pd.to_datetime(
        registros_n01['AFL_FECHA_NACIMIENTO'], errors='coerce'
    ).dt.strftime('%d/%m/%Y')

    # Crear una máscara donde todos los campos son iguales
    mask_iguales = (
        (registros_n01['COD_1_NOVEDAD_STR'] == registros_n01['TPS_IDN_ID_STR']) &
        (registros_n01['COD_2_NOVEDAD_STR'] == registros_n01['HST_IDN_NUMERO_IDENTIFICACION_STR']) &
        (registros_n01['COD_3_CONVERTIDA'].fillna('') == registros_n01['AFL_FECHA_NACIMIENTO_STR'].fillna(''))
    )
    
    # Registros a mover
    registros_a_mover = registros_n01[mask_iguales].copy()
    total_movidos = len(registros_a_mover)
    
    print(f"   Registros sin cambio de datos (a mover): {total_movidos}")
    
    if total_movidos > 0:
        # Mover al DF_No_Envio y agregar motivo
        registros_a_mover['motivo'] = "Datos a reportar iguales a los actuales"
        DF_No_Envio = pd.concat([DF_No_Envio, registros_a_mover.drop(columns=[
            'COD_3_CONVERTIDA', 'COD_1_NOVEDAD_STR', 'TPS_IDN_ID_STR',
            'COD_2_NOVEDAD_STR', 'HST_IDN_NUMERO_IDENTIFICACION_STR', 
            'AFL_FECHA_NACIMIENTO_STR'
        ])], ignore_index=True)
        
        # Eliminar del lote de N01
        registros_n01 = registros_n01[~mask_iguales].copy()
        
        # Actualizar la máscara N01 en el DF principal para reflejar la eliminación
        indices_a_eliminar = df.loc[mask_n01][mask_iguales].index
        df = df.drop(indices_a_eliminar)
        
        # Actualizar la máscara después de la eliminación
        mask_n01 = df['NOVEDAD'] == 'N01'
    
    registros_n01_actuales = len(registros_n01)
    print(f"   Registros N01 restantes para validación: {registros_n01_actuales}")
    
    # Si no quedan registros N01 después del filtro de no-cambio
    if registros_n01_actuales == 0:
        print("⚠️ Todos los registros N01 fueron movidos por no-cambio.")
        # Devolver el DF original (reducido) y el DF_No_Envio
        return df, DF_No_Envio
    
    # =========================================================================
    # PASO 3: VALIDACIONES ORIGINALES (en el subconjunto restante)
    # =========================================================================
    
    # ========== VALIDAR COD_2_NOVEDAD (debe ser entero) ==========
    print(f"\n🔧 VALIDANDO COD_2_NOVEDAD (debe ser entero)")
    
    def es_entero_valido(valor):
        # Mantenemos esta función para la validación de formato
        if pd.isna(valor) or str(valor).strip() == '':
            return False
        try:
            int(str(valor).strip())
            return True
        except:
            return False
    
    # Aplicar validación
    registros_n01['COD_2_VALIDO'] = registros_n01['COD_2_NOVEDAD'].apply(es_entero_valido)
    invalidos_cod2 = (~registros_n01['COD_2_VALIDO']).sum()
    print(f"   Valores inválidos encontrados: {invalidos_cod2}")
    
    # ========== VALIDAR COD_3_NOVEDAD (Fecha) ==========
    print(f"\n🔧 VALIDANDO COD_3_NOVEDAD (fecha DD/MM/YYYY)")
    # (Ya se aplicó la conversión, solo se reporta)
    fechas_validas = registros_n01['COD_3_CONVERTIDA'].notna().sum()
    fechas_invalidas = registros_n01['COD_3_CONVERTIDA'].isna().sum()
    print(f"   Fechas válidas después de conversión: {fechas_validas}")
    print(f"   Fechas que no pudieron convertirse: {fechas_invalidas}")
    
    # ========== VALIDAR COD_4_NOVEDAD (no puede estar vacío) ==========
    print(f"\n🔧 VALIDANDO COD_4_NOVEDAD (no puede estar vacío)")
    vacios_cod4_antes = registros_n01['COD_4_NOVEDAD'].isna().sum()
    print(f"   Valores vacíos: {vacios_cod4_antes}")
    
    # =========================================================================
    # PASO 4: APLICAR CORRECCIONES Y RESUMEN FINAL
    # =========================================================================
    print(f"\n🔄 APLICANDO CORRECCIONES AL DATAFRAME PRINCIPAL")
    
    # Aplicar corrección de fechas (solo a los registros que quedan en el DF principal)
    df.loc[mask_n01, 'COD_3_NOVEDAD'] = registros_n01['COD_3_CONVERTIDA']
    
    # Revalidar después de correcciones (en el DF principal ya reducido)
    registros_n01_final = df[mask_n01].copy()

    print(f"\n📋 RESUMEN FINAL DE VALIDACIÓN N01:")
    
    # Mostrar registros problemáticos finales (registros N01 que quedan en df)
    problematicos = registros_n01_final[
        registros_n01_final['COD_2_NOVEDAD'].isna() | 
        (~registros_n01['COD_2_VALIDO']) | # Usar el resultado de validación
        registros_n01_final['COD_3_NOVEDAD'].isna() | 
        registros_n01_final['COD_4_NOVEDAD'].isna()
    ]
    
    if len(problematicos) > 0:
        print(f"\n⚠️ REGISTROS PROBLEMÁTICOS (PENDIENTES) ENCONTRADOS: {len(problematicos)}")
    else:
        print(f"   ✅ Todos los registros N01 restantes son válidos")
    
    
    # =========================================================================
    # PASO 5: RESUMEN DE CONTEOS
    # =========================================================================
    
    registros_finales_df = len(df)
    registros_finales_n01_df = len(df[df['NOVEDAD'] == 'N01'])
    registros_finales_no_envio = len(DF_No_Envio)
    
    print("\n\n=== RESUMEN DE MOVIMIENTO DE REGISTROS ===")
    print(f"  Total registros en DF original (df): {registros_iniciales_df}")
    print(f"  Total registros con NOVEDAD='N01' iniciales: {total_n01_inicial}")
    print("-" * 40)
    print(f"  ➡️ Registros movidos a DF_No_Envio (por No-Cambio): {total_movidos}")
    print("-" * 40)
    print(f"  Total registros N01 restantes en df_NS: {registros_finales_n01_df}")
    print(f"  Total registros en DF final (df): {registros_finales_df}")
    print(f"  Total registros en DF_No_Envio: {registros_finales_no_envio}")
    
    # Devolver ambos DataFrames
    return df, DF_No_Envio


df_NS, DF_No_Envio = validar_novedades_N01(df_NS)

## 5.2. N02 Actualización o corrección de nombres del afiliado.

In [ ]:
def validar_novedades_N02(df, df_no_envio):
    """
    Valida registros con NOVEDAD = 'N02' verificando que los nombres actuales
    sean diferentes a los nombres que se quieren reportar.
    Si son iguales, los mueve a DF_No_Envio.
    """
    print("🔍 INICIANDO VALIDACIÓN DE NOVEDADES N02")
    print("="*60)
    
    # Filtrar solo registros N02
    mask_n02 = df['NOVEDAD'] == 'N02'
    registros_n02 = df[mask_n02].copy()
    total_n02_inicial = len(registros_n02)
    
    print(f"📊 Total de registros N02 iniciales: {total_n02_inicial}")
    
    if total_n02_inicial == 0:
        print("⚠️ No se encontraron registros N02")
        return df, df_no_envio
    
    # Estandarizar nombres para comparación (quitar espacios y convertir a mayúsculas)
    def normalizar_nombre(nombre):
        if pd.isna(nombre) or str(nombre).strip() == '':
            return ''
        return str(nombre).strip().upper()
    
    # Normalizar nombres actuales
    registros_n02['AFL_PRIMER_NOMBRE_NORM'] = registros_n02['AFL_PRIMER_NOMBRE'].apply(normalizar_nombre)
    registros_n02['AFL_SEGUNDO_NOMBRE_NORM'] = registros_n02['AFL_SEGUNDO_NOMBRE'].apply(normalizar_nombre)
    
    # Normalizar nombres nuevos (códigos de novedad)
    registros_n02['COD_1_NOVEDAD_NORM'] = registros_n02['COD_1_NOVEDAD'].apply(normalizar_nombre)
    registros_n02['COD_2_NOVEDAD_NORM'] = registros_n02['COD_2_NOVEDAD'].apply(normalizar_nombre)
    
    # Crear máscara para identificar registros donde los nombres son iguales
    mask_nombres_iguales = (
        (registros_n02['AFL_PRIMER_NOMBRE_NORM'] == registros_n02['COD_1_NOVEDAD_NORM']) &
        (registros_n02['AFL_SEGUNDO_NOMBRE_NORM'] == registros_n02['COD_2_NOVEDAD_NORM'])
    )
    
    # Registros a mover (sin cambios)
    registros_sin_cambios = registros_n02[mask_nombres_iguales].copy()
    total_sin_cambios = len(registros_sin_cambios)
    
    print(f"📋 ANÁLISIS DE NOMBRES:")
    print(f"   Registros con nombres actuales = nombres nuevos: {total_sin_cambios}")
    print(f"   Registros con cambios válidos: {total_n02_inicial - total_sin_cambios}")
    
    if total_sin_cambios > 0:
        # Mostrar ejemplos de registros sin cambios
        print(f"\n📋 EJEMPLOS DE REGISTROS SIN CAMBIOS:")
        ejemplos = registros_sin_cambios[['NUM_SOLICITUD_NOVEDAD', 'HST_IDN_NUMERO_IDENTIFICACION', 
                                        'AFL_PRIMER_NOMBRE', 'AFL_SEGUNDO_NOMBRE', 
                                        'COD_1_NOVEDAD', 'COD_2_NOVEDAD']].head(5)
        print(ejemplos.to_string(index=False))
        
        # Agregar motivo y mover a DF_No_Envio
        registros_sin_cambios_clean = registros_sin_cambios.drop(columns=[
            'AFL_PRIMER_NOMBRE_NORM', 'AFL_SEGUNDO_NOMBRE_NORM',
            'COD_1_NOVEDAD_NORM', 'COD_2_NOVEDAD_NORM'
        ])
        registros_sin_cambios_clean['motivo'] = "Afiliado sin cambios a reportar"
        
        # Concatenar con DF_No_Envio existente
        df_no_envio = pd.concat([df_no_envio, registros_sin_cambios_clean], ignore_index=True)
        
        # Obtener índices originales para eliminar del DataFrame principal
        indices_a_eliminar = df.loc[mask_n02][mask_nombres_iguales].index
        df = df.drop(indices_a_eliminar).reset_index(drop=True)
        
        print(f"✅ Movidos {total_sin_cambios} registros a DF_No_Envio")
    else:
        print(f"✅ Todos los registros N02 tienen cambios válidos")
    
    # Estadísticas finales
    registros_n02_final = len(df[df['NOVEDAD'] == 'N02'])
    
    print(f"\n📈 RESUMEN FINAL:")
    print(f"   Registros N02 iniciales: {total_n02_inicial}")
    print(f"   Registros movidos (sin cambios): {total_sin_cambios}")
    print(f"   Registros N02 restantes: {registros_n02_final}")
    print(f"   Total en DF_No_Envio actual: {len(df_no_envio)}")
    
    return df, df_no_envio

# Aplicar validación N02
df_NS, DF_No_Envio = validar_novedades_N02(df_NS, DF_No_Envio)

## 5.3. N03 Actualización o corrección de apellidos del afiliado.

In [ ]:
def validar_novedades_N03(df, df_no_envio):
    """
    Valida registros con NOVEDAD = 'N03' verificando que los apellidos actuales
    sean diferentes a los apellidos que se quieren reportar.
    Si son iguales, los mueve a DF_No_Envio.
    """
    print("🔍 INICIANDO VALIDACIÓN DE NOVEDADES N03")
    print("="*60)
    
    # Filtrar solo registros N03
    mask_n03 = df['NOVEDAD'] == 'N03'
    registros_n03 = df[mask_n03].copy()
    total_n03_inicial = len(registros_n03)
    
    print(f"📊 Total de registros N03 iniciales: {total_n03_inicial}")
    
    if total_n03_inicial == 0:
        print("⚠️ No se encontraron registros N03")
        return df, df_no_envio
    
    # Estandarizar apellidos para comparación (quitar espacios y convertir a mayúsculas)
    def normalizar_apellido(apellido):
        if pd.isna(apellido) or str(apellido).strip() == '':
            return ''
        return str(apellido).strip().upper()
    
    # Normalizar apellidos actuales
    registros_n03['AFL_PRIMER_APELLIDO_NORM'] = registros_n03['AFL_PRIMER_APELLIDO'].apply(normalizar_apellido)
    registros_n03['AFL_SEGUNDO_APELLIDO_NORM'] = registros_n03['AFL_SEGUNDO_APELLIDO'].apply(normalizar_apellido)
    
    # Normalizar apellidos nuevos (códigos de novedad)
    registros_n03['COD_1_NOVEDAD_NORM'] = registros_n03['COD_1_NOVEDAD'].apply(normalizar_apellido)
    registros_n03['COD_2_NOVEDAD_NORM'] = registros_n03['COD_2_NOVEDAD'].apply(normalizar_apellido)
    
    # Crear máscara para identificar registros donde los apellidos son iguales
    mask_apellidos_iguales = (
        (registros_n03['AFL_PRIMER_APELLIDO_NORM'] == registros_n03['COD_1_NOVEDAD_NORM']) &
        (registros_n03['AFL_SEGUNDO_APELLIDO_NORM'] == registros_n03['COD_2_NOVEDAD_NORM'])
    )
    
    # Registros a mover (sin cambios)
    registros_sin_cambios = registros_n03[mask_apellidos_iguales].copy()
    total_sin_cambios = len(registros_sin_cambios)
    
    print(f"📋 ANÁLISIS DE APELLIDOS:")
    print(f"   Registros con apellidos actuales = apellidos nuevos: {total_sin_cambios}")
    print(f"   Registros con cambios válidos: {total_n03_inicial - total_sin_cambios}")
    
    if total_sin_cambios > 0:
        # Mostrar ejemplos de registros sin cambios
        print(f"\n📋 EJEMPLOS DE REGISTROS SIN CAMBIOS:")
        ejemplos = registros_sin_cambios[['NUM_SOLICITUD_NOVEDAD', 'HST_IDN_NUMERO_IDENTIFICACION', 
                                        'AFL_PRIMER_APELLIDO', 'AFL_SEGUNDO_APELLIDO', 
                                        'COD_1_NOVEDAD', 'COD_2_NOVEDAD']].head(5)
        print(ejemplos.to_string(index=False))
        
        # Agregar motivo y mover a DF_No_Envio
        registros_sin_cambios_clean = registros_sin_cambios.drop(columns=[
            'AFL_PRIMER_APELLIDO_NORM', 'AFL_SEGUNDO_APELLIDO_NORM',
            'COD_1_NOVEDAD_NORM', 'COD_2_NOVEDAD_NORM'
        ])
        registros_sin_cambios_clean['motivo'] = "Afiliado sin cambios a reportar"
        
        # Concatenar con DF_No_Envio existente
        df_no_envio = pd.concat([df_no_envio, registros_sin_cambios_clean], ignore_index=True)
        
        # Obtener índices originales para eliminar del DataFrame principal
        indices_a_eliminar = df.loc[mask_n03][mask_apellidos_iguales].index
        df = df.drop(indices_a_eliminar).reset_index(drop=True)
        
        print(f"✅ Movidos {total_sin_cambios} registros a DF_No_Envio")
    else:
        print(f"✅ Todos los registros N03 tienen cambios válidos")
    
    # Estadísticas finales
    registros_n03_final = len(df[df['NOVEDAD'] == 'N03'])
    
    print(f"\n📈 RESUMEN FINAL:")
    print(f"   Registros N03 iniciales: {total_n03_inicial}")
    print(f"   Registros movidos (sin cambios): {total_sin_cambios}")
    print(f"   Registros N03 restantes: {registros_n03_final}")
    print(f"   Total en DF_No_Envio actual: {len(df_no_envio)}")
    
    return df, df_no_envio

# Aplicar validación N03
df_NS, DF_No_Envio = validar_novedades_N03(df_NS, DF_No_Envio)

## 5.4. N04 Actualización o cambio de departamento y municipio de afiliación en la misma Entidad de Salud

In [ ]:
def validar_novedades_N04(df, df_no_envio, maestro_adres):
    """
    Valida registros con NOVEDAD = 'N04' (cambio de departamento y municipio).
    Proceso:
    1. Actualizar DPR_ID y MNC_ID desde maestro_ADRES
    2. Validar formato de códigos (DPR_ID: 2 dígitos, MNC_ID: 3 dígitos)
    3. Verificar que hay cambios reales en departamento/municipio
    """
    print("🔍 INICIANDO VALIDACIÓN DE NOVEDADES N04")
    print("="*60)
    
    # Filtrar solo registros N04
    mask_n04 = df['NOVEDAD'] == 'N04'
    registros_n04 = df[mask_n04].copy()
    total_n04_inicial = len(registros_n04)
    
    print(f"📊 Total de registros N04 iniciales: {total_n04_inicial}")
    
    if total_n04_inicial == 0:
        print("⚠️ No se encontraron registros N04")
        return df, df_no_envio
    
    # =========================================================================
    # PASO 1: VERIFICAR Y CORREGIR NOMBRES DE COLUMNAS
    # =========================================================================
    print(f"\n🔍 PASO 1: Verificando estructura de columnas")
    
    # Verificar qué columnas de municipio existen en df_NS
    columna_municipio_df = None
    if 'MNS_ID' in df.columns:
        columna_municipio_df = 'MNS_ID'
        print(f"   📊 df_NS usa: MNS_ID")
    elif 'MNC_ID' in df.columns:
        columna_municipio_df = 'MNC_ID'
        print(f"   📊 df_NS usa: MNC_ID")
    else:
        print(f"   ❌ No se encontró columna de municipio en df_NS")
        return df, df_no_envio
    
    # Verificar qué columnas existen en maestro_adres
    print(f"   📊 maestro_ADRES tiene: DPR_ID={('DPR_ID' in maestro_adres.columns)}, MNC_ID={('MNC_ID' in maestro_adres.columns)}")
    
    # =========================================================================
    # PASO 2: ACTUALIZAR DESDE MAESTRO_ADRES CON LÓGICA CORREGIDA
    # =========================================================================
    print(f"\n🔄 PASO 2: Actualizando ubicación desde maestro_ADRES")
    
    # Crear subset del maestro con nombres únicos y renombrados para evitar conflictos
    maestro_ubicacion = maestro_adres[['ID_User', 'DPR_ID', 'MNC_ID']].drop_duplicates(subset=['ID_User']).rename(columns={
        'DPR_ID': 'DPR_ID_from_maestro',
        'MNC_ID': 'MNC_ID_from_maestro'
    })
    
    print(f"   📊 Registros únicos en maestro_ADRES: {len(maestro_ubicacion):,}")
    
    # Hacer merge para traer datos del maestro
    registros_n04_con_maestro = registros_n04.merge(
        maestro_ubicacion,
        on='ID_User',
        how='left'
    )
    
    # ✅ ACTUALIZAR COLUMNAS CON DATOS DEL MAESTRO
    print(f"   🔄 Actualizando columnas desde maestro...")
    
    # Actualizar DPR_ID
    registros_n04_con_maestro['DPR_ID'] = registros_n04_con_maestro['DPR_ID_from_maestro'].fillna(
        registros_n04_con_maestro['DPR_ID']
    )
    
    # Actualizar columna de municipio (usar el nombre correcto según df_NS)
    if columna_municipio_df == 'MNS_ID':
        # df_NS usa MNS_ID, maestro_ADRES usa MNC_ID
        registros_n04_con_maestro['MNS_ID'] = registros_n04_con_maestro['MNC_ID_from_maestro'].fillna(
            registros_n04_con_maestro['MNS_ID']
        )
        columna_municipio_actual = 'MNS_ID'
    else:
        # df_NS usa MNC_ID
        registros_n04_con_maestro['MNC_ID'] = registros_n04_con_maestro['MNC_ID_from_maestro'].fillna(
            registros_n04_con_maestro['MNC_ID']
        )
        columna_municipio_actual = 'MNC_ID'
    
    # Limpiar columnas temporales
    registros_n04_con_maestro = registros_n04_con_maestro.drop(columns=[
        'DPR_ID_from_maestro', 'MNC_ID_from_maestro'
    ])
    
    # Contar actualizaciones
    actualizaciones_dpr = registros_n04_con_maestro['DPR_ID'].notna().sum()
    actualizaciones_municipio = registros_n04_con_maestro[columna_municipio_actual].notna().sum()
    print(f"   ✅ Registros con DPR_ID válido: {actualizaciones_dpr}")
    print(f"   ✅ Registros con {columna_municipio_actual} válido: {actualizaciones_municipio}")
    
    # =========================================================================
    # PASO 3: VALIDAR Y ESTANDARIZAR FORMATOS
    # =========================================================================
    print(f"\n🔧 PASO 3: Validando y estandarizando formatos de códigos")
    
    def validar_y_formatear_departamento(codigo):
        if pd.isna(codigo) or str(codigo).strip() == '':
            return None
        try:
            codigo_num = int(str(codigo).strip())
            if 1 <= codigo_num <= 99:
                return f"{codigo_num:02d}"
            else:
                return None
        except:
            return None
    
    def validar_y_formatear_municipio(codigo):
        if pd.isna(codigo) or str(codigo).strip() == '':
            return None
        try:
            codigo_num = int(str(codigo).strip())
            if 1 <= codigo_num <= 999:
                return f"{codigo_num:03d}"
            else:
                return None
        except:
            return None
    
    # Aplicar formateo a códigos actuales
    registros_n04_con_maestro['DPR_ID_FORMATTED'] = registros_n04_con_maestro['DPR_ID'].apply(validar_y_formatear_departamento)
    registros_n04_con_maestro['MUN_ID_FORMATTED'] = registros_n04_con_maestro[columna_municipio_actual].apply(validar_y_formatear_municipio)
    
    # Aplicar formateo a códigos nuevos
    registros_n04_con_maestro['COD_1_NOVEDAD_FORMATTED'] = registros_n04_con_maestro['COD_1_NOVEDAD'].apply(validar_y_formatear_departamento)
    registros_n04_con_maestro['COD_2_NOVEDAD_FORMATTED'] = registros_n04_con_maestro['COD_2_NOVEDAD'].apply(validar_y_formatear_municipio)
    
    # Estadísticas de validación
    dpr_actuales_validos = registros_n04_con_maestro['DPR_ID_FORMATTED'].notna().sum()
    mun_actuales_validos = registros_n04_con_maestro['MUN_ID_FORMATTED'].notna().sum()
    dpr_nuevos_validos = registros_n04_con_maestro['COD_1_NOVEDAD_FORMATTED'].notna().sum()
    mun_nuevos_validos = registros_n04_con_maestro['COD_2_NOVEDAD_FORMATTED'].notna().sum()
    
    print(f"   📊 Códigos actuales válidos:")
    print(f"     DPR_ID: {dpr_actuales_validos}/{total_n04_inicial}")
    print(f"     {columna_municipio_actual}: {mun_actuales_validos}/{total_n04_inicial}")
    print(f"   📊 Códigos nuevos válidos:")
    print(f"     COD_1_NOVEDAD (Dpto): {dpr_nuevos_validos}/{total_n04_inicial}")
    print(f"     COD_2_NOVEDAD (Mpio): {mun_nuevos_validos}/{total_n04_inicial}")
    
    # =========================================================================
    # PASO 4: IDENTIFICAR REGISTROS SIN CAMBIOS
    # =========================================================================
    print(f"\n🔍 PASO 4: Identificando registros sin cambios de ubicación")
    
    # Crear máscara para registros donde no hay cambios
    mask_sin_cambios = (
        (registros_n04_con_maestro['DPR_ID_FORMATTED'] == registros_n04_con_maestro['COD_1_NOVEDAD_FORMATTED']) &
        (registros_n04_con_maestro['MUN_ID_FORMATTED'] == registros_n04_con_maestro['COD_2_NOVEDAD_FORMATTED']) &
        registros_n04_con_maestro['DPR_ID_FORMATTED'].notna() &
        registros_n04_con_maestro['MUN_ID_FORMATTED'].notna() &
        registros_n04_con_maestro['COD_1_NOVEDAD_FORMATTED'].notna() &
        registros_n04_con_maestro['COD_2_NOVEDAD_FORMATTED'].notna()
    )
    
    registros_sin_cambios = registros_n04_con_maestro[mask_sin_cambios].copy()
    total_sin_cambios = len(registros_sin_cambios)
    
    print(f"   📋 Registros sin cambios de ubicación: {total_sin_cambios}")
    print(f"   📋 Registros con cambios válidos: {total_n04_inicial - total_sin_cambios}")
    
    if total_sin_cambios > 0:
        # Mostrar ejemplos
        print(f"\n📋 EJEMPLOS DE REGISTROS SIN CAMBIOS:")
        ejemplos = registros_sin_cambios[[
            'NUM_SOLICITUD_NOVEDAD', 'HST_IDN_NUMERO_IDENTIFICACION', 
            'DPR_ID_FORMATTED', 'MUN_ID_FORMATTED', 
            'COD_1_NOVEDAD_FORMATTED', 'COD_2_NOVEDAD_FORMATTED'
        ]].head(5)
        print(ejemplos.to_string(index=False))
        
        # Limpiar y mover a DF_No_Envio
        registros_sin_cambios_clean = registros_sin_cambios.drop(columns=[
            'DPR_ID_FORMATTED', 'MUN_ID_FORMATTED',
            'COD_1_NOVEDAD_FORMATTED', 'COD_2_NOVEDAD_FORMATTED'
        ])
        registros_sin_cambios_clean['motivo'] = "Afiliado sin cambios a reportar"
        
        # Concatenar con DF_No_Envio
        df_no_envio = pd.concat([df_no_envio, registros_sin_cambios_clean], ignore_index=True)
        
        # Eliminar del DataFrame principal
        indices_a_eliminar = registros_sin_cambios.index
        df = df.drop(indices_a_eliminar).reset_index(drop=True)
        
        print(f"✅ Movidos {total_sin_cambios} registros a DF_No_Envio")
    
    # =========================================================================
    # PASO 5: ACTUALIZAR DF PRINCIPAL CON DATOS CORREGIDOS
    # =========================================================================
    print(f"\n🔄 PASO 5: Actualizando DataFrame principal")
    
    # Filtrar registros restantes
    registros_restantes = registros_n04_con_maestro[~mask_sin_cambios]
    
    if len(registros_restantes) > 0:
        # Actualizar df_NS principal con los datos corregidos
        for idx, row in registros_restantes.iterrows():
            # Encontrar el índice correspondiente en df_NS
            mask_update = (df['ID_User'] == row['ID_User']) & (df['NOVEDAD'] == 'N04')
            if mask_update.any():
                idx_df = df[mask_update].index[0]
                
                # Actualizar con valores formateados o originales
                df.loc[idx_df, 'DPR_ID'] = row['DPR_ID_FORMATTED'] if pd.notna(row['DPR_ID_FORMATTED']) else row['DPR_ID']
                df.loc[idx_df, columna_municipio_actual] = row['MUN_ID_FORMATTED'] if pd.notna(row['MUN_ID_FORMATTED']) else row[columna_municipio_actual]
                df.loc[idx_df, 'COD_1_NOVEDAD'] = row['COD_1_NOVEDAD_FORMATTED'] if pd.notna(row['COD_1_NOVEDAD_FORMATTED']) else row['COD_1_NOVEDAD']
                df.loc[idx_df, 'COD_2_NOVEDAD'] = row['COD_2_NOVEDAD_FORMATTED'] if pd.notna(row['COD_2_NOVEDAD_FORMATTED']) else row['COD_2_NOVEDAD']
        
        print(f"   ✅ Actualizados {len(registros_restantes)} registros en df_NS")
    
    # =========================================================================
    # PASO 6: ESTADÍSTICAS FINALES
    # =========================================================================
    registros_n04_final = len(df[df['NOVEDAD'] == 'N04'])
    
    print(f"\n📈 RESUMEN FINAL:")
    print(f"   Registros N04 iniciales: {total_n04_inicial}")
    print(f"   Registros movidos (sin cambios): {total_sin_cambios}")
    print(f"   Registros N04 restantes: {registros_n04_final}")
    print(f"   Total en DF_No_Envio actual: {len(df_no_envio)}")
    
    # Mostrar ejemplos finales
    if registros_n04_final > 0:
        print(f"\n📋 EJEMPLOS DE REGISTROS N04 VÁLIDOS RESTANTES:")
        ejemplos_finales = df[df['NOVEDAD'] == 'N04'][[
            'NUM_SOLICITUD_NOVEDAD', 'HST_IDN_NUMERO_IDENTIFICACION',
            'DPR_ID', columna_municipio_actual, 'COD_1_NOVEDAD', 'COD_2_NOVEDAD'
        ]].head(5)
        print(ejemplos_finales.to_string(index=False))
    
    return df, df_no_envio

# Aplicar validación N04
df_NS, DF_No_Envio = validar_novedades_N04(df_NS, DF_No_Envio, maestro_ADRES)

## 5.5 N09  Retiro por muerte. 

In [ ]:
def validar_novedades_N09(df, df_no_envio):
    """
    Valida registros con NOVEDAD = 'N09' (Retiro por muerte).
    Si TPS_EST_AFL_ID_from_adres es 'AF' (ya fallecido en ADRES),
    los mueve a DF_No_Envio porque no hay cambio a reportar.
    """
    print("🔍 INICIANDO VALIDACIÓN DE NOVEDADES N09")
    print("="*60)
    
    # Filtrar solo registros N09
    mask_n09 = df['NOVEDAD'] == 'N09'
    registros_n09 = df[mask_n09].copy()
    total_n09_inicial = len(registros_n09)
    
    print(f"📊 Total de registros N09 iniciales: {total_n09_inicial}")
    
    if total_n09_inicial == 0:
        print("⚠️ No se encontraron registros N09")
        return df, df_no_envio
    
    # Identificar registros que ya están como fallecidos (AF) en ADRES
    mask_ya_fallecido = registros_n09['TPS_EST_AFL_ID_from_adres'] == 'AF'
    registros_ya_fallecidos = registros_n09[mask_ya_fallecido].copy()
    total_ya_fallecidos = len(registros_ya_fallecidos)
    
    print(f"📋 ANÁLISIS DE ESTADO EN ADRES:")
    print(f"   Registros ya marcados como fallecidos (AF) en ADRES: {total_ya_fallecidos}")
    print(f"   Registros con cambio válido (no AF): {total_n09_inicial - total_ya_fallecidos}")
    
    if total_ya_fallecidos > 0:
        # Mostrar ejemplos de registros ya fallecidos
        print(f"\n📋 EJEMPLOS DE REGISTROS YA FALLECIDOS EN ADRES:")
        ejemplos = registros_ya_fallecidos[['NUM_SOLICITUD_NOVEDAD', 'HST_IDN_NUMERO_IDENTIFICACION', 
                                           'TPS_EST_AFL_ID_from_adres', 'FECHA_NOVEDAD']].head(5)
        print(ejemplos.to_string(index=False))
        
        # Agregar motivo y mover a DF_No_Envio
        registros_ya_fallecidos['motivo'] = "Afiliado sin cambios a reportar"
        
        # Concatenar con DF_No_Envio existente
        df_no_envio = pd.concat([df_no_envio, registros_ya_fallecidos], ignore_index=True)
        
        # Obtener índices originales para eliminar del DataFrame principal
        indices_a_eliminar = df.loc[mask_n09][mask_ya_fallecido].index
        df = df.drop(indices_a_eliminar).reset_index(drop=True)
        
        print(f"✅ Movidos {total_ya_fallecidos} registros a DF_No_Envio")
    else:
        print(f"✅ Todos los registros N09 tienen cambios válidos para reportar")
    
    # Estadísticas finales
    registros_n09_final = len(df[df['NOVEDAD'] == 'N09'])
    
    print(f"\n📈 RESUMEN FINAL:")
    print(f"   Registros N09 iniciales: {total_n09_inicial}")
    print(f"   Registros movidos (ya fallecidos en ADRES): {total_ya_fallecidos}")
    print(f"   Registros N09 restantes (válidos): {registros_n09_final}")
    print(f"   Total en DF_No_Envio actual: {len(df_no_envio)}")
    
    # Mostrar distribución de estados en registros restantes si hay
    if registros_n09_final > 0:
        print(f"\n📋 ESTADOS EN REGISTROS N09 RESTANTES:")
        estados_restantes = df[df['NOVEDAD'] == 'N09']['TPS_EST_AFL_ID_from_adres'].value_counts()
        for estado, count in estados_restantes.items():
            print(f"   {estado}: {count} registros")
    
    return df, df_no_envio

# Aplicar validación N09
df_NS, DF_No_Envio = validar_novedades_N09(df_NS, DF_No_Envio)

## 5.6. N12 Actualización de condición de discapacidad

In [ ]:
def validar_novedades_N12(df, df_no_envio, maestro_adres):
    """
    Valida registros con NOVEDAD = 'N12' (Actualización de condición de discapacidad).
    Proceso:
    1. Validar que COD_1_NOVEDAD sea igual a 'D'
    2. Verificar si el usuario ya está reportado con discapacidad en ADRES (TPS_CND_BNF_ID = 'D')
    """
    print("🔍 INICIANDO VALIDACIÓN DE NOVEDADES N12")
    print("="*60)
    
    # Filtrar solo registros N12
    mask_n12 = df['NOVEDAD'] == 'N12'
    registros_n12 = df[mask_n12].copy()
    total_n12_inicial = len(registros_n12)
    
    print(f"📊 Total de registros N12 iniciales: {total_n12_inicial}")
    
    if total_n12_inicial == 0:
        print("⚠️ No se encontraron registros N12")
        return df, df_no_envio
    
    # =========================================================================
    # VALIDACIÓN 1: COD_1_NOVEDAD debe ser igual a 'D'
    # =========================================================================
    print(f"\n🔍 VALIDACIÓN 1: COD_1_NOVEDAD debe ser igual a 'D'")
    
    # Normalizar valores (quitar espacios y convertir a mayúsculas)
    registros_n12['COD_1_NOVEDAD_NORM'] = registros_n12['COD_1_NOVEDAD'].astype(str).str.strip().str.upper()
    
    # Identificar registros con COD_1_NOVEDAD diferente de 'D'
    mask_cod_invalido = registros_n12['COD_1_NOVEDAD_NORM'] != 'D'
    registros_cod_invalido = registros_n12[mask_cod_invalido].copy()
    total_cod_invalido = len(registros_cod_invalido)
    
    print(f"   Registros con COD_1_NOVEDAD válido ('D'): {total_n12_inicial - total_cod_invalido}")
    print(f"   Registros con COD_1_NOVEDAD inválido: {total_cod_invalido}")
    
    if total_cod_invalido > 0:
        # Mostrar ejemplos
        print(f"\n📋 EJEMPLOS DE REGISTROS CON COD_1_NOVEDAD INVÁLIDO:")
        ejemplos_invalidos = registros_cod_invalido[['NUM_SOLICITUD_NOVEDAD', 'HST_IDN_NUMERO_IDENTIFICACION', 
                                                     'COD_1_NOVEDAD', 'COD_1_NOVEDAD_NORM']].head(5)
        print(ejemplos_invalidos.to_string(index=False))
        
        # Limpiar y mover a DF_No_Envio
        registros_cod_invalido_clean = registros_cod_invalido.drop(columns=['COD_1_NOVEDAD_NORM'])
        registros_cod_invalido_clean['motivo'] = "Condición de discapacidad no válida"
        
        # Concatenar con DF_No_Envio
        df_no_envio = pd.concat([df_no_envio, registros_cod_invalido_clean], ignore_index=True)
        
        # Eliminar del DataFrame principal
        indices_a_eliminar = df.loc[mask_n12][mask_cod_invalido].index
        df = df.drop(indices_a_eliminar).reset_index(drop=True)
        
        # Actualizar registros_n12 para la siguiente validación
        registros_n12 = registros_n12[~mask_cod_invalido].copy()
        
        print(f"✅ Movidos {total_cod_invalido} registros a DF_No_Envio por COD_1_NOVEDAD inválido")
    else:
        print(f"✅ Todos los registros tienen COD_1_NOVEDAD = 'D'")
    
    # =========================================================================
    # VALIDACIÓN 2: Verificar si ya está reportado con discapacidad en ADRES
    # =========================================================================
    registros_n12_restantes = len(registros_n12)
    
    if registros_n12_restantes == 0:
        print(f"\n⚠️ No quedan registros N12 para validar")
        return df, df_no_envio
    
    print(f"\n🔍 VALIDACIÓN 2: Verificar si ya está reportado con discapacidad en ADRES")
    print(f"   Registros a validar: {registros_n12_restantes}")
    
    # Crear subset del maestro con TPS_CND_BNF_ID
    maestro_discapacidad = maestro_adres[['ID_User', 'TPS_CND_BNF_ID']].drop_duplicates(subset=['ID_User'])
    
    # Normalizar TPS_CND_BNF_ID en el maestro
    maestro_discapacidad['TPS_CND_BNF_ID_NORM'] = maestro_discapacidad['TPS_CND_BNF_ID'].astype(str).str.strip().str.upper()
    
    # Hacer merge para traer TPS_CND_BNF_ID del maestro
    registros_n12_con_maestro = registros_n12.merge(
        maestro_discapacidad[['ID_User', 'TPS_CND_BNF_ID_NORM']],
        on='ID_User',
        how='left'
    )
    
    # Identificar registros que ya tienen discapacidad reportada en ADRES
    mask_ya_con_discapacidad = registros_n12_con_maestro['TPS_CND_BNF_ID_NORM'] == 'D'
    registros_ya_con_discapacidad = registros_n12_con_maestro[mask_ya_con_discapacidad].copy()
    total_ya_con_discapacidad = len(registros_ya_con_discapacidad)
    
    print(f"   Registros ya con discapacidad en ADRES: {total_ya_con_discapacidad}")
    print(f"   Registros con cambio válido: {registros_n12_restantes - total_ya_con_discapacidad}")
    
    if total_ya_con_discapacidad > 0:
        # Mostrar ejemplos
        print(f"\n📋 EJEMPLOS DE REGISTROS YA CON DISCAPACIDAD EN ADRES:")
        ejemplos_ya_discapacidad = registros_ya_con_discapacidad[['NUM_SOLICITUD_NOVEDAD', 'HST_IDN_NUMERO_IDENTIFICACION', 
                                                                  'COD_1_NOVEDAD_NORM', 'TPS_CND_BNF_ID_NORM']].head(5)
        print(ejemplos_ya_discapacidad.to_string(index=False))
        
        # Limpiar y mover a DF_No_Envio
        registros_ya_con_discapacidad_clean = registros_ya_con_discapacidad.drop(columns=[
            'COD_1_NOVEDAD_NORM', 'TPS_CND_BNF_ID_NORM'
        ])
        registros_ya_con_discapacidad_clean['motivo'] = "Afiliado sin cambios a reportar"
        
        # Concatenar con DF_No_Envio
        df_no_envio = pd.concat([df_no_envio, registros_ya_con_discapacidad_clean], ignore_index=True)
        
        # Obtener índices originales para eliminar del DataFrame principal
        # Necesitamos recalcular la máscara en el df original
        mask_n12_original = df['NOVEDAD'] == 'N12'
        registros_n12_original = df[mask_n12_original].copy()
        
        # Merge con el maestro en el df original
        registros_n12_original_con_maestro = registros_n12_original.merge(
            maestro_discapacidad[['ID_User', 'TPS_CND_BNF_ID_NORM']],
            on='ID_User',
            how='left'
        )
        registros_n12_original_con_maestro['COD_1_NOVEDAD_NORM'] = registros_n12_original_con_maestro['COD_1_NOVEDAD'].astype(str).str.strip().str.upper()
        
        # Identificar índices a eliminar
        mask_eliminar_original = registros_n12_original_con_maestro['TPS_CND_BNF_ID_NORM'] == 'D'
        indices_a_eliminar = df.loc[mask_n12_original][mask_eliminar_original].index
        
        df = df.drop(indices_a_eliminar).reset_index(drop=True)
        
        print(f"✅ Movidos {total_ya_con_discapacidad} registros a DF_No_Envio por ya tener discapacidad en ADRES")
    else:
        print(f"✅ Todos los registros N12 son cambios válidos para reportar")
    
    # =========================================================================
    # RESUMEN FINAL
    # =========================================================================
    registros_n12_final = len(df[df['NOVEDAD'] == 'N12'])
    total_movidos = total_cod_invalido + total_ya_con_discapacidad
    
    print(f"\n📈 RESUMEN FINAL:")
    print(f"   Registros N12 iniciales: {total_n12_inicial}")
    print(f"   Registros movidos por COD_1_NOVEDAD inválido: {total_cod_invalido}")
    print(f"   Registros movidos por ya tener discapacidad en ADRES: {total_ya_con_discapacidad}")
    print(f"   Total movidos: {total_movidos}")
    print(f"   Registros N12 restantes (válidos): {registros_n12_final}")
    print(f"   Total en DF_No_Envio actual: {len(df_no_envio)}")
    
    # Mostrar ejemplos de registros válidos restantes si hay
    if registros_n12_final > 0:
        print(f"\n📋 EJEMPLOS DE REGISTROS N12 VÁLIDOS RESTANTES:")
        ejemplos_validos = df[df['NOVEDAD'] == 'N12'][['NUM_SOLICITUD_NOVEDAD', 'HST_IDN_NUMERO_IDENTIFICACION', 
                                                       'COD_1_NOVEDAD']].head(5)
        print(ejemplos_validos.to_string(index=False))
    
    return df, df_no_envio

# Aplicar validación N12
df_NS, DF_No_Envio = validar_novedades_N12(df_NS, DF_No_Envio, maestro_ADRES)

## 5.7. N14 Actualización o cambio de estado de afiliación

In [ ]:
def validar_novedades_N14(df, df_no_envio):
    """
    Valida registros con NOVEDAD = 'N14' (Actualización o cambio de estado de afiliación).
    
    Proceso:
    1. Validar y corregir formato de COD_2_NOVEDAD según COD_1_NOVEDAD:
       - Si COD_1_NOVEDAD es 'RE' o 'SM': COD_2_NOVEDAD debe ser numérico (1-9), sino asignar '7'
       - Si COD_1_NOVEDAD es 'SD': COD_2_NOVEDAD debe estar vacío
    2. Verificar cambios reales: si COD_1_NOVEDAD == TPS_EST_AFL_ID_from_adres, mover a DF_No_Envio
    """
    print("🔍 INICIANDO VALIDACIÓN DE NOVEDADES N14")
    print("="*60)
    
    # Filtrar solo registros N14
    mask_n14 = df['NOVEDAD'] == 'N14'
    registros_n14 = df[mask_n14].copy()
    total_n14_inicial = len(registros_n14)
    
    print(f"📊 Total de registros N14 iniciales: {total_n14_inicial}")
    
    if total_n14_inicial == 0:
        print("⚠️ No se encontraron registros N14")
        return df, df_no_envio
    
    # =========================================================================
    # PASO 1: VALIDAR Y CORREGIR FORMATO DE COD_2_NOVEDAD
    # =========================================================================
    print(f"\n🔧 PASO 1: Validando y corrigiendo formato de COD_2_NOVEDAD")
    
    # Normalizar COD_1_NOVEDAD (quitar espacios y mayúsculas)
    registros_n14['COD_1_NOVEDAD_NORM'] = registros_n14['COD_1_NOVEDAD'].astype(str).str.strip().str.upper()
    
    # Mostrar distribución de valores en COD_1_NOVEDAD
    print(f"\n📊 Distribución de COD_1_NOVEDAD:")
    distribucion_cod1 = registros_n14['COD_1_NOVEDAD_NORM'].value_counts()
    for valor, count in distribucion_cod1.items():
        print(f"   {valor}: {count} registros")
    
    # Función para validar si es número del 1 al 9
    def es_numero_valido_1_a_9(valor):
        if pd.isna(valor) or str(valor).strip() == '':
            return False
        try:
            num = int(str(valor).strip())
            return 1 <= num <= 9
        except:
            return False
    
    # Contadores de correcciones
    correcciones_re_sm = 0
    correcciones_sd = 0
    
    # PROCESAR REGISTROS RE Y SM
    mask_re_sm = registros_n14['COD_1_NOVEDAD_NORM'].isin(['RE', 'SM'])
    registros_re_sm = registros_n14[mask_re_sm].copy()
    
    print(f"\n🔍 Procesando registros RE y SM ({len(registros_re_sm)} registros):")
    
    if len(registros_re_sm) > 0:
        # Identificar registros sin valor válido en COD_2_NOVEDAD
        registros_re_sm['COD_2_VALIDO'] = registros_re_sm['COD_2_NOVEDAD'].apply(es_numero_valido_1_a_9)
        invalidos_re_sm = (~registros_re_sm['COD_2_VALIDO']).sum()
        
        print(f"   Registros con COD_2_NOVEDAD válido (1-9): {registros_re_sm['COD_2_VALIDO'].sum()}")
        print(f"   Registros sin COD_2_NOVEDAD válido: {invalidos_re_sm}")
        
        if invalidos_re_sm > 0:
            # Mostrar ejemplos antes de corregir
            print(f"\n   📋 Ejemplos de valores inválidos a corregir:")
            ejemplos_invalidos = registros_re_sm[~registros_re_sm['COD_2_VALIDO']][
                ['NUM_SOLICITUD_NOVEDAD', 'HST_IDN_NUMERO_IDENTIFICACION', 
                 'COD_1_NOVEDAD_NORM', 'COD_2_NOVEDAD']
            ].head(5)
            print(ejemplos_invalidos.to_string(index=False))
            
            # Asignar '7' por defecto a los inválidos
            mask_corregir_re_sm = mask_re_sm & (~registros_re_sm['COD_2_VALIDO'])
            df.loc[df[mask_n14][mask_corregir_re_sm].index, 'COD_2_NOVEDAD'] = '7'
            correcciones_re_sm = invalidos_re_sm
            
            print(f"   ✅ Asignado '7' por defecto a {correcciones_re_sm} registros")
    
    # PROCESAR REGISTROS SD
    mask_sd = registros_n14['COD_1_NOVEDAD_NORM'] == 'SD'
    registros_sd = registros_n14[mask_sd].copy()
    
    print(f"\n🔍 Procesando registros SD ({len(registros_sd)} registros):")
    
    if len(registros_sd) > 0:
        # Identificar registros SD con valor en COD_2_NOVEDAD
        registros_sd['COD_2_NO_VACIO'] = registros_sd['COD_2_NOVEDAD'].notna() & (registros_sd['COD_2_NOVEDAD'].astype(str).str.strip() != '')
        con_valor_sd = registros_sd['COD_2_NO_VACIO'].sum()
        
        print(f"   Registros SD con COD_2_NOVEDAD vacío (correcto): {len(registros_sd) - con_valor_sd}")
        print(f"   Registros SD con COD_2_NOVEDAD con valor (a limpiar): {con_valor_sd}")
        
        if con_valor_sd > 0:
            # Mostrar ejemplos antes de limpiar
            print(f"\n   📋 Ejemplos de valores a limpiar:")
            ejemplos_sd = registros_sd[registros_sd['COD_2_NO_VACIO']][
                ['NUM_SOLICITUD_NOVEDAD', 'HST_IDN_NUMERO_IDENTIFICACION', 
                 'COD_1_NOVEDAD_NORM', 'COD_2_NOVEDAD']
            ].head(5)
            print(ejemplos_sd.to_string(index=False))
            
            # Limpiar COD_2_NOVEDAD para registros SD
            mask_limpiar_sd = mask_sd & registros_sd['COD_2_NO_VACIO']
            df.loc[df[mask_n14][mask_limpiar_sd].index, 'COD_2_NOVEDAD'] = ''
            correcciones_sd = con_valor_sd
            
            print(f"   ✅ Limpiado COD_2_NOVEDAD en {correcciones_sd} registros SD")
    
    # Resumen de correcciones de formato
    total_correcciones_formato = correcciones_re_sm + correcciones_sd
    print(f"\n📈 RESUMEN DE CORRECCIONES DE FORMATO:")
    print(f"   Registros RE/SM corregidos (asignado '7'): {correcciones_re_sm}")
    print(f"   Registros SD limpiados: {correcciones_sd}")
    print(f"   Total correcciones de formato: {total_correcciones_formato}")
    
    # =========================================================================
    # PASO 2: IDENTIFICAR REGISTROS SIN CAMBIOS
    # =========================================================================
    print(f"\n🔍 PASO 2: Identificando registros sin cambios de estado")
    
    # Recargar registros N14 con las correcciones aplicadas
    registros_n14_actualizado = df[mask_n14].copy()
    
    # Normalizar TPS_EST_AFL_ID_from_adres para comparación
    registros_n14_actualizado['TPS_EST_AFL_ID_NORM'] = registros_n14_actualizado['TPS_EST_AFL_ID_from_adres'].astype(str).str.strip().str.upper()
    registros_n14_actualizado['COD_1_NOVEDAD_NORM'] = registros_n14_actualizado['COD_1_NOVEDAD'].astype(str).str.strip().str.upper()
    
    # Crear máscara para registros sin cambios
    mask_sin_cambios = registros_n14_actualizado['COD_1_NOVEDAD_NORM'] == registros_n14_actualizado['TPS_EST_AFL_ID_NORM']
    registros_sin_cambios = registros_n14_actualizado[mask_sin_cambios].copy()
    total_sin_cambios = len(registros_sin_cambios)
    
    print(f"   Registros con COD_1_NOVEDAD = TPS_EST_AFL_ID_from_adres: {total_sin_cambios}")
    print(f"   Registros con cambios válidos: {total_n14_inicial - total_sin_cambios}")
    
    if total_sin_cambios > 0:
        # Mostrar ejemplos
        print(f"\n📋 EJEMPLOS DE REGISTROS SIN CAMBIOS:")
        ejemplos_sin_cambios = registros_sin_cambios[[
            'NUM_SOLICITUD_NOVEDAD', 'HST_IDN_NUMERO_IDENTIFICACION',
            'COD_1_NOVEDAD_NORM', 'TPS_EST_AFL_ID_NORM'
        ]].head(5)
        print(ejemplos_sin_cambios.to_string(index=False))
        
        # Limpiar y mover a DF_No_Envio
        registros_sin_cambios_clean = registros_sin_cambios.drop(columns=[
            'TPS_EST_AFL_ID_NORM', 'COD_1_NOVEDAD_NORM'
        ])
        registros_sin_cambios_clean['motivo'] = "Afiliado sin cambios a reportar"
        
        # Concatenar con DF_No_Envio
        df_no_envio = pd.concat([df_no_envio, registros_sin_cambios_clean], ignore_index=True)
        
        # Eliminar del DataFrame principal
        indices_a_eliminar = df.loc[mask_n14][mask_sin_cambios].index
        df = df.drop(indices_a_eliminar).reset_index(drop=True)
        
        print(f"✅ Movidos {total_sin_cambios} registros a DF_No_Envio")
    else:
        print(f"✅ Todos los registros N14 tienen cambios válidos para reportar")
    
    # =========================================================================
    # PASO 3: ESTADÍSTICAS FINALES
    # =========================================================================
    registros_n14_final = len(df[df['NOVEDAD'] == 'N14'])
    
    print(f"\n📈 RESUMEN FINAL:")
    print(f"   Registros N14 iniciales: {total_n14_inicial}")
    print(f"   Correcciones de formato aplicadas: {total_correcciones_formato}")
    print(f"   Registros movidos (sin cambios): {total_sin_cambios}")
    print(f"   Registros N14 restantes (válidos): {registros_n14_final}")
    print(f"   Total en DF_No_Envio actual: {len(df_no_envio)}")
    
    # Mostrar ejemplos de registros válidos restantes si hay
    if registros_n14_final > 0:
        print(f"\n📋 EJEMPLOS DE REGISTROS N14 VÁLIDOS RESTANTES:")
        ejemplos_validos = df[df['NOVEDAD'] == 'N14'][[
            'NUM_SOLICITUD_NOVEDAD', 'HST_IDN_NUMERO_IDENTIFICACION',
            'COD_1_NOVEDAD', 'COD_2_NOVEDAD', 'TPS_EST_AFL_ID_from_adres'
        ]].head(5)
        print(ejemplos_validos.to_string(index=False))
        
        # Mostrar distribución de estados en registros válidos
        print(f"\n📊 Distribución de estados en registros válidos:")
        distribucion_estados = df[df['NOVEDAD'] == 'N14']['COD_1_NOVEDAD'].value_counts()
        for estado, count in distribucion_estados.items():
            print(f"   {estado}: {count} registros")
    
    return df, df_no_envio

# Aplicar validación N14
df_NS, DF_No_Envio = validar_novedades_N14(df_NS, DF_No_Envio)

### 5.7.1. N14 Masiva

In [ ]:
def aplicar_retiros_masivos_N14(df_ns, maestro_adres, fecha_reporte):
    """
    Aplica retiros masivos N14 (SD - Suspensión por Documento) para 3 casos específicos:
    1. Afiliados con PE activos
    2. CN activos con edad >= 1 año
    3. Identificaciones alfanuméricas activas
    
    Args:
        df_ns: DataFrame de novedades
        maestro_adres: DataFrame maestro ADRES
        fecha_reporte: fecha del reporte (datetime)
    
    Returns:
        df_ns actualizado con las nuevas novedades N14
    """
    print("🔍 INICIANDO APLICACIÓN DE RETIROS MASIVOS N14 (SD)")
    print("="*80)
    
    # Convertir fecha_reporte a formato DD/MM/YYYY
    fecha_novedad_str = fecha_reporte.strftime('%d/%m/%Y')
    print(f"📅 Fecha de novedad: {fecha_novedad_str}")
    
    # =========================================================================
    # CASO 1: PE ACTIVOS
    # =========================================================================
    print(f"\n🔍 CASO 1: Afiliados con PE activos")
    
    mask_pe = (
        (maestro_adres['ENT_ID'] == 'EPS025') &
        (maestro_adres['TPS_IDN_ID'] == 'PE') &
        (maestro_adres['TPS_EST_AFL_ID'] == 'AC')
    )
    
    registros_pe = maestro_adres[mask_pe].copy().reset_index(drop=True)  # ✅ RESET INDEX
    total_pe = len(registros_pe)
    
    print(f"   📊 Registros PE activos encontrados: {total_pe}")
    
    # =========================================================================
    # CASO 2: CN ACTIVOS CON EDAD >= 1
    # =========================================================================
    print(f"\n🔍 CASO 2: CN activos con edad >= 1 año")
    
    mask_cn = (
        (maestro_adres['ENT_ID'] == 'EPS025') &
        (maestro_adres['TPS_IDN_ID'] == 'CN') &
        (maestro_adres['TPS_EST_AFL_ID'] == 'AC') &
        (maestro_adres['EDAD'] >= 1)
    )
    
    registros_cn = maestro_adres[mask_cn].copy().reset_index(drop=True)  # ✅ RESET INDEX
    total_cn = len(registros_cn)
    
    print(f"   📊 Registros CN >= 1 año encontrados: {total_cn}")
    
    # =========================================================================
    # CASO 3: IDENTIFICACIONES ALFANUMÉRICAS ACTIVAS
    # =========================================================================
    print(f"\n🔍 CASO 3: Identificaciones alfanuméricas activas")
    
    def es_alfanumerico(valor):
        """Verifica si un valor contiene letras y números"""
        if pd.isna(valor) or str(valor).strip() == '':
            return False
        
        valor_str = str(valor).strip()
        tiene_letras = any(c.isalpha() for c in valor_str)
        tiene_numeros = any(c.isdigit() for c in valor_str)
        
        return tiene_letras and tiene_numeros
    
    # Aplicar validación alfanumérica
    maestro_adres_temp = maestro_adres.copy()
    maestro_adres_temp['ES_ALFANUMERICO'] = maestro_adres_temp['HST_IDN_NUMERO_IDENTIFICACION'].apply(es_alfanumerico)
    
    mask_alfanum = (
        (maestro_adres_temp['ENT_ID'] == 'EPS025') &
        (maestro_adres_temp['ES_ALFANUMERICO'] == True) &
        (maestro_adres_temp['TPS_EST_AFL_ID'] == 'AC')
    )
    
    registros_alfanum = maestro_adres_temp[mask_alfanum].copy().reset_index(drop=True)  # ✅ RESET INDEX
    # Eliminar columna temporal
    registros_alfanum = registros_alfanum.drop(columns=['ES_ALFANUMERICO'])
    total_alfanum = len(registros_alfanum)
    
    print(f"   📊 Registros alfanuméricos activos encontrados: {total_alfanum}")
    
    if total_alfanum > 0:
        print(f"   📋 Ejemplos de identificaciones alfanuméricas:")
        ejemplos_alfanum = registros_alfanum['HST_IDN_NUMERO_IDENTIFICACION'].head(10).tolist()
        print(f"      {ejemplos_alfanum}")
    
    # =========================================================================
    # COMBINAR TODOS LOS CASOS
    # =========================================================================
    print(f"\n📊 RESUMEN DE REGISTROS A PROCESAR:")
    print(f"   Caso 1 (PE activos): {total_pe}")
    print(f"   Caso 2 (CN >= 1 año): {total_cn}")
    print(f"   Caso 3 (Alfanuméricos): {total_alfanum}")
    
    total_registros = total_pe + total_cn + total_alfanum
    print(f"   TOTAL: {total_registros}")
    
    if total_registros == 0:
        print(f"\n⚠️ No se encontraron registros para aplicar retiros masivos")
        return df_ns
    
    # ✅ COMBINAR CON MANEJO SEGURO DE DATAFRAMES VACÍOS
    dataframes_a_combinar = []
    
    if total_pe > 0:
        dataframes_a_combinar.append(registros_pe)
    if total_cn > 0:
        dataframes_a_combinar.append(registros_cn)
    if total_alfanum > 0:
        dataframes_a_combinar.append(registros_alfanum)
    
    if len(dataframes_a_combinar) == 0:
        print(f"\n⚠️ No hay DataFrames válidos para combinar")
        return df_ns
    
    # Combinar todos los registros
    try:
        registros_combinados = pd.concat(dataframes_a_combinar, ignore_index=True)
    except Exception as e:
        print(f"❌ Error al concatenar DataFrames: {e}")
        print(f"   DataFrames a combinar: {len(dataframes_a_combinar)}")
        for i, df_temp in enumerate(dataframes_a_combinar):
            print(f"   DF {i}: {len(df_temp)} filas, columnas: {list(df_temp.columns)[:5]}...")
        return df_ns
    
    # Eliminar duplicados por ID_User (por si hay casos que cumplan múltiples condiciones)
    registros_combinados = registros_combinados.drop_duplicates(subset=['ID_User'])
    total_unicos = len(registros_combinados)
    
    print(f"\n   📊 Total de registros únicos (sin duplicados): {total_unicos}")
    
    # =========================================================================
    # CONSTRUIR NOVEDADES N14
    # =========================================================================
    print(f"\n🔧 CONSTRUYENDO NOVEDADES N14 (SD)")
    
    # Verificar que tenemos registros para procesar
    if total_unicos == 0:
        print(f"⚠️ No hay registros únicos para procesar")
        return df_ns
    
    # ✅ VERIFICAR QUE EXISTEN LAS COLUMNAS NECESARIAS
    columnas_requeridas = [
        'ENT_ID', 'TPS_IDN_ID', 'HST_IDN_NUMERO_IDENTIFICACION',
        'AFL_PRIMER_APELLIDO', 'AFL_SEGUNDO_APELLIDO', 
        'AFL_PRIMER_NOMBRE', 'AFL_SEGUNDO_NOMBRE',
        'AFL_FECHA_NACIMIENTO', 'DPR_ID', 'MNC_ID', 'TPS_EST_AFL_ID'
    ]
    
    columnas_faltantes = [col for col in columnas_requeridas if col not in registros_combinados.columns]
    if columnas_faltantes:
        print(f"❌ Columnas faltantes en registros_combinados: {columnas_faltantes}")
        return df_ns
    
    # Mapear columnas de maestro_ADRES a df_NS
    try:
        nuevas_novedades = pd.DataFrame({
            'ENT_ID': registros_combinados['ENT_ID'],
            'TPS_IDN_ID': registros_combinados['TPS_IDN_ID'],
            'HST_IDN_NUMERO_IDENTIFICACION': registros_combinados['HST_IDN_NUMERO_IDENTIFICACION'],
            'AFL_PRIMER_APELLIDO': registros_combinados['AFL_PRIMER_APELLIDO'],
            'AFL_SEGUNDO_APELLIDO': registros_combinados['AFL_SEGUNDO_APELLIDO'],
            'AFL_PRIMER_NOMBRE': registros_combinados['AFL_PRIMER_NOMBRE'],
            'AFL_SEGUNDO_NOMBRE': registros_combinados['AFL_SEGUNDO_NOMBRE'],
            'AFL_FECHA_NACIMIENTO': registros_combinados['AFL_FECHA_NACIMIENTO'],
            'DPR_ID': registros_combinados['DPR_ID'],
            'MNS_ID': registros_combinados['MNC_ID'],  # Nota: MNC_ID -> MNS_ID
            'NOVEDAD': 'N14',
            'FECHA_NOVEDAD': fecha_novedad_str,
            'COD_1_NOVEDAD': 'SD',
            'ENT_ID_ADRES': registros_combinados['ENT_ID'],
            'TPS_EST_AFL_ID_from_adres': registros_combinados['TPS_EST_AFL_ID'],
            'Where': 'Novedad Masiva'
        })
        
        # Crear ID_User e ID_Register
        nuevas_novedades['ID_User'] = (
            nuevas_novedades['TPS_IDN_ID'].astype(str) + 
            nuevas_novedades['HST_IDN_NUMERO_IDENTIFICACION'].astype(str)
        )
        
        nuevas_novedades['ID_Register'] = (
            nuevas_novedades['TPS_IDN_ID'].astype(str) + 
            nuevas_novedades['HST_IDN_NUMERO_IDENTIFICACION'].astype(str) + 
            nuevas_novedades['NOVEDAD'].astype(str)
        )
        
        # Rellenar columnas faltantes con valores vacíos/None
        columnas_faltantes = [col for col in df_ns.columns if col not in nuevas_novedades.columns]
        for col in columnas_faltantes:
            nuevas_novedades[col] = None
        
        # Ordenar columnas en el mismo orden que df_NS
        nuevas_novedades = nuevas_novedades[df_ns.columns]
        
        print(f"   ✅ Novedades N14 construidas: {len(nuevas_novedades)}")
        
    except Exception as e:
        print(f"❌ Error al construir nuevas_novedades: {e}")
        return df_ns
    
    # Mostrar ejemplos
    if len(nuevas_novedades) > 0:
        print(f"\n📋 EJEMPLOS DE NOVEDADES N14 CREADAS:")
        ejemplos_columnas = ['TPS_IDN_ID', 'HST_IDN_NUMERO_IDENTIFICACION', 
                           'AFL_PRIMER_NOMBRE', 'AFL_PRIMER_APELLIDO',
                           'NOVEDAD', 'COD_1_NOVEDAD', 'FECHA_NOVEDAD', 'Where']
        ejemplos_columnas_disponibles = [col for col in ejemplos_columnas if col in nuevas_novedades.columns]
        ejemplos = nuevas_novedades[ejemplos_columnas_disponibles].head(10)
        print(ejemplos.to_string(index=False))
    
    # =========================================================================
    # AGREGAR AL DATAFRAME PRINCIPAL
    # =========================================================================
    print(f"\n🔄 AGREGANDO NOVEDADES A df_NS")
    
    try:
        registros_antes = len(df_ns)
        df_ns = pd.concat([df_ns, nuevas_novedades], ignore_index=True)
        registros_despues = len(df_ns)
        
        print(f"   📊 Registros antes: {registros_antes}")
        print(f"   📊 Registros después: {registros_despues}")
        print(f"   📊 Registros agregados: {registros_despues - registros_antes}")
        
    except Exception as e:
        print(f"❌ Error al agregar novedades a df_NS: {e}")
        return df_ns
    
    # =========================================================================
    # ESTADÍSTICAS FINALES POR TIPO DE DOCUMENTO
    # =========================================================================
    print(f"\n📈 ESTADÍSTICAS DE NOVEDADES N14 AGREGADAS:")
    
    try:
        if 'Where' in df_ns.columns:
            novedades_n14_masivas = df_ns[df_ns['Where'] == 'Novedad Masiva']
            if len(novedades_n14_masivas) > 0:
                print(f"   Por tipo de documento:")
                distribucion_tps = novedades_n14_masivas['TPS_IDN_ID'].value_counts()
                
                for tipo_doc, count in distribucion_tps.items():
                    print(f"     {tipo_doc}: {count} registros")
            else:
                print(f"   ⚠️ No se encontraron novedades masivas en el DataFrame final")
        else:
            print(f"   ⚠️ Columna 'Where' no encontrada en df_NS")
            
    except Exception as e:
        print(f"⚠️ Error al generar estadísticas finales: {e}")
    
    print(f"\n✅ RETIROS MASIVOS N14 APLICADOS EXITOSAMENTE")
    
    return df_ns

# Aplicar retiros masivos
df_NS = aplicar_retiros_masivos_N14(df_NS, maestro_ADRES, fecha_reporte)

## 5.8 N17 Actualización de Sexo biológico del afiliado

In [ ]:
def validar_novedades_N17(df, df_no_envio, maestro_adres):
    """
    Valida registros con NOVEDAD = 'N17' (Actualización de Sexo biológico del afiliado).
    
    Proceso:
    1. Traer el sexo actual desde maestro_ADRES (AFL_SEXO_IDENTIFICACION)
    2. Comparar con el sexo a reportar (COD_1_NOVEDAD)
    3. Si son iguales, mover a DF_No_Envio
    """
    print("🔍 INICIANDO VALIDACIÓN DE NOVEDADES N17")
    print("="*60)
    
    # Filtrar solo registros N17
    mask_n17 = df['NOVEDAD'] == 'N17'
    registros_n17 = df[mask_n17].copy()
    total_n17_inicial = len(registros_n17)
    
    print(f"📊 Total de registros N17 iniciales: {total_n17_inicial}")
    
    if total_n17_inicial == 0:
        print("⚠️ No se encontraron registros N17")
        return df, df_no_envio
    
    # =========================================================================
    # PASO 1: TRAER SEXO ACTUAL DESDE MAESTRO_ADRES
    # =========================================================================
    print(f"\n🔍 PASO 1: Obteniendo sexo actual desde maestro_ADRES")
    
    # Crear subset del maestro con sexo actual
    maestro_sexo = maestro_adres[['ID_User', 'AFL_SEXO_IDENTIFICACION']].drop_duplicates(subset=['ID_User'])
    
    print(f"   📊 Registros únicos en maestro_ADRES: {len(maestro_sexo):,}")
    
    # Hacer merge para traer AFL_SEXO_IDENTIFICACION del maestro
    registros_n17_con_maestro = registros_n17.merge(
        maestro_sexo,
        on='ID_User',
        how='left'
    )
    
    # Verificar cuántos registros tienen sexo en ADRES
    registros_con_sexo_adres = registros_n17_con_maestro['AFL_SEXO_IDENTIFICACION'].notna().sum()
    registros_sin_sexo_adres = registros_n17_con_maestro['AFL_SEXO_IDENTIFICACION'].isna().sum()
    
    print(f"   ✅ Registros con sexo en ADRES: {registros_con_sexo_adres}")
    print(f"   ⚠️ Registros sin sexo en ADRES: {registros_sin_sexo_adres}")
    
    # =========================================================================
    # PASO 2: NORMALIZAR Y COMPARAR SEXOS
    # =========================================================================
    print(f"\n🔧 PASO 2: Normalizando y comparando valores de sexo")
    
    # Normalizar valores (quitar espacios y convertir a mayúsculas)
    registros_n17_con_maestro['COD_1_NOVEDAD_NORM'] = registros_n17_con_maestro['COD_1_NOVEDAD'].astype(str).str.strip().str.upper()
    registros_n17_con_maestro['AFL_SEXO_IDENTIFICACION_NORM'] = registros_n17_con_maestro['AFL_SEXO_IDENTIFICACION'].astype(str).str.strip().str.upper()
    
    # Mostrar distribución de valores
    print(f"\n📊 Distribución de sexo a reportar (COD_1_NOVEDAD):")
    distribucion_nuevo = registros_n17_con_maestro['COD_1_NOVEDAD_NORM'].value_counts()
    for sexo, count in distribucion_nuevo.items():
        print(f"   {sexo}: {count} registros")
    
    print(f"\n📊 Distribución de sexo actual en ADRES:")
    distribucion_actual = registros_n17_con_maestro['AFL_SEXO_IDENTIFICACION_NORM'].value_counts()
    for sexo, count in distribucion_actual.items():
        print(f"   {sexo}: {count} registros")
    
    # =========================================================================
    # PASO 3: IDENTIFICAR REGISTROS SIN CAMBIOS
    # =========================================================================
    print(f"\n🔍 PASO 3: Identificando registros sin cambios de sexo")
    
    # Crear máscara para registros donde el sexo es igual
    mask_sin_cambios = (
        (registros_n17_con_maestro['COD_1_NOVEDAD_NORM'] == registros_n17_con_maestro['AFL_SEXO_IDENTIFICACION_NORM']) &
        registros_n17_con_maestro['AFL_SEXO_IDENTIFICACION'].notna()  # Solo comparar si existe sexo en ADRES
    )
    
    registros_sin_cambios = registros_n17_con_maestro[mask_sin_cambios].copy()
    total_sin_cambios = len(registros_sin_cambios)
    
    print(f"   Registros con mismo sexo en ADRES y a reportar: {total_sin_cambios}")
    print(f"   Registros con cambios válidos: {total_n17_inicial - total_sin_cambios}")
    
    if total_sin_cambios > 0:
        # Mostrar ejemplos
        print(f"\n📋 EJEMPLOS DE REGISTROS SIN CAMBIOS:")
        ejemplos_sin_cambios = registros_sin_cambios[[
            'NUM_SOLICITUD_NOVEDAD', 'HST_IDN_NUMERO_IDENTIFICACION',
            'COD_1_NOVEDAD_NORM', 'AFL_SEXO_IDENTIFICACION_NORM'
        ]].head(5)
        print(ejemplos_sin_cambios.to_string(index=False))
        
        # Limpiar y mover a DF_No_Envio
        registros_sin_cambios_clean = registros_sin_cambios.drop(columns=[
            'COD_1_NOVEDAD_NORM', 'AFL_SEXO_IDENTIFICACION_NORM', 'AFL_SEXO_IDENTIFICACION'
        ])
        registros_sin_cambios_clean['motivo'] = "Afiliado sin cambios a reportar"
        
        # Concatenar con DF_No_Envio
        df_no_envio = pd.concat([df_no_envio, registros_sin_cambios_clean], ignore_index=True)
        
        # Obtener índices originales para eliminar del DataFrame principal
        # Necesitamos recalcular la máscara en el df original
        mask_n17_original = df['NOVEDAD'] == 'N17'
        registros_n17_original = df[mask_n17_original].copy()
        
        # Merge con el maestro en el df original
        registros_n17_original_con_maestro = registros_n17_original.merge(
            maestro_sexo,
            on='ID_User',
            how='left'
        )
        registros_n17_original_con_maestro['COD_1_NOVEDAD_NORM'] = registros_n17_original_con_maestro['COD_1_NOVEDAD'].astype(str).str.strip().str.upper()
        registros_n17_original_con_maestro['AFL_SEXO_IDENTIFICACION_NORM'] = registros_n17_original_con_maestro['AFL_SEXO_IDENTIFICACION'].astype(str).str.strip().str.upper()
        
        # Identificar índices a eliminar
        mask_eliminar_original = (
            (registros_n17_original_con_maestro['COD_1_NOVEDAD_NORM'] == registros_n17_original_con_maestro['AFL_SEXO_IDENTIFICACION_NORM']) &
            registros_n17_original_con_maestro['AFL_SEXO_IDENTIFICACION'].notna()
        )
        #indices_a_eliminar = df.loc[mask_n17_original][mask_eliminar_original].index
        #df = df.drop(indices_a_eliminar).reset_index(drop=True)
        
        # Identificar índices a eliminar
        # CORRECCIÓN: Usar ID_User para eliminar registros
        ids_a_eliminar = registros_n17_original_con_maestro[mask_eliminar_original]['ID_User'].unique()
        mask_eliminar_df = (df['NOVEDAD'] == 'N17') & (df['ID_User'].isin(ids_a_eliminar))
        df = df[~mask_eliminar_df].reset_index(drop=True)
        
        print(f"✅ Movidos {total_sin_cambios} registros a DF_No_Envio")
    else:
        print(f"✅ Todos los registros N17 son cambios válidos para reportar")
    
    # =========================================================================
    # PASO 4: ANÁLISIS DE REGISTROS SIN SEXO EN ADRES
    # =========================================================================
    if registros_sin_sexo_adres > 0:
        print(f"\n⚠️ REGISTROS SIN SEXO EN MAESTRO_ADRES:")
        registros_sin_match = registros_n17_con_maestro[registros_n17_con_maestro['AFL_SEXO_IDENTIFICACION'].isna()]
        print(f"   Total: {registros_sin_sexo_adres}")
        print(f"   📋 Ejemplos de registros sin match:")
        ejemplos_sin_match = registros_sin_match[[
            'NUM_SOLICITUD_NOVEDAD', 'HST_IDN_NUMERO_IDENTIFICACION',
            'COD_1_NOVEDAD', 'ID_User'
        ]].head(5)
        print(ejemplos_sin_match.to_string(index=False))
        print(f"   ℹ️ Estos registros se mantendrán en df_NS (asumiendo que son cambios válidos)")
    
    # =========================================================================
    # PASO 5: RESUMEN FINAL
    # =========================================================================
    registros_n17_final = len(df[df['NOVEDAD'] == 'N17'])
    
    print(f"\n📈 RESUMEN FINAL:")
    print(f"   Registros N17 iniciales: {total_n17_inicial}")
    print(f"   Registros movidos (sin cambios): {total_sin_cambios}")
    print(f"   Registros N17 restantes (válidos): {registros_n17_final}")
    print(f"   Total en DF_No_Envio actual: {len(df_no_envio)}")
    
    # Mostrar ejemplos de registros válidos restantes si hay
    if registros_n17_final > 0:
        print(f"\n📋 EJEMPLOS DE REGISTROS N17 VÁLIDOS RESTANTES:")
        ejemplos_validos = df[df['NOVEDAD'] == 'N17'][[
            'NUM_SOLICITUD_NOVEDAD', 'HST_IDN_NUMERO_IDENTIFICACION',
            'COD_1_NOVEDAD'
        ]].head(5)
        print(ejemplos_validos.to_string(index=False))
        
        # Mostrar distribución de sexos a reportar en registros válidos
        print(f"\n📊 Distribución de sexo a reportar en registros válidos:")
        distribucion_validos = df[df['NOVEDAD'] == 'N17']['COD_1_NOVEDAD'].value_counts()
        for sexo, count in distribucion_validos.items():
            print(f"   {sexo}: {count} registros")
    
    return df, df_no_envio

# Aplicar validación N17
df_NS, DF_No_Envio = validar_novedades_N17(df_NS, DF_No_Envio, maestro_ADRES)

## 5.9. N19 Actualización de zona de afiliación

In [ ]:
def validar_novedades_N19(df, df_no_envio, maestro_adres):
    """
    Valida registros con NOVEDAD = 'N19' (Actualización de zona de afiliación).
    
    Proceso:
    1. Validar formato de COD_1_NOVEDAD: debe ser R1, R2, U1, U2, R, o U
    2. Corregir formato: Si es "R" → "R1", Si es "U" → "U1" 
    3. Traer zona actual desde maestro_ADRES (ZNS_ID)
    4. Verificar cambios reales: si COD_1_NOVEDAD == ZNS_ID, mover a DF_No_Envio
    """
    print("🔍 INICIANDO VALIDACIÓN DE NOVEDADES N19")
    print("="*60)
    
    # Filtrar solo registros N19
    mask_n19 = df['NOVEDAD'] == 'N19'
    registros_n19 = df[mask_n19].copy()
    total_n19_inicial = len(registros_n19)
    
    print(f"📊 Total de registros N19 iniciales: {total_n19_inicial}")
    
    if total_n19_inicial == 0:
        print("⚠️ No se encontraron registros N19")
        return df, df_no_envio
    
    # =========================================================================
    # PASO 1: VALIDAR Y CORREGIR FORMATO DE COD_1_NOVEDAD
    # =========================================================================
    print(f"\n🔧 PASO 1: Validando y corrigiendo formato de COD_1_NOVEDAD")
    
    # Normalizar COD_1_NOVEDAD (quitar espacios y mayúsculas)
    registros_n19['COD_1_NOVEDAD_NORM'] = registros_n19['COD_1_NOVEDAD'].astype(str).str.strip().str.upper()
    
    # Mostrar distribución inicial de valores
    print(f"\n📊 Distribución inicial de COD_1_NOVEDAD:")
    distribucion_inicial = registros_n19['COD_1_NOVEDAD_NORM'].value_counts()
    for valor, count in distribucion_inicial.items():
        print(f"   {valor}: {count} registros")
    
    # Valores válidos finales
    valores_validos_finales = ['R1', 'R2', 'U1', 'U2']
    valores_corregibles = ['R', 'U']
    
    # Función para corregir formato
    def corregir_zona(valor):
        if pd.isna(valor) or str(valor).strip() == '':
            return None
        
        valor_norm = str(valor).strip().upper()
        
        # Si ya es válido, mantener
        if valor_norm in valores_validos_finales:
            return valor_norm
        # Si es corregible, aplicar corrección por defecto
        elif valor_norm == 'R':
            return 'R1'
        elif valor_norm == 'U':
            return 'U1'
        else:
            return valor_norm  # Mantener para identificar como inválido
    
    # Aplicar corrección
    registros_n19['COD_1_NOVEDAD_CORREGIDO'] = registros_n19['COD_1_NOVEDAD_NORM'].apply(corregir_zona)
    
    # Contar correcciones aplicadas
    correcciones_r = (registros_n19['COD_1_NOVEDAD_NORM'] == 'R').sum()
    correcciones_u = (registros_n19['COD_1_NOVEDAD_NORM'] == 'U').sum()
    total_correcciones = correcciones_r + correcciones_u
    
    print(f"\n📈 CORRECCIONES APLICADAS:")
    print(f"   'R' → 'R1': {correcciones_r} registros")
    print(f"   'U' → 'U1': {correcciones_u} registros")
    print(f"   Total correcciones: {total_correcciones}")
    
    # Mostrar distribución después de correcciones
    print(f"\n📊 Distribución después de correcciones:")
    distribucion_corregida = registros_n19['COD_1_NOVEDAD_CORREGIDO'].value_counts()
    for valor, count in distribucion_corregida.items():
        print(f"   {valor}: {count} registros")
    
    # =========================================================================
    # PASO 2: IDENTIFICAR Y MOVER VALORES INVÁLIDOS
    # =========================================================================
    print(f"\n🔍 PASO 2: Identificando valores inválidos")
    
    # Identificar registros con valores inválidos
    mask_invalidos = ~registros_n19['COD_1_NOVEDAD_CORREGIDO'].isin(valores_validos_finales)
    registros_invalidos = registros_n19[mask_invalidos].copy()
    total_invalidos = len(registros_invalidos)
    
    print(f"   Registros con valores válidos: {total_n19_inicial - total_invalidos}")
    print(f"   Registros con valores inválidos: {total_invalidos}")
    
    if total_invalidos > 0:
        # Mostrar ejemplos de valores inválidos
        print(f"\n📋 EJEMPLOS DE VALORES INVÁLIDOS:")
        ejemplos_invalidos = registros_invalidos[[
            'NUM_SOLICITUD_NOVEDAD', 'HST_IDN_NUMERO_IDENTIFICACION',
            'COD_1_NOVEDAD', 'COD_1_NOVEDAD_CORREGIDO'
        ]].head(5)
        print(ejemplos_invalidos.to_string(index=False))
        
        # Limpiar y mover a DF_No_Envio
        registros_invalidos_clean = registros_invalidos.drop(columns=[
            'COD_1_NOVEDAD_NORM', 'COD_1_NOVEDAD_CORREGIDO'
        ])
        registros_invalidos_clean['motivo'] = "Valor no valido en la novedad"
        
        # Concatenar con DF_No_Envio
        df_no_envio = pd.concat([df_no_envio, registros_invalidos_clean], ignore_index=True)
        
        # ✅ CORRECCIÓN: Obtener los índices originales correctamente
        indices_originales_invalidos = registros_n19[mask_invalidos].index
        df = df.drop(indices_originales_invalidos).reset_index(drop=True)
        
        # Actualizar registros_n19 para la siguiente validación
        registros_n19 = registros_n19[~mask_invalidos].copy()
        
        print(f"✅ Movidos {total_invalidos} registros a DF_No_Envio por valores inválidos")
    else:
        print(f"✅ Todos los registros tienen valores válidos")
    
    # =========================================================================
    # PASO 3: APLICAR CORRECCIONES AL DATAFRAME PRINCIPAL
    # =========================================================================
    registros_n19_restantes = len(registros_n19)
    
    if registros_n19_restantes == 0:
        print(f"\n⚠️ No quedan registros N19 para validar")
        return df, df_no_envio
    
    print(f"\n🔄 PASO 3: Aplicando correcciones al DataFrame principal")
    print(f"   Registros a actualizar: {registros_n19_restantes}")
    
    # ✅ CORRECCIÓN: Aplicar correcciones usando los índices originales
    for idx in registros_n19.index:
        if idx in df.index and df.loc[idx, 'NOVEDAD'] == 'N19':
            valor_corregido = registros_n19.loc[idx, 'COD_1_NOVEDAD_CORREGIDO']
            if pd.notna(valor_corregido):
                df.loc[idx, 'COD_1_NOVEDAD'] = valor_corregido
    
    print(f"   ✅ Correcciones aplicadas al DataFrame principal")
    
    # =========================================================================
    # PASO 4: TRAER ZONA ACTUAL DESDE MAESTRO_ADRES
    # =========================================================================
    print(f"\n🔍 PASO 4: Obteniendo zona actual desde maestro_ADRES")
    
    # Crear subset del maestro con zona actual
    maestro_zona = maestro_adres[['ID_User', 'ZNS_ID']].drop_duplicates(subset=['ID_User'])
    
    print(f"   📊 Registros únicos en maestro_ADRES: {len(maestro_zona):,}")
    
    # ✅ CORRECCIÓN: Recargar registros N19 desde el df actualizado
    registros_n19_actualizado = df[df['NOVEDAD'] == 'N19'].copy()
    
    # Hacer merge para traer ZNS_ID del maestro
    registros_n19_con_maestro = registros_n19_actualizado.merge(
        maestro_zona,
        on='ID_User',
        how='left'
    )
    
    # Verificar cuántos registros tienen zona en ADRES
    registros_con_zona_adres = registros_n19_con_maestro['ZNS_ID'].notna().sum()
    registros_sin_zona_adres = registros_n19_con_maestro['ZNS_ID'].isna().sum()
    
    print(f"   ✅ Registros con zona en ADRES: {registros_con_zona_adres}")
    print(f"   ⚠️ Registros sin zona en ADRES: {registros_sin_zona_adres}")
    
    # =========================================================================
    # PASO 5: IDENTIFICAR REGISTROS SIN CAMBIOS
    # =========================================================================
    print(f"\n🔍 PASO 5: Identificando registros sin cambios de zona")
    
    # Normalizar valores para comparación
    registros_n19_con_maestro['COD_1_NOVEDAD_NORM'] = registros_n19_con_maestro['COD_1_NOVEDAD'].astype(str).str.strip().str.upper()
    registros_n19_con_maestro['ZNS_ID_NORM'] = registros_n19_con_maestro['ZNS_ID'].astype(str).str.strip().str.upper()
    
    # Crear máscara para registros sin cambios
    mask_sin_cambios = (
        (registros_n19_con_maestro['COD_1_NOVEDAD_NORM'] == registros_n19_con_maestro['ZNS_ID_NORM']) &
        registros_n19_con_maestro['ZNS_ID'].notna()  # Solo comparar si existe zona en ADRES
    )
    
    registros_sin_cambios = registros_n19_con_maestro[mask_sin_cambios].copy()
    total_sin_cambios = len(registros_sin_cambios)
    
    print(f"   Registros con misma zona en ADRES y a reportar: {total_sin_cambios}")
    print(f"   Registros con cambios válidos: {len(registros_n19_con_maestro) - total_sin_cambios}")
    
    if total_sin_cambios > 0:
        # Mostrar ejemplos
        print(f"\n📋 EJEMPLOS DE REGISTROS SIN CAMBIOS:")
        ejemplos_sin_cambios = registros_sin_cambios[[
            'NUM_SOLICITUD_NOVEDAD', 'HST_IDN_NUMERO_IDENTIFICACION',
            'COD_1_NOVEDAD_NORM', 'ZNS_ID_NORM'
        ]].head(5)
        print(ejemplos_sin_cambios.to_string(index=False))
        
        # Limpiar y mover a DF_No_Envio
        registros_sin_cambios_clean = registros_sin_cambios.drop(columns=[
            'COD_1_NOVEDAD_NORM', 'ZNS_ID_NORM', 'ZNS_ID'
        ])
        registros_sin_cambios_clean['motivo'] = "Afiliado sin cambios a reportar"
        
        # Concatenar con DF_No_Envio
        df_no_envio = pd.concat([df_no_envio, registros_sin_cambios_clean], ignore_index=True)
        
        # ✅ CORRECCIÓN: Usar ID_User para identificar registros a eliminar
        ids_a_eliminar = registros_sin_cambios['ID_User'].unique()
        
        # Crear máscara en el df original usando ID_User
        mask_eliminar_df = (df['NOVEDAD'] == 'N19') & (df['ID_User'].isin(ids_a_eliminar))
        
        # Eliminar registros
        df = df[~mask_eliminar_df].reset_index(drop=True)
        
        print(f"✅ Movidos {total_sin_cambios} registros a DF_No_Envio")
    else:
        print(f"✅ Todos los registros N19 son cambios válidos para reportar")
    
    # =========================================================================
    # PASO 6: ANÁLISIS DE REGISTROS SIN ZONA EN ADRES
    # =========================================================================
    if registros_sin_zona_adres > 0:
        print(f"\n⚠️ REGISTROS SIN ZONA EN MAESTRO_ADRES:")
        registros_sin_match = registros_n19_con_maestro[registros_n19_con_maestro['ZNS_ID'].isna()]
        print(f"   Total: {registros_sin_zona_adres}")
        print(f"   📋 Ejemplos de registros sin match:")
        ejemplos_sin_match = registros_sin_match[[
            'NUM_SOLICITUD_NOVEDAD', 'HST_IDN_NUMERO_IDENTIFICACION',
            'COD_1_NOVEDAD', 'ID_User'
        ]].head(5)
        print(ejemplos_sin_match.to_string(index=False))
        print(f"   ℹ️ Estos registros se mantendrán en df_NS (asumiendo que son cambios válidos)")
    
    # =========================================================================
    # PASO 7: RESUMEN FINAL
    # =========================================================================
    registros_n19_final = len(df[df['NOVEDAD'] == 'N19'])
    total_movidos = total_invalidos + total_sin_cambios
    
    print(f"\n📈 RESUMEN FINAL:")
    print(f"   Registros N19 iniciales: {total_n19_inicial}")
    print(f"   Correcciones de formato aplicadas: {total_correcciones}")
    print(f"   Registros movidos por valores inválidos: {total_invalidos}")
    print(f"   Registros movidos por sin cambios: {total_sin_cambios}")
    print(f"   Total movidos: {total_movidos}")
    print(f"   Registros N19 restantes (válidos): {registros_n19_final}")
    print(f"   Total en DF_No_Envio actual: {len(df_no_envio)}")
    
    # Mostrar ejemplos de registros válidos restantes si hay
    if registros_n19_final > 0:
        print(f"\n📋 EJEMPLOS DE REGISTROS N19 VÁLIDOS RESTANTES:")
        ejemplos_validos = df[df['NOVEDAD'] == 'N19'][[
            'NUM_SOLICITUD_NOVEDAD', 'HST_IDN_NUMERO_IDENTIFICACION',
            'COD_1_NOVEDAD'
        ]].head(5)
        print(ejemplos_validos.to_string(index=False))
        
        # Mostrar distribución de zonas a reportar en registros válidos
        print(f"\n📊 Distribución de zona a reportar en registros válidos:")
        distribucion_validos = df[df['NOVEDAD'] == 'N19']['COD_1_NOVEDAD'].value_counts()
        for zona, count in distribucion_validos.items():
            print(f"   {zona}: {count} registros")
    
    return df, df_no_envio

# Aplicar validación N19
df_NS, DF_No_Envio = validar_novedades_N19(df_NS, DF_No_Envio, maestro_ADRES)

### 5.9.1. N19 Masiva

In [ ]:
def aplicar_actualizaciones_masivas_N19(df_ns, maestro_adres, fecha_reporte):
    """
    Aplica actualizaciones masivas N19 (cambio de zona) para afiliados activos.
    
    Proceso:
    1. Identificar afiliados activos (TPS_EST_AFL_ID = 'AC') en maestro_ADRES
    2. Filtrar por zonas R o U
    3. Convertir zonas: R → R1, U → U1
    4. Construir novedades N19 con la nueva zona
    
    Args:
        df_ns: DataFrame de novedades
        maestro_adres: DataFrame maestro ADRES
        fecha_reporte: fecha del reporte (datetime)
    
    Returns:
        df_ns actualizado con las nuevas novedades N19
    """
    print("🔍 INICIANDO APLICACIÓN DE ACTUALIZACIONES MASIVAS N19")
    print("="*80)
    
    # Convertir fecha_reporte a formato DD/MM/YYYY
    fecha_novedad_str = fecha_reporte.strftime('%d/%m/%Y')
    print(f"📅 Fecha de novedad: {fecha_novedad_str}")
    
    # =========================================================================
    # PASO 1: FILTRAR AFILIADOS ACTIVOS CON ZONAS R o U
    # =========================================================================
    print(f"\n🔍 PASO 1: Identificando afiliados activos con zonas R o U")
    
    mask_activos_zona_ru = (
        (maestro_adres['ENT_ID'] == 'EPS025') &
        (maestro_adres['TPS_EST_AFL_ID'] == 'AC') &
        (maestro_adres['ZNS_ID'].isin(['R', 'U']))
    )
    
    registros_zona_ru = maestro_adres[mask_activos_zona_ru].copy().reset_index(drop=True)
    total_zona_ru = len(registros_zona_ru)
    
    print(f"   📊 Registros activos con zona R o U encontrados: {total_zona_ru}")
    
    if total_zona_ru == 0:
        print(f"\n⚠️ No se encontraron registros para aplicar actualizaciones masivas N19")
        return df_ns
    
    # Mostrar distribución de zonas
    print(f"\n📊 Distribución de zonas a actualizar:")
    distribucion_zonas = registros_zona_ru['ZNS_ID'].value_counts()
    for zona, count in distribucion_zonas.items():
        print(f"   Zona {zona}: {count} registros")
    
    # =========================================================================
    # PASO 2: CONVERTIR ZONAS R → R1, U → U1
    # =========================================================================
    print(f"\n🔧 PASO 2: Convirtiendo zonas al formato correcto")
    
    def convertir_zona(zona):
        """Convierte R → R1, U → U1"""
        if pd.isna(zona) or str(zona).strip() == '':
            return None
        
        zona_norm = str(zona).strip().upper()
        
        if zona_norm == 'R':
            return 'R1'
        elif zona_norm == 'U':
            return 'U1'
        else:
            return zona_norm  # Por si ya tiene el formato correcto
    
    # Aplicar conversión
    registros_zona_ru['COD_1_NOVEDAD_CONVERTIDO'] = registros_zona_ru['ZNS_ID'].apply(convertir_zona)
    
    # Estadísticas de conversión
    conversiones_r = (registros_zona_ru['ZNS_ID'] == 'R').sum()
    conversiones_u = (registros_zona_ru['ZNS_ID'] == 'U').sum()
    
    print(f"   ✅ Conversiones aplicadas:")
    print(f"     R → R1: {conversiones_r} registros")
    print(f"     U → U1: {conversiones_u} registros")
    
    # Verificar que todas las conversiones son válidas
    zonas_convertidas_validas = registros_zona_ru['COD_1_NOVEDAD_CONVERTIDO'].isin(['R1', 'U1']).sum()
    print(f"   📊 Zonas convertidas válidas: {zonas_convertidas_validas}/{total_zona_ru}")
    
    if zonas_convertidas_validas < total_zona_ru:
        print(f"   ⚠️ Algunas zonas no se convirtieron correctamente")
        zonas_invalidas = registros_zona_ru[~registros_zona_ru['COD_1_NOVEDAD_CONVERTIDO'].isin(['R1', 'U1'])]['ZNS_ID'].unique()
        print(f"      Zonas problemáticas: {zonas_invalidas}")
    
    # =========================================================================
    # PASO 3: CONSTRUIR NOVEDADES N19
    # =========================================================================
    print(f"\n🔧 PASO 3: CONSTRUYENDO NOVEDADES N19")
    
    # Verificar columnas necesarias
    columnas_requeridas = [
        'ENT_ID', 'TPS_IDN_ID', 'HST_IDN_NUMERO_IDENTIFICACION',
        'AFL_PRIMER_APELLIDO', 'AFL_SEGUNDO_APELLIDO', 
        'AFL_PRIMER_NOMBRE', 'AFL_SEGUNDO_NOMBRE',
        'AFL_FECHA_NACIMIENTO', 'DPR_ID', 'MNC_ID', 'TPS_EST_AFL_ID', 'ZNS_ID'
    ]
    
    columnas_faltantes = [col for col in columnas_requeridas if col not in registros_zona_ru.columns]
    if columnas_faltantes:
        print(f"❌ Columnas faltantes en registros_zona_ru: {columnas_faltantes}")
        return df_ns
    
    try:
        # Mapear columnas de maestro_ADRES a df_NS
        nuevas_novedades = pd.DataFrame({
            'ENT_ID': registros_zona_ru['ENT_ID'],
            'TPS_IDN_ID': registros_zona_ru['TPS_IDN_ID'],
            'HST_IDN_NUMERO_IDENTIFICACION': registros_zona_ru['HST_IDN_NUMERO_IDENTIFICACION'],
            'AFL_PRIMER_APELLIDO': registros_zona_ru['AFL_PRIMER_APELLIDO'],
            'AFL_SEGUNDO_APELLIDO': registros_zona_ru['AFL_SEGUNDO_APELLIDO'],
            'AFL_PRIMER_NOMBRE': registros_zona_ru['AFL_PRIMER_NOMBRE'],
            'AFL_SEGUNDO_NOMBRE': registros_zona_ru['AFL_SEGUNDO_NOMBRE'],
            'AFL_FECHA_NACIMIENTO': registros_zona_ru['AFL_FECHA_NACIMIENTO'],
            'DPR_ID': registros_zona_ru['DPR_ID'],
            'MNS_ID': registros_zona_ru['MNC_ID'],  # Nota: MNC_ID -> MNS_ID
            'NOVEDAD': 'N19',
            'FECHA_NOVEDAD': fecha_novedad_str,
            'COD_1_NOVEDAD': registros_zona_ru['COD_1_NOVEDAD_CONVERTIDO'],  # Zona convertida (R1 o U1)
            'ENT_ID_ADRES': registros_zona_ru['ENT_ID'],
            'TPS_EST_AFL_ID_from_adres': registros_zona_ru['TPS_EST_AFL_ID'],
            'Where': 'Novedad Masiva'
        })
        
        # Crear ID_User e ID_Register
        nuevas_novedades['ID_User'] = (
            nuevas_novedades['TPS_IDN_ID'].astype(str) + 
            nuevas_novedades['HST_IDN_NUMERO_IDENTIFICACION'].astype(str)
        )
        
        nuevas_novedades['ID_Register'] = (
            nuevas_novedades['TPS_IDN_ID'].astype(str) + 
            nuevas_novedades['HST_IDN_NUMERO_IDENTIFICACION'].astype(str) + 
            nuevas_novedades['NOVEDAD'].astype(str)
        )
        
        # Rellenar columnas faltantes con valores vacíos/None
        columnas_faltantes_df = [col for col in df_ns.columns if col not in nuevas_novedades.columns]
        for col in columnas_faltantes_df:
            nuevas_novedades[col] = None
        
        # Ordenar columnas en el mismo orden que df_NS
        nuevas_novedades = nuevas_novedades[df_ns.columns]
        
        print(f"   ✅ Novedades N19 construidas: {len(nuevas_novedades)}")
        
    except Exception as e:
        print(f"❌ Error al construir nuevas_novedades: {e}")
        return df_ns
    
    # Mostrar ejemplos
    if len(nuevas_novedades) > 0:
        print(f"\n📋 EJEMPLOS DE NOVEDADES N19 CREADAS:")
        ejemplos_columnas = ['TPS_IDN_ID', 'HST_IDN_NUMERO_IDENTIFICACION', 
                           'AFL_PRIMER_NOMBRE', 'AFL_PRIMER_APELLIDO',
                           'NOVEDAD', 'COD_1_NOVEDAD', 'FECHA_NOVEDAD', 'Where']
        ejemplos_columnas_disponibles = [col for col in ejemplos_columnas if col in nuevas_novedades.columns]
        ejemplos = nuevas_novedades[ejemplos_columnas_disponibles].head(10)
        print(ejemplos.to_string(index=False))
    
    # =========================================================================
    # PASO 4: AGREGAR AL DATAFRAME PRINCIPAL
    # =========================================================================
    print(f"\n🔄 PASO 4: AGREGANDO NOVEDADES A df_NS")
    
    try:
        registros_antes = len(df_ns)
        df_ns = pd.concat([df_ns, nuevas_novedades], ignore_index=True)
        registros_despues = len(df_ns)
        
        print(f"   📊 Registros antes: {registros_antes:,}")
        print(f"   📊 Registros después: {registros_despues:,}")
        print(f"   📊 Registros agregados: {registros_despues - registros_antes:,}")
        
    except Exception as e:
        print(f"❌ Error al agregar novedades a df_NS: {e}")
        return df_ns
    
    # =========================================================================
    # PASO 5: ESTADÍSTICAS FINALES
    # =========================================================================
    print(f"\n📈 ESTADÍSTICAS DE NOVEDADES N19 AGREGADAS:")
    
    try:
        if 'Where' in df_ns.columns:
            novedades_n19_masivas = df_ns[
                (df_ns['Where'] == 'Novedad Masiva') & 
                (df_ns['NOVEDAD'] == 'N19')
            ]
            
            if len(novedades_n19_masivas) > 0:
                print(f"   📊 Total novedades N19 masivas: {len(novedades_n19_masivas):,}")
                
                # Distribución por zona convertida
                print(f"   📊 Por zona convertida:")
                distribucion_zona_final = novedades_n19_masivas['COD_1_NOVEDAD'].value_counts()
                for zona, count in distribucion_zona_final.items():
                    print(f"     {zona}: {count:,} registros")
                
                # Distribución por tipo de documento
                print(f"\n   📊 Por tipo de documento:")
                distribucion_tps = novedades_n19_masivas['TPS_IDN_ID'].value_counts()
                for tipo_doc, count in distribucion_tps.items():
                    print(f"     {tipo_doc}: {count:,} registros")
            else:
                print(f"   ⚠️ No se encontraron novedades N19 masivas en el DataFrame final")
        else:
            print(f"   ⚠️ Columna 'Where' no encontrada en df_NS")
            
    except Exception as e:
        print(f"⚠️ Error al generar estadísticas finales: {e}")
    
    # =========================================================================
    # PASO 6: VERIFICACIÓN DE CALIDAD
    # =========================================================================
    print(f"\n🔍 VERIFICACIÓN DE CALIDAD:")
    
    try:
        # Verificar que todas las zonas en N19 masivas son válidas
        novedades_n19_masivas = df_ns[
            (df_ns['Where'] == 'Novedad Masiva') & 
            (df_ns['NOVEDAD'] == 'N19')
        ]
        
        if len(novedades_n19_masivas) > 0:
            zonas_validas = novedades_n19_masivas['COD_1_NOVEDAD'].isin(['R1', 'U1']).sum()
            total_n19_masivas = len(novedades_n19_masivas)
            
            print(f"   ✅ Zonas válidas (R1/U1): {zonas_validas}/{total_n19_masivas}")
            
            if zonas_validas < total_n19_masivas:
                print(f"   ⚠️ Hay {total_n19_masivas - zonas_validas} novedades con zonas no válidas")
                zonas_invalidas_final = novedades_n19_masivas[
                    ~novedades_n19_masivas['COD_1_NOVEDAD'].isin(['R1', 'U1'])
                ]['COD_1_NOVEDAD'].unique()
                print(f"      Zonas inválidas encontradas: {zonas_invalidas_final}")
            
            # Verificar que todos tienen estado AC
            estados_activos = novedades_n19_masivas['TPS_EST_AFL_ID_from_adres'].eq('AC').sum()
            print(f"   ✅ Registros con estado AC: {estados_activos}/{total_n19_masivas}")
            
        else:
            print(f"   ⚠️ No hay novedades N19 masivas para verificar")
            
    except Exception as e:
        print(f"⚠️ Error en verificación de calidad: {e}")
    
    print(f"\n✅ ACTUALIZACIONES MASIVAS N19 APLICADAS EXITOSAMENTE")
    
    return df_ns

# Aplicar actualizaciones masivas N19
df_NS = aplicar_actualizaciones_masivas_N19(df_NS, maestro_ADRES, fecha_reporte)

## 5.10. N21 Actualización condiciones Sisbén, etnia y comunidad indígena o resguardo

In [ ]:
def validar_novedades_N21(df, df_no_envio):
    """
    Valida registros con NOVEDAD = 'N21' (Actualización condiciones Sisbén, etnia y comunidad indígena o resguardo).
    
    Proceso:
    1. Si COD_3_NOVEDAD = 5:
       - COD_2_NOVEDAD debe tener formato válido (A01-C99)
       - COD_1_NOVEDAD debe ser 2 (corregir si es diferente)
       - COD_4_NOVEDAD: 1 para A/B, 2 para C
    2. Si COD_3_NOVEDAD ≠ 5:
       - COD_2_NOVEDAD debe estar vacío
       - COD_1_NOVEDAD debe ser 3
       - COD_4_NOVEDAD debe ser N
    3. COD_5_NOVEDAD:
       - Por defecto "06"
       - Si COD_3_NOVEDAD = 17: "01" y COD_6_NOVEDAD = "999"
    """
    print("🔍 INICIANDO VALIDACIÓN DE NOVEDADES N21")
    print("="*60)
    
    # Filtrar solo registros N21
    mask_n21 = df['NOVEDAD'] == 'N21'
    registros_n21 = df[mask_n21].copy()
    total_n21_inicial = len(registros_n21)
    
    print(f"📊 Total de registros N21 iniciales: {total_n21_inicial}")
    
    if total_n21_inicial == 0:
        print("⚠️ No se encontraron registros N21")
        return df, df_no_envio
    
    # Normalizar COD_3_NOVEDAD para comparaciones
    registros_n21['COD_3_NOVEDAD_NORM'] = registros_n21['COD_3_NOVEDAD'].astype(str).str.strip()
    
    # Mostrar distribución inicial de COD_3_NOVEDAD
    print(f"\n📊 Distribución de COD_3_NOVEDAD:")
    distribucion_cod3 = registros_n21['COD_3_NOVEDAD_NORM'].value_counts()
    for valor, count in distribucion_cod3.items():
        print(f"   {valor}: {count} registros")
    
    # =========================================================================
    # VALIDACIÓN 1: REGISTROS CON COD_3_NOVEDAD = 5
    # =========================================================================
    print(f"\n🔍 VALIDACIÓN 1: Registros con COD_3_NOVEDAD = 5")
    
    mask_cod3_es_5 = registros_n21['COD_3_NOVEDAD_NORM'] == '5'
    registros_cod3_5 = registros_n21[mask_cod3_es_5].copy()
    total_cod3_5 = len(registros_cod3_5)
    
    print(f"   📊 Registros con COD_3_NOVEDAD = 5: {total_cod3_5}")
    
    registros_invalidos_cod3_5 = 0
    
    if total_cod3_5 > 0:
        # Validar formato de COD_2_NOVEDAD para registros con COD_3_NOVEDAD = 5
        def validar_formato_cod2_sisben(valor):
            """Valida formato A01-C99 para códigos Sisbén"""
            if pd.isna(valor) or str(valor).strip() == '':
                return False
            
            valor_str = str(valor).strip().upper()
            
            # Debe tener exactamente 3 caracteres
            if len(valor_str) != 3:
                return False
            
            # Primer carácter debe ser A, B o C
            if valor_str[0] not in ['A', 'B', 'C']:
                return False
            
            # Los siguientes 2 caracteres deben ser dígitos
            if not valor_str[1:].isdigit():
                return False
            
            # El número debe estar entre 01 y 99
            numero = int(valor_str[1:])
            if not (1 <= numero <= 99):
                return False
            
            return True
        
        # Aplicar validación
        registros_cod3_5['COD_2_VALIDO'] = registros_cod3_5['COD_2_NOVEDAD'].apply(validar_formato_cod2_sisben)
        invalidos_cod2 = (~registros_cod3_5['COD_2_VALIDO']).sum()
        
        print(f"   📋 COD_2_NOVEDAD con formato válido: {registros_cod3_5['COD_2_VALIDO'].sum()}")
        print(f"   📋 COD_2_NOVEDAD con formato inválido: {invalidos_cod2}")
        
        if invalidos_cod2 > 0:
            # Mostrar ejemplos de valores inválidos
            print(f"\n   📋 Ejemplos de COD_2_NOVEDAD inválidos:")
            ejemplos_invalidos = registros_cod3_5[~registros_cod3_5['COD_2_VALIDO']][
                ['NUM_SOLICITUD_NOVEDAD', 'HST_IDN_NUMERO_IDENTIFICACION', 'COD_2_NOVEDAD', 'COD_3_NOVEDAD']
            ].head(5)
            print(ejemplos_invalidos.to_string(index=False))
            
            # Mover registros inválidos a DF_No_Envio
            registros_invalidos = registros_cod3_5[~registros_cod3_5['COD_2_VALIDO']].copy()
            registros_invalidos_clean = registros_invalidos.drop(columns=['COD_3_NOVEDAD_NORM', 'COD_2_VALIDO'])
            registros_invalidos_clean['motivo'] = "Datos incompletos en la novedad"
            
            df_no_envio = pd.concat([df_no_envio, registros_invalidos_clean], ignore_index=True)
            
            # Eliminar del DataFrame principal
            indices_invalidos = registros_n21[mask_cod3_es_5][~registros_cod3_5['COD_2_VALIDO']].index
            df = df.drop(indices_invalidos).reset_index(drop=True)
            
            # Actualizar registros_n21 para siguientes validaciones
            registros_n21 = registros_n21.drop(indices_invalidos)
            
            registros_invalidos_cod3_5 = invalidos_cod2
            print(f"   ✅ Movidos {invalidos_cod2} registros a DF_No_Envio por COD_2_NOVEDAD inválido")
    
    # =========================================================================
    # CORRECCIÓN 1: APLICAR CORRECCIONES PARA COD_3_NOVEDAD = 5
    # =========================================================================
    registros_n21_actualizados = df[df['NOVEDAD'] == 'N21'].copy()
    mask_cod3_es_5_actualizado = registros_n21_actualizados['COD_3_NOVEDAD'].astype(str).str.strip() == '5'
    
    correcciones_cod1 = 0
    correcciones_cod4 = 0
    
    if mask_cod3_es_5_actualizado.any():
        print(f"\n🔧 APLICANDO CORRECCIONES PARA COD_3_NOVEDAD = 5")
        
        # Corregir COD_1_NOVEDAD a "2"
        indices_cod3_5 = registros_n21_actualizados[mask_cod3_es_5_actualizado].index
        cod1_diferentes = df.loc[indices_cod3_5, 'COD_1_NOVEDAD'].astype(str).str.strip() != '2'
        correcciones_cod1 = cod1_diferentes.sum()
        
        df.loc[indices_cod3_5, 'COD_1_NOVEDAD'] = '2'
        
        print(f"   📋 COD_1_NOVEDAD corregidos a '2': {correcciones_cod1}")
        
        # Corregir COD_4_NOVEDAD basado en COD_2_NOVEDAD
        for idx in indices_cod3_5:
            cod2_valor = str(df.loc[idx, 'COD_2_NOVEDAD']).strip().upper()
            if len(cod2_valor) >= 1:
                primera_letra = cod2_valor[0]
                if primera_letra in ['A', 'B']:
                    nuevo_cod4 = '1'
                elif primera_letra == 'C':
                    nuevo_cod4 = '2'
                else:
                    nuevo_cod4 = '1'  # Por defecto
                
                if str(df.loc[idx, 'COD_4_NOVEDAD']).strip() != nuevo_cod4:
                    correcciones_cod4 += 1
                
                df.loc[idx, 'COD_4_NOVEDAD'] = nuevo_cod4
        
        print(f"   📋 COD_4_NOVEDAD corregidos: {correcciones_cod4}")
    
    # =========================================================================
    # CORRECCIÓN 2: APLICAR CORRECCIONES PARA COD_3_NOVEDAD ≠ 5
    # =========================================================================
    print(f"\n🔧 APLICANDO CORRECCIONES PARA COD_3_NOVEDAD ≠ 5")
    
    registros_n21_actualizados = df[df['NOVEDAD'] == 'N21'].copy()
    mask_cod3_no_es_5 = registros_n21_actualizados['COD_3_NOVEDAD'].astype(str).str.strip() != '5'
    
    correcciones_cod2_vacio = 0
    correcciones_cod1_a_3 = 0
    correcciones_cod4_a_n = 0
    
    if mask_cod3_no_es_5.any():
        indices_cod3_no_5 = registros_n21_actualizados[mask_cod3_no_es_5].index
        
        # Vaciar COD_2_NOVEDAD
        cod2_no_vacios = df.loc[indices_cod3_no_5, 'COD_2_NOVEDAD'].notna() & (df.loc[indices_cod3_no_5, 'COD_2_NOVEDAD'].astype(str).str.strip() != '')
        correcciones_cod2_vacio = cod2_no_vacios.sum()
        df.loc[indices_cod3_no_5, 'COD_2_NOVEDAD'] = ''
        
        # Corregir COD_1_NOVEDAD a "3"
        cod1_diferentes = df.loc[indices_cod3_no_5, 'COD_1_NOVEDAD'].astype(str).str.strip() != '3'
        correcciones_cod1_a_3 = cod1_diferentes.sum()
        df.loc[indices_cod3_no_5, 'COD_1_NOVEDAD'] = '3'
        
        # Corregir COD_4_NOVEDAD a "N"
        cod4_diferentes = df.loc[indices_cod3_no_5, 'COD_4_NOVEDAD'].astype(str).str.strip().str.upper() != 'N'
        correcciones_cod4_a_n = cod4_diferentes.sum()
        df.loc[indices_cod3_no_5, 'COD_4_NOVEDAD'] = 'N'
        
        print(f"   📋 COD_2_NOVEDAD vaciados: {correcciones_cod2_vacio}")
        print(f"   📋 COD_1_NOVEDAD corregidos a '3': {correcciones_cod1_a_3}")
        print(f"   📋 COD_4_NOVEDAD corregidos a 'N': {correcciones_cod4_a_n}")
    
    # =========================================================================
    # CORRECCIÓN 3: APLICAR CORRECCIONES PARA COD_5_NOVEDAD Y COD_6_NOVEDAD
    # =========================================================================
    print(f"\n🔧 APLICANDO CORRECCIONES PARA COD_5_NOVEDAD Y COD_6_NOVEDAD")
    
    registros_n21_final = df[df['NOVEDAD'] == 'N21'].copy()
    correcciones_cod5 = 0
    correcciones_cod6 = 0
    
    for idx in registros_n21_final.index:
        cod3_valor = str(df.loc[idx, 'COD_3_NOVEDAD']).strip()
        
        if cod3_valor == '17':
            # COD_5_NOVEDAD = "01" y COD_6_NOVEDAD = "999"
            if str(df.loc[idx, 'COD_5_NOVEDAD']).strip() != '01':
                correcciones_cod5 += 1
            if str(df.loc[idx, 'COD_6_NOVEDAD']).strip() != '999':
                correcciones_cod6 += 1
            
            df.loc[idx, 'COD_5_NOVEDAD'] = '01'
            df.loc[idx, 'COD_6_NOVEDAD'] = '999'
        else:
            # COD_5_NOVEDAD = "06"
            if str(df.loc[idx, 'COD_5_NOVEDAD']).strip() != '06':
                correcciones_cod5 += 1
            
            df.loc[idx, 'COD_5_NOVEDAD'] = '06'
    
    print(f"   📋 COD_5_NOVEDAD corregidos: {correcciones_cod5}")
    print(f"   📋 COD_6_NOVEDAD corregidos: {correcciones_cod6}")
    
    # =========================================================================
    # RESUMEN FINAL
    # =========================================================================
    registros_n21_final = len(df[df['NOVEDAD'] == 'N21'])
    total_correcciones = (correcciones_cod1 + correcciones_cod4 + correcciones_cod2_vacio + 
                         correcciones_cod1_a_3 + correcciones_cod4_a_n + correcciones_cod5 + correcciones_cod6)
    
    print(f"\n📈 RESUMEN FINAL:")
    print(f"   Registros N21 iniciales: {total_n21_inicial}")
    print(f"   Registros movidos (datos incompletos): {registros_invalidos_cod3_5}")
    print(f"   Registros N21 restantes: {registros_n21_final}")
    print(f"   Total de correcciones aplicadas: {total_correcciones}")
    print(f"   Total en DF_No_Envio actual: {len(df_no_envio)}")
    
    # Desglose de correcciones
    print(f"\n📋 DESGLOSE DE CORRECCIONES:")
    print(f"   COD_1_NOVEDAD → '2' (COD_3=5): {correcciones_cod1}")
    print(f"   COD_4_NOVEDAD según COD_2 (COD_3=5): {correcciones_cod4}")
    print(f"   COD_2_NOVEDAD vaciados (COD_3≠5): {correcciones_cod2_vacio}")
    print(f"   COD_1_NOVEDAD → '3' (COD_3≠5): {correcciones_cod1_a_3}")
    print(f"   COD_4_NOVEDAD → 'N' (COD_3≠5): {correcciones_cod4_a_n}")
    print(f"   COD_5_NOVEDAD corregidos: {correcciones_cod5}")
    print(f"   COD_6_NOVEDAD corregidos: {correcciones_cod6}")
    
    # Mostrar ejemplos de registros finales si hay
    if registros_n21_final > 0:
        print(f"\n📋 EJEMPLOS DE REGISTROS N21 PROCESADOS:")
        ejemplos_finales = df[df['NOVEDAD'] == 'N21'][[
            'NUM_SOLICITUD_NOVEDAD', 'HST_IDN_NUMERO_IDENTIFICACION',
            'COD_1_NOVEDAD', 'COD_2_NOVEDAD', 'COD_3_NOVEDAD', 
            'COD_4_NOVEDAD', 'COD_5_NOVEDAD', 'COD_6_NOVEDAD'
        ]].head(10)
        print(ejemplos_finales.to_string(index=False))
        
        # Mostrar distribución final de COD_3_NOVEDAD
        print(f"\n📊 Distribución final de COD_3_NOVEDAD:")
        distribucion_final = df[df['NOVEDAD'] == 'N21']['COD_3_NOVEDAD'].value_counts()
        for valor, count in distribucion_final.items():
            print(f"   {valor}: {count} registros")
    
    return df, df_no_envio

# Aplicar validación N21
df_NS, DF_No_Envio = validar_novedades_N21(df_NS, DF_No_Envio)

### 5.10.1. N21 veracidad

In [ ]:
def validar_veracidad_N21(df_ns, df_no_envio, maestro_adres):
    """
    Valida la veracidad de las novedades N21 verificando:
    1. Para COD_3_NOVEDAD = 5 (Sisbén):
       - Que Gr_Poblacional_Actual sea 5 (no listado censal ni "No sisben")
       - Que SUB_SISBEN_IV sea diferente a COD_2_NOVEDAD (hay cambio)
       - Que N_Sisben_Actual sea igual a COD_2_NOVEDAD (coincide con tablas de referencia)
    
    2. Para COD_3_NOVEDAD ≠ 5 (otros grupos poblacionales):
       - Que CNT_AFL_TPS_GRP_PBL_ID sea diferente a COD_3_NOVEDAD (hay cambio)
       - Que Gr_Poblacional_Actual sea igual a COD_3_NOVEDAD (coincide con tablas de referencia)
    
    Args:
        df_ns: DataFrame con novedades
        df_no_envio: DataFrame con registros rechazados
        maestro_adres: DataFrame maestro ADRES
    
    Returns:
        df_ns actualizado, df_no_envio actualizado
    """
    print("🔍 INICIANDO VALIDACIÓN DE VERACIDAD N21")
    print("="*80)
    
    # Filtrar registros N21
    mask_n21 = df_ns['NOVEDAD'] == 'N21'
    registros_n21 = df_ns[mask_n21].copy()
    total_n21_inicial = len(registros_n21)
    
    print(f"📊 Total de registros N21 a validar: {total_n21_inicial}")
    
    if total_n21_inicial == 0:
        print("⚠️ No hay registros N21 para validar")
        return df_ns, df_no_envio
    
    # =========================================================================
    # PASO 1: TRAER DATOS DEL MAESTRO ADRES
    # =========================================================================
    print(f"\n🔍 PASO 1: Trayendo datos de veracidad desde maestro_ADRES")
    
    maestro_veracidad = maestro_adres[[
        'ID_User', 'Gr_Poblacional_Actual', 'N_Sisben_Actual', 
        'SUB_SISBEN_IV', 'CNT_AFL_TPS_GRP_PBL_ID'
    ]].drop_duplicates(subset=['ID_User'])
    
    print(f"   📊 Registros únicos en maestro: {len(maestro_veracidad):,}")
    
    # Hacer merge
    registros_n21_con_adres = registros_n21.merge(
        maestro_veracidad,
        on='ID_User',
        how='left'
    )
    
    # Verificar match
    registros_con_match = registros_n21_con_adres['Gr_Poblacional_Actual'].notna().sum()
    registros_sin_match = registros_n21_con_adres['Gr_Poblacional_Actual'].isna().sum()
    
    print(f"   ✅ Registros con match en maestro: {registros_con_match}")
    print(f"   ⚠️ Registros sin match en maestro: {registros_sin_match}")
    
    # =========================================================================
    # PASO 2: VALIDAR REGISTROS CON COD_3_NOVEDAD = 5 (SISBÉN)
    # =========================================================================
    print(f"\n🔍 PASO 2: Validando registros Sisbén (COD_3_NOVEDAD = 5)")
    
    mask_sisben = registros_n21_con_adres['COD_3_NOVEDAD'].astype(str).str.strip() == '5'
    registros_sisben = registros_n21_con_adres[mask_sisben].copy()
    total_sisben = len(registros_sisben)
    
    print(f"   📊 Total registros Sisbén: {total_sisben}")
    
    registros_rechazados_sisben = []
    motivos_rechazo_sisben = {}
    
    if total_sisben > 0:
        # VALIDACIÓN 2.1: Verificar Gr_Poblacional_Actual
        print(f"\n   🔍 VALIDACIÓN 2.1: Grupo poblacional en ADRES")
        
        # Normalizar valores
        registros_sisben['Gr_Poblacional_Actual_NORM'] = registros_sisben['Gr_Poblacional_Actual'].astype(str).str.strip()
        
        # Listado censal (diferente de 5 y diferente de "No sisben")
        mask_listado_censal = (
            (registros_sisben['Gr_Poblacional_Actual_NORM'] != '5') &
            (registros_sisben['Gr_Poblacional_Actual_NORM'] != 'No sisben') &
            registros_sisben['Gr_Poblacional_Actual'].notna()
        )
        
        rechazados_listado = registros_sisben[mask_listado_censal].copy()
        total_listado = len(rechazados_listado)
        
        # No sisben
        mask_no_sisben = registros_sisben['Gr_Poblacional_Actual_NORM'] == 'No sisben'
        rechazados_no_sisben = registros_sisben[mask_no_sisben].copy()
        total_no_sisben = len(rechazados_no_sisben)
        
        print(f"     Listado censal (Gr_Poblacional ≠ 5): {total_listado}")
        print(f"     No sisben en ADRES: {total_no_sisben}")
        print(f"     Válidos (Gr_Poblacional = 5): {total_sisben - total_listado - total_no_sisben}")
        
        # Agregar rechazados de listado censal
        if total_listado > 0:
            rechazados_listado['motivo'] = 'Novedad no efectiva, afiliado listado censal'
            registros_rechazados_sisben.append(rechazados_listado)
            motivos_rechazo_sisben['Listado censal'] = total_listado
            
            print(f"\n     📋 Ejemplos de listados censales:")
            ejemplos = rechazados_listado[[
                'NUM_SOLICITUD_NOVEDAD', 'HST_IDN_NUMERO_IDENTIFICACION',
                'COD_2_NOVEDAD', 'Gr_Poblacional_Actual_NORM'
            ]].head(5)
            print(ejemplos.to_string(index=False))
        
        # Agregar rechazados sin sisben
        if total_no_sisben > 0:
            rechazados_no_sisben['motivo'] = 'Sisben no actualizado en tablas de referencia ADRES'
            registros_rechazados_sisben.append(rechazados_no_sisben)
            motivos_rechazo_sisben['No sisben en ADRES'] = total_no_sisben
            
            print(f"\n     📋 Ejemplos sin sisben en ADRES:")
            ejemplos = rechazados_no_sisben[[
                'NUM_SOLICITUD_NOVEDAD', 'HST_IDN_NUMERO_IDENTIFICACION',
                'COD_2_NOVEDAD', 'Gr_Poblacional_Actual_NORM'
            ]].head(5)
            print(ejemplos.to_string(index=False))
        
        # VALIDACIÓN 2.2: Para los que tienen Gr_Poblacional = 5, validar cambios
        print(f"\n   🔍 VALIDACIÓN 2.2: Validando cambios en grupo Sisbén")
        
        mask_gr_5 = registros_sisben['Gr_Poblacional_Actual_NORM'] == '5'
        registros_sisben_validos_gr = registros_sisben[mask_gr_5].copy()
        total_sisben_gr5 = len(registros_sisben_validos_gr)
        
        print(f"     Registros con Gr_Poblacional = 5: {total_sisben_gr5}")
        
        if total_sisben_gr5 > 0:
            # Normalizar valores para comparación
            registros_sisben_validos_gr['COD_2_NOVEDAD_NORM'] = registros_sisben_validos_gr['COD_2_NOVEDAD'].astype(str).str.strip().str.upper()
            registros_sisben_validos_gr['SUB_SISBEN_IV_NORM'] = registros_sisben_validos_gr['SUB_SISBEN_IV'].astype(str).str.strip().str.upper()
            registros_sisben_validos_gr['N_Sisben_Actual_NORM'] = registros_sisben_validos_gr['N_Sisben_Actual'].astype(str).str.strip().str.upper()
            
            # Sin cambios: COD_2_NOVEDAD == SUB_SISBEN_IV
            mask_sin_cambios = registros_sisben_validos_gr['COD_2_NOVEDAD_NORM'] == registros_sisben_validos_gr['SUB_SISBEN_IV_NORM']
            rechazados_sin_cambios = registros_sisben_validos_gr[mask_sin_cambios].copy()
            total_sin_cambios = len(rechazados_sin_cambios)
            
            # Diferente a tablas: COD_2_NOVEDAD != N_Sisben_Actual
            mask_dif_tablas = (
                (registros_sisben_validos_gr['COD_2_NOVEDAD_NORM'] != registros_sisben_validos_gr['N_Sisben_Actual_NORM']) &
                (~mask_sin_cambios)  # Excluir los ya rechazados por sin cambios
            )
            rechazados_dif_tablas = registros_sisben_validos_gr[mask_dif_tablas].copy()
            total_dif_tablas = len(rechazados_dif_tablas)
            
            print(f"     Sin cambios (COD_2 = SUB_SISBEN_IV): {total_sin_cambios}")
            print(f"     Diferente a tablas (COD_2 ≠ N_Sisben_Actual): {total_dif_tablas}")
            print(f"     Válidos: {total_sisben_gr5 - total_sin_cambios - total_dif_tablas}")
            
            # Agregar rechazados sin cambios
            if total_sin_cambios > 0:
                rechazados_sin_cambios['motivo'] = 'Afiliado sin cambios a reportar'
                registros_rechazados_sisben.append(rechazados_sin_cambios)
                motivos_rechazo_sisben['Sin cambios Sisbén'] = total_sin_cambios
                
                print(f"\n     📋 Ejemplos sin cambios:")
                ejemplos = rechazados_sin_cambios[[
                    'NUM_SOLICITUD_NOVEDAD', 'HST_IDN_NUMERO_IDENTIFICACION',
                    'COD_2_NOVEDAD_NORM', 'SUB_SISBEN_IV_NORM'
                ]].head(5)
                print(ejemplos.to_string(index=False))
            
            # Agregar rechazados diferente a tablas
            if total_dif_tablas > 0:
                rechazados_dif_tablas['motivo'] = 'Diferente a tablas de referencia ADRES'
                registros_rechazados_sisben.append(rechazados_dif_tablas)
                motivos_rechazo_sisben['Diferente a tablas Sisbén'] = total_dif_tablas
                
                print(f"\n     📋 Ejemplos diferentes a tablas:")
                ejemplos = rechazados_dif_tablas[[
                    'NUM_SOLICITUD_NOVEDAD', 'HST_IDN_NUMERO_IDENTIFICACION',
                    'COD_2_NOVEDAD_NORM', 'N_Sisben_Actual_NORM'
                ]].head(5)
                print(ejemplos.to_string(index=False))
    
    # =========================================================================
    # PASO 3: VALIDAR REGISTROS CON COD_3_NOVEDAD ≠ 5 (OTROS GRUPOS)
    # =========================================================================
    print(f"\n🔍 PASO 3: Validando otros grupos poblacionales (COD_3_NOVEDAD ≠ 5)")
    
    mask_otros_grupos = registros_n21_con_adres['COD_3_NOVEDAD'].astype(str).str.strip() != '5'
    registros_otros = registros_n21_con_adres[mask_otros_grupos].copy()
    total_otros = len(registros_otros)
    
    print(f"   📊 Total otros grupos: {total_otros}")
    
    registros_rechazados_otros = []
    motivos_rechazo_otros = {}
    
    if total_otros > 0:
        # Normalizar valores
        registros_otros['COD_3_NOVEDAD_NORM'] = registros_otros['COD_3_NOVEDAD'].astype(str).str.strip()
        registros_otros['CNT_AFL_TPS_GRP_PBL_ID_NORM'] = registros_otros['CNT_AFL_TPS_GRP_PBL_ID'].astype(str).str.strip()
        registros_otros['Gr_Poblacional_Actual_NORM'] = registros_otros['Gr_Poblacional_Actual'].astype(str).str.strip()
        
        # Sin cambios: COD_3_NOVEDAD == CNT_AFL_TPS_GRP_PBL_ID
        mask_sin_cambios_otros = registros_otros['COD_3_NOVEDAD_NORM'] == registros_otros['CNT_AFL_TPS_GRP_PBL_ID_NORM']
        rechazados_sin_cambios_otros = registros_otros[mask_sin_cambios_otros].copy()
        total_sin_cambios_otros = len(rechazados_sin_cambios_otros)
        
        # Diferente a tablas: COD_3_NOVEDAD != Gr_Poblacional_Actual
        mask_dif_tablas_otros = (
            (registros_otros['COD_3_NOVEDAD_NORM'] != registros_otros['Gr_Poblacional_Actual_NORM']) &
            (~mask_sin_cambios_otros)  # Excluir ya rechazados
        )
        rechazados_dif_tablas_otros = registros_otros[mask_dif_tablas_otros].copy()
        total_dif_tablas_otros = len(rechazados_dif_tablas_otros)
        
        print(f"   Sin cambios (COD_3 = CNT_AFL_TPS_GRP_PBL_ID): {total_sin_cambios_otros}")
        print(f"   Diferente a tablas (COD_3 ≠ Gr_Poblacional_Actual): {total_dif_tablas_otros}")
        print(f"   Válidos: {total_otros - total_sin_cambios_otros - total_dif_tablas_otros}")
        
        # Agregar rechazados sin cambios
        if total_sin_cambios_otros > 0:
            rechazados_sin_cambios_otros['motivo'] = 'Afiliado sin cambios a reportar'
            registros_rechazados_otros.append(rechazados_sin_cambios_otros)
            motivos_rechazo_otros['Sin cambios grupo poblacional'] = total_sin_cambios_otros
            
            print(f"\n   📋 Ejemplos sin cambios:")
            ejemplos = rechazados_sin_cambios_otros[[
                'NUM_SOLICITUD_NOVEDAD', 'HST_IDN_NUMERO_IDENTIFICACION',
                'COD_3_NOVEDAD_NORM', 'CNT_AFL_TPS_GRP_PBL_ID_NORM'
            ]].head(5)
            print(ejemplos.to_string(index=False))
        
        # Agregar rechazados diferente a tablas
        if total_dif_tablas_otros > 0:
            rechazados_dif_tablas_otros['motivo'] = 'Diferente a tablas de referencia ADRES'
            registros_rechazados_otros.append(rechazados_dif_tablas_otros)
            motivos_rechazo_otros['Diferente a tablas grupo poblacional'] = total_dif_tablas_otros
            
            print(f"\n   📋 Ejemplos diferentes a tablas:")
            ejemplos = rechazados_dif_tablas_otros[[
                'NUM_SOLICITUD_NOVEDAD', 'HST_IDN_NUMERO_IDENTIFICACION',
                'COD_3_NOVEDAD_NORM', 'Gr_Poblacional_Actual_NORM'
            ]].head(5)
            print(ejemplos.to_string(index=False))
    
    # =========================================================================
    # PASO 4: CONSOLIDAR Y MOVER RECHAZADOS
    # =========================================================================
    print(f"\n🔄 PASO 4: Consolidando registros rechazados")
    
    # Combinar todos los rechazados
    todos_rechazados = []
    
    if registros_rechazados_sisben:
        todos_rechazados.extend(registros_rechazados_sisben)
    
    if registros_rechazados_otros:
        todos_rechazados.extend(registros_rechazados_otros)
    
    total_rechazados = 0
    
    if todos_rechazados:
        # Combinar en un solo DataFrame
        df_rechazados = pd.concat(todos_rechazados, ignore_index=True)
        total_rechazados = len(df_rechazados)
        
        print(f"   📊 Total de registros a rechazar: {total_rechazados}")
        
        # Limpiar columnas temporales
        columnas_temp = [
            'Gr_Poblacional_Actual', 'N_Sisben_Actual', 'SUB_SISBEN_IV', 'CNT_AFL_TPS_GRP_PBL_ID',
            'Gr_Poblacional_Actual_NORM', 'COD_2_NOVEDAD_NORM', 'SUB_SISBEN_IV_NORM', 
            'N_Sisben_Actual_NORM', 'COD_3_NOVEDAD_NORM', 'CNT_AFL_TPS_GRP_PBL_ID_NORM'
        ]
        
        df_rechazados = df_rechazados.drop(columns=[col for col in columnas_temp if col in df_rechazados.columns])
        
        # Agregar a DF_No_Envio
        df_no_envio = pd.concat([df_no_envio, df_rechazados], ignore_index=True)
        
        # Eliminar del DataFrame principal usando ID_User
        ids_a_eliminar = df_rechazados['ID_User'].unique()
        mask_eliminar = (df_ns['NOVEDAD'] == 'N21') & (df_ns['ID_User'].isin(ids_a_eliminar))
        df_ns = df_ns[~mask_eliminar].reset_index(drop=True)
        
        print(f"   ✅ Registros movidos a DF_No_Envio: {total_rechazados}")
    else:
        print(f"   ✅ No hay registros para rechazar")
    
    # =========================================================================
    # PASO 5: ESTADÍSTICAS FINALES
    # =========================================================================
    registros_n21_final = len(df_ns[df_ns['NOVEDAD'] == 'N21'])
    
    print(f"\n📈 RESUMEN FINAL DE VERACIDAD N21:")
    print("="*60)
    print(f"   Registros N21 iniciales: {total_n21_inicial}")
    print(f"   Registros rechazados: {total_rechazados}")
    print(f"   Registros N21 válidos finales: {registros_n21_final}")
    print(f"   Total en DF_No_Envio: {len(df_no_envio)}")
    
    # Desglose por motivo
    if motivos_rechazo_sisben or motivos_rechazo_otros:
        print(f"\n📋 DESGLOSE POR MOTIVO DE RECHAZO:")
        
        if motivos_rechazo_sisben:
            print(f"   🔹 Registros Sisbén (COD_3 = 5):")
            for motivo, cantidad in motivos_rechazo_sisben.items():
                print(f"     • {motivo}: {cantidad}")
        
        if motivos_rechazo_otros:
            print(f"   🔹 Otros grupos poblacionales (COD_3 ≠ 5):")
            for motivo, cantidad in motivos_rechazo_otros.items():
                print(f"     • {motivo}: {cantidad}")
    
    # Mostrar ejemplos finales válidos
    if registros_n21_final > 0:
        print(f"\n📋 EJEMPLOS DE REGISTROS N21 VÁLIDOS FINALES:")
        ejemplos_validos = df_ns[df_ns['NOVEDAD'] == 'N21'][[
            'NUM_SOLICITUD_NOVEDAD', 'HST_IDN_NUMERO_IDENTIFICACION',
            'COD_1_NOVEDAD', 'COD_2_NOVEDAD', 'COD_3_NOVEDAD'
        ]].head(10)
        print(ejemplos_validos.to_string(index=False))
    
    print(f"\n✅ VALIDACIÓN DE VERACIDAD N21 COMPLETADA")
    
    return df_ns, df_no_envio

# Aplicar validación de veracidad N21
df_NS, DF_No_Envio = validar_veracidad_N21(df_NS, DF_No_Envio, maestro_ADRES)

### 5.10.2. N21 masiva Listado censal

In [ ]:
def aplicar_actualizaciones_masivas_N21(df_ns, maestro_adres, fecha_reporte):
    """
    Aplica actualizaciones masivas N21 (actualización de condiciones Sisbén, etnia y comunidad).
    
    Proceso:
    1. CASO 1: Listado censal (Gr_Poblacional_Actual = 1, 2, 9, 16, 28)
       - Afiliados con TPS_GRP_PBL_ID = 5 (o vacío) en ADRES
       - Pero con Gr_Poblacional_Actual de listado censal
       - Reportar N21 para cambiar a listado censal
    
    2. CASO 2: Grupo poblacional 17 (indígenas con código especial)
       - TPS_GRP_PBL_ID = 5 y Gr_Poblacional_Actual = 17
       - Reportar con códigos especiales (COD_5 = "01", COD_6 = "999")
    
    Args:
        df_ns: DataFrame de novedades
        maestro_adres: DataFrame maestro ADRES
        fecha_reporte: fecha del reporte (datetime)
    
    Returns:
        df_ns actualizado con las nuevas novedades N21
    """
    print("🔍 INICIANDO APLICACIÓN DE ACTUALIZACIONES MASIVAS N21")
    print("="*80)
    
    # Convertir fecha_reporte a formato DD/MM/YYYY
    fecha_novedad_str = fecha_reporte.strftime('%d/%m/%Y')
    print(f"📅 Fecha de novedad: {fecha_novedad_str}")
    
    # =========================================================================
    # PASO 1: FILTRAR AFILIADOS ACTIVOS
    # =========================================================================
    print(f"\n🔍 PASO 1: Identificando afiliados activos (AC) en EPS025")
    
    mask_activos_eps025 = (
        (maestro_adres['ENT_ID'] == 'EPS025') &
        (maestro_adres['TPS_EST_AFL_ID'] == 'AC')
    )
    
    registros_activos = maestro_adres[mask_activos_eps025].copy().reset_index(drop=True)
    total_activos = len(registros_activos)
    
    print(f"   📊 Registros activos en EPS025: {total_activos:,}")
    
    if total_activos == 0:
        print(f"\n⚠️ No se encontraron registros activos para procesar")
        return df_ns
    
    # =========================================================================
    # CASO 1: LISTADO CENSAL (Gr_Poblacional_Actual = 1, 2, 9, 16, 28)
    # =========================================================================
    print(f"\n🔍 CASO 1: Procesando afiliados de listado censal")
    
    # Filtrar por TPS_GRP_PBL_ID = 5 o vacío/null
    mask_sisben_o_vacio = (
        (registros_activos['TPS_GRP_PBL_ID'].isna()) |
        (registros_activos['TPS_GRP_PBL_ID'].astype(str).str.strip() == '') |
        (registros_activos['TPS_GRP_PBL_ID'].astype(str).str.strip() == '5')
    )
    
    # Filtrar por Gr_Poblacional_Actual de listado censal
    grupos_listado_censal = ['1', '2', '9', '16', '28']
    mask_listado_censal = registros_activos['Gr_Poblacional_Actual'].astype(str).str.strip().isin(grupos_listado_censal)
    
    # Combinar máscaras
    mask_caso1 = mask_sisben_o_vacio & mask_listado_censal
    registros_caso1 = registros_activos[mask_caso1].copy().reset_index(drop=True)
    total_caso1 = len(registros_caso1)
    
    print(f"   📊 Registros encontrados: {total_caso1:,}")
    
    if total_caso1 > 0:
        # Mostrar distribución por grupo poblacional
        print(f"\n   📊 Distribución por grupo poblacional:")
        distribucion_gr_caso1 = registros_caso1['Gr_Poblacional_Actual'].value_counts()
        for gr, count in distribucion_gr_caso1.items():
            print(f"     Grupo {gr}: {count:,} registros")
    
    # =========================================================================
    # CASO 2: GRUPO POBLACIONAL 17 (INDÍGENAS CON CÓDIGO ESPECIAL)
    # =========================================================================
    print(f"\n🔍 CASO 2: Procesando grupo poblacional 17 (indígenas)")
    
    mask_caso2 = (
        (registros_activos['TPS_GRP_PBL_ID'].astype(str).str.strip() == '5') &
        (registros_activos['Gr_Poblacional_Actual'].astype(str).str.strip() == '17')
    )
    
    registros_caso2 = registros_activos[mask_caso2].copy().reset_index(drop=True)
    total_caso2 = len(registros_caso2)
    
    print(f"   📊 Registros encontrados: {total_caso2:,}")
    
    # =========================================================================
    # PASO 2: CONSTRUIR NOVEDADES N21 - CASO 1 (LISTADO CENSAL)
    # =========================================================================
    dataframes_nuevos = []
    
    if total_caso1 > 0:
        print(f"\n🔧 CONSTRUYENDO NOVEDADES N21 - CASO 1 (Listado Censal)")
        
        # Verificar columnas necesarias
        columnas_requeridas = [
            'ENT_ID', 'TPS_IDN_ID', 'HST_IDN_NUMERO_IDENTIFICACION',
            'AFL_PRIMER_APELLIDO', 'AFL_SEGUNDO_APELLIDO', 
            'AFL_PRIMER_NOMBRE', 'AFL_SEGUNDO_NOMBRE',
            'AFL_FECHA_NACIMIENTO', 'DPR_ID', 'MNC_ID', 
            'TPS_EST_AFL_ID', 'Gr_Poblacional_Actual'
        ]
        
        columnas_faltantes = [col for col in columnas_requeridas if col not in registros_caso1.columns]
        if columnas_faltantes:
            print(f"❌ Columnas faltantes en registros_caso1: {columnas_faltantes}")
        else:
            try:
                # Construir DataFrame de novedades para CASO 1
                nuevas_novedades_caso1 = pd.DataFrame({
                    'ENT_ID': registros_caso1['ENT_ID'],
                    'TPS_IDN_ID': registros_caso1['TPS_IDN_ID'],
                    'HST_IDN_NUMERO_IDENTIFICACION': registros_caso1['HST_IDN_NUMERO_IDENTIFICACION'],
                    'AFL_PRIMER_APELLIDO': registros_caso1['AFL_PRIMER_APELLIDO'],
                    'AFL_SEGUNDO_APELLIDO': registros_caso1['AFL_SEGUNDO_APELLIDO'],
                    'AFL_PRIMER_NOMBRE': registros_caso1['AFL_PRIMER_NOMBRE'],
                    'AFL_SEGUNDO_NOMBRE': registros_caso1['AFL_SEGUNDO_NOMBRE'],
                    'AFL_FECHA_NACIMIENTO': registros_caso1['AFL_FECHA_NACIMIENTO'],
                    'DPR_ID': registros_caso1['DPR_ID'],
                    'MNS_ID': registros_caso1['MNC_ID'],  # MNC_ID -> MNS_ID
                    'NOVEDAD': 'N21',
                    'FECHA_NOVEDAD': fecha_novedad_str,
                    'COD_1_NOVEDAD': '3',  # Grupo poblacional
                    'COD_2_NOVEDAD': '',   # Vacío para listado censal
                    'COD_3_NOVEDAD': registros_caso1['Gr_Poblacional_Actual'].astype(str).str.strip(),
                    'COD_4_NOVEDAD': 'N',  # N para grupos no Sisbén
                    'COD_5_NOVEDAD': '06', # Código de etnia por defecto
                    'ENT_ID_ADRES': registros_caso1['ENT_ID'],
                    'TPS_EST_AFL_ID_from_adres': registros_caso1['TPS_EST_AFL_ID'],
                    'Where': 'Novedad Masiva'
                })
                
                # Crear ID_User e ID_Register
                nuevas_novedades_caso1['ID_User'] = (
                    nuevas_novedades_caso1['TPS_IDN_ID'].astype(str) + 
                    nuevas_novedades_caso1['HST_IDN_NUMERO_IDENTIFICACION'].astype(str)
                )
                
                nuevas_novedades_caso1['ID_Register'] = (
                    nuevas_novedades_caso1['TPS_IDN_ID'].astype(str) + 
                    nuevas_novedades_caso1['HST_IDN_NUMERO_IDENTIFICACION'].astype(str) + 
                    nuevas_novedades_caso1['NOVEDAD'].astype(str)
                )
                
                # Rellenar columnas faltantes
                columnas_faltantes_df = [col for col in df_ns.columns if col not in nuevas_novedades_caso1.columns]
                for col in columnas_faltantes_df:
                    nuevas_novedades_caso1[col] = None
                
                # Ordenar columnas
                nuevas_novedades_caso1 = nuevas_novedades_caso1[df_ns.columns]
                
                dataframes_nuevos.append(nuevas_novedades_caso1)
                
                print(f"   ✅ Novedades N21 CASO 1 construidas: {len(nuevas_novedades_caso1):,}")
                
                # Mostrar ejemplos
                print(f"\n   📋 Ejemplos de novedades CASO 1:")
                ejemplos_columnas = ['TPS_IDN_ID', 'HST_IDN_NUMERO_IDENTIFICACION', 
                                   'AFL_PRIMER_NOMBRE', 'AFL_PRIMER_APELLIDO',
                                   'COD_1_NOVEDAD', 'COD_2_NOVEDAD', 'COD_3_NOVEDAD', 
                                   'COD_4_NOVEDAD', 'COD_5_NOVEDAD']
                ejemplos_disponibles = [col for col in ejemplos_columnas if col in nuevas_novedades_caso1.columns]
                ejemplos = nuevas_novedades_caso1[ejemplos_disponibles].head(10)
                print(ejemplos.to_string(index=False))
                
            except Exception as e:
                print(f"❌ Error al construir novedades CASO 1: {e}")
    
    # =========================================================================
    # PASO 3: CONSTRUIR NOVEDADES N21 - CASO 2 (GRUPO 17)
    # =========================================================================
    if total_caso2 > 0:
        print(f"\n🔧 CONSTRUYENDO NOVEDADES N21 - CASO 2 (Grupo 17 - Indígenas)")
        
        # Verificar columnas necesarias
        columnas_requeridas = [
            'ENT_ID', 'TPS_IDN_ID', 'HST_IDN_NUMERO_IDENTIFICACION',
            'AFL_PRIMER_APELLIDO', 'AFL_SEGUNDO_APELLIDO', 
            'AFL_PRIMER_NOMBRE', 'AFL_SEGUNDO_NOMBRE',
            'AFL_FECHA_NACIMIENTO', 'DPR_ID', 'MNC_ID', 
            'TPS_EST_AFL_ID', 'Gr_Poblacional_Actual'
        ]
        
        columnas_faltantes = [col for col in columnas_requeridas if col not in registros_caso2.columns]
        if columnas_faltantes:
            print(f"❌ Columnas faltantes en registros_caso2: {columnas_faltantes}")
        else:
            try:
                # Construir DataFrame de novedades para CASO 2
                nuevas_novedades_caso2 = pd.DataFrame({
                    'ENT_ID': registros_caso2['ENT_ID'],
                    'TPS_IDN_ID': registros_caso2['TPS_IDN_ID'],
                    'HST_IDN_NUMERO_IDENTIFICACION': registros_caso2['HST_IDN_NUMERO_IDENTIFICACION'],
                    'AFL_PRIMER_APELLIDO': registros_caso2['AFL_PRIMER_APELLIDO'],
                    'AFL_SEGUNDO_APELLIDO': registros_caso2['AFL_SEGUNDO_APELLIDO'],
                    'AFL_PRIMER_NOMBRE': registros_caso2['AFL_PRIMER_NOMBRE'],
                    'AFL_SEGUNDO_NOMBRE': registros_caso2['AFL_SEGUNDO_NOMBRE'],
                    'AFL_FECHA_NACIMIENTO': registros_caso2['AFL_FECHA_NACIMIENTO'],
                    'DPR_ID': registros_caso2['DPR_ID'],
                    'MNS_ID': registros_caso2['MNC_ID'],  # MNC_ID -> MNS_ID
                    'NOVEDAD': 'N21',
                    'FECHA_NOVEDAD': fecha_novedad_str,
                    'COD_1_NOVEDAD': '3',  # Grupo poblacional
                    'COD_2_NOVEDAD': '',   # Vacío para grupo 17
                    'COD_3_NOVEDAD': '17', # Grupo indígena
                    'COD_4_NOVEDAD': 'N',  # N para grupos no Sisbén
                    'COD_5_NOVEDAD': '01', # Código especial para grupo 17
                    'COD_6_NOVEDAD': '999',# Código especial para grupo 17
                    'ENT_ID_ADRES': registros_caso2['ENT_ID'],
                    'TPS_EST_AFL_ID_from_adres': registros_caso2['TPS_EST_AFL_ID'],
                    'Where': 'Novedad Masiva'
                })
                
                # Crear ID_User e ID_Register
                nuevas_novedades_caso2['ID_User'] = (
                    nuevas_novedades_caso2['TPS_IDN_ID'].astype(str) + 
                    nuevas_novedades_caso2['HST_IDN_NUMERO_IDENTIFICACION'].astype(str)
                )
                
                nuevas_novedades_caso2['ID_Register'] = (
                    nuevas_novedades_caso2['TPS_IDN_ID'].astype(str) + 
                    nuevas_novedades_caso2['HST_IDN_NUMERO_IDENTIFICACION'].astype(str) + 
                    nuevas_novedades_caso2['NOVEDAD'].astype(str)
                )
                
                # Rellenar columnas faltantes
                columnas_faltantes_df = [col for col in df_ns.columns if col not in nuevas_novedades_caso2.columns]
                for col in columnas_faltantes_df:
                    nuevas_novedades_caso2[col] = None
                
                # Ordenar columnas
                nuevas_novedades_caso2 = nuevas_novedades_caso2[df_ns.columns]
                
                dataframes_nuevos.append(nuevas_novedades_caso2)
                
                print(f"   ✅ Novedades N21 CASO 2 construidas: {len(nuevas_novedades_caso2):,}")
                
                # Mostrar ejemplos
                print(f"\n   📋 Ejemplos de novedades CASO 2:")
                ejemplos_columnas = ['TPS_IDN_ID', 'HST_IDN_NUMERO_IDENTIFICACION', 
                                   'AFL_PRIMER_NOMBRE', 'AFL_PRIMER_APELLIDO',
                                   'COD_1_NOVEDAD', 'COD_3_NOVEDAD', 
                                   'COD_4_NOVEDAD', 'COD_5_NOVEDAD', 'COD_6_NOVEDAD']
                ejemplos_disponibles = [col for col in ejemplos_columnas if col in nuevas_novedades_caso2.columns]
                ejemplos = nuevas_novedades_caso2[ejemplos_disponibles].head(10)
                print(ejemplos.to_string(index=False))
                
            except Exception as e:
                print(f"❌ Error al construir novedades CASO 2: {e}")
    
    # =========================================================================
    # PASO 4: AGREGAR AL DATAFRAME PRINCIPAL
    # =========================================================================
    if len(dataframes_nuevos) == 0:
        print(f"\n⚠️ No se generaron novedades N21 masivas")
        return df_ns
    
    print(f"\n🔄 PASO 4: AGREGANDO NOVEDADES A df_NS")
    
    try:
        # Combinar todos los casos
        todas_nuevas_novedades = pd.concat(dataframes_nuevos, ignore_index=True)
        total_nuevas_novedades = len(todas_nuevas_novedades)
        
        print(f"   📊 Total de novedades N21 masivas a agregar: {total_nuevas_novedades:,}")
        
        # Agregar al DataFrame principal
        registros_antes = len(df_ns)
        df_ns = pd.concat([df_ns, todas_nuevas_novedades], ignore_index=True)
        registros_despues = len(df_ns)
        
        print(f"   📊 Registros antes: {registros_antes:,}")
        print(f"   📊 Registros después: {registros_despues:,}")
        print(f"   📊 Registros agregados: {registros_despues - registros_antes:,}")
        
    except Exception as e:
        print(f"❌ Error al agregar novedades a df_NS: {e}")
        return df_ns
    
    # =========================================================================
    # PASO 5: ESTADÍSTICAS FINALES
    # =========================================================================
    print(f"\n📈 ESTADÍSTICAS DE NOVEDADES N21 AGREGADAS:")
    
    try:
        if 'Where' in df_ns.columns:
            novedades_n21_masivas = df_ns[
                (df_ns['Where'] == 'Novedad Masiva') & 
                (df_ns['NOVEDAD'] == 'N21')
            ]
            
            if len(novedades_n21_masivas) > 0:
                print(f"   📊 Total novedades N21 masivas: {len(novedades_n21_masivas):,}")
                
                # Distribución por COD_3_NOVEDAD (grupo poblacional)
                print(f"\n   📊 Por grupo poblacional (COD_3_NOVEDAD):")
                distribucion_cod3 = novedades_n21_masivas['COD_3_NOVEDAD'].value_counts()
                for cod3, count in distribucion_cod3.items():
                    print(f"     Grupo {cod3}: {count:,} registros")
                
                # Distribución por tipo de documento
                print(f"\n   📊 Por tipo de documento:")
                distribucion_tps = novedades_n21_masivas['TPS_IDN_ID'].value_counts().head(10)
                for tipo_doc, count in distribucion_tps.items():
                    print(f"     {tipo_doc}: {count:,} registros")
                
                # Casos con COD_6_NOVEDAD (grupo 17)
                casos_cod6 = novedades_n21_masivas['COD_6_NOVEDAD'].notna().sum()
                if casos_cod6 > 0:
                    print(f"\n   📊 Registros con COD_6_NOVEDAD (Grupo 17): {casos_cod6:,}")
                
            else:
                print(f"   ⚠️ No se encontraron novedades N21 masivas en el DataFrame final")
        else:
            print(f"   ⚠️ Columna 'Where' no encontrada en df_NS")
            
    except Exception as e:
        print(f"⚠️ Error al generar estadísticas finales: {e}")
    
    # =========================================================================
    # PASO 6: VERIFICACIÓN DE CALIDAD
    # =========================================================================
    print(f"\n🔍 VERIFICACIÓN DE CALIDAD:")
    
    try:
        novedades_n21_masivas = df_ns[
            (df_ns['Where'] == 'Novedad Masiva') & 
            (df_ns['NOVEDAD'] == 'N21')
        ]
        
        if len(novedades_n21_masivas) > 0:
            # Verificar COD_1_NOVEDAD = '3'
            cod1_correctos = novedades_n21_masivas['COD_1_NOVEDAD'].eq('3').sum()
            print(f"   ✅ COD_1_NOVEDAD = '3': {cod1_correctos}/{len(novedades_n21_masivas)}")
            
            # Verificar COD_2_NOVEDAD vacío
            cod2_vacios = (novedades_n21_masivas['COD_2_NOVEDAD'].isna() | 
                          (novedades_n21_masivas['COD_2_NOVEDAD'] == '')).sum()
            print(f"   ✅ COD_2_NOVEDAD vacío: {cod2_vacios}/{len(novedades_n21_masivas)}")
            
            # Verificar COD_4_NOVEDAD = 'N'
            cod4_correctos = novedades_n21_masivas['COD_4_NOVEDAD'].eq('N').sum()
            print(f"   ✅ COD_4_NOVEDAD = 'N': {cod4_correctos}/{len(novedades_n21_masivas)}")
            
            # Verificar COD_5_NOVEDAD
            cod5_validos = novedades_n21_masivas['COD_5_NOVEDAD'].isin(['06', '01']).sum()
            print(f"   ✅ COD_5_NOVEDAD válido ('06' o '01'): {cod5_validos}/{len(novedades_n21_masivas)}")
            
            # Verificar estado AC
            estados_activos = novedades_n21_masivas['TPS_EST_AFL_ID_from_adres'].eq('AC').sum()
            print(f"   ✅ Estado AC: {estados_activos}/{len(novedades_n21_masivas)}")
            
        else:
            print(f"   ⚠️ No hay novedades N21 masivas para verificar")
            
    except Exception as e:
        print(f"⚠️ Error en verificación de calidad: {e}")
    
    print(f"\n✅ ACTUALIZACIONES MASIVAS N21 APLICADAS EXITOSAMENTE")
    
    return df_ns

# Aplicar actualizaciones masivas N21 Listado censal
df_NS = aplicar_actualizaciones_masivas_N21(df_NS, maestro_ADRES, fecha_reporte)

## 5.10.3. N21 Masiva sisben

In [ ]:
def aplicar_actualizaciones_sisben_N21(df_ns, maestro_adres, fecha_reporte):
    """
    Aplica actualizaciones masivas N21 para corregir el Sisbén IV desactualizado 
    en ADRES (SUB_SISBEN_IV) utilizando el valor local (N_Sisben_Actual).
    
    Proceso:
    1. Filtra afiliados Activos (AC) en EPS025.
    2. Identifica registros que cumplen las condiciones de Sisbén desactualizado:
        - TPS_GRP_PBL_ID = '5' o vacío/null (Regímenes Subsidiados/No clasificados)
        - N_Sisben_Actual NO está vacío (Sisbén local válido)
        - N_Sisben_Actual es DIFERENTE a SUB_SISBEN_IV (Valor desactualizado en ADRES)
    3. Construye la novedad N21 con los códigos Sisbén correctos.
        - COD_1_NOVEDAD = '5' (Código Sisbén)
        - COD_3_NOVEDAD = N_Sisben_Actual
        - COD_4_NOVEDAD: '1' (Grupos A y B) o '2' (Grupo C)
    
    Args:
        df_ns: DataFrame de novedades existente (donde se añadirán las nuevas novedades)
        maestro_adres: DataFrame maestro con la información de afiliados y Sisbén.
        fecha_reporte: fecha del reporte (datetime)
    
    Returns:
        df_ns actualizado con las nuevas novedades N21
    """
    print("🔍 INICIANDO APLICACIÓN DE ACTUALIZACIONES SISBÉN N21")
    print("="*80)
    
    # Convertir fecha_reporte a formato DD/MM/YYYY
    fecha_novedad_str = fecha_reporte.strftime('%d/%m/%Y')
    print(f"📅 Fecha de novedad: {fecha_novedad_str}")
    
    # =========================================================================
    # PASO 1: FILTRAR AFILIADOS ACTIVOS EN EPS025
    # =========================================================================
    print(f"\n🔍 PASO 1: Identificando afiliados activos (AC) en EPS025")
    
    # Asegurar que las columnas de Sisbén sean strings para evitar errores de comparación
    maestro_adres['TPS_GRP_PBL_ID'] = maestro_adres['TPS_GRP_PBL_ID'].astype(str).str.strip()
    maestro_adres['N_Sisben_Actual'] = maestro_adres['N_Sisben_Actual'].astype(str).str.strip().str.upper()
    maestro_adres['SUB_SISBEN_IV'] = maestro_adres['SUB_SISBEN_IV'].astype(str).str.strip().str.upper()
    
    mask_activos_eps025 = (
        (maestro_adres['ENT_ID'] == 'EPS025') &
        (maestro_adres['TPS_EST_AFL_ID'] == 'AC')
    )
    
    registros_activos = maestro_adres[mask_activos_eps025].copy().reset_index(drop=True)
    total_activos = len(registros_activos)
    
    print(f"   📊 Registros activos en EPS025: {total_activos:,}")
    
    if total_activos == 0:
        print(f"\n⚠️ No se encontraron registros activos para procesar")
        return df_ns
    
    # =========================================================================
    # CASO ÚNICO: SISBÉN DESACTUALIZADO
    # =========================================================================
    print(f"\n🔍 CASO SISBÉN: Identificando afiliados con Sisbén desactualizado")
    
    # Condición 1: TPS_GRP_PBL_ID = '5' o vacío/nulo
    mask_regimen_sisben = (
        (registros_activos['TPS_GRP_PBL_ID'].isna()) |
        (registros_activos['TPS_GRP_PBL_ID'] == '') |
        (registros_activos['TPS_GRP_PBL_ID'] == '5')
    )
    
    # Condición 2: N_Sisben_Actual NO está vacío (tiene valor local)
    mask_sisben_local_valido = (registros_activos['N_Sisben_Actual'] != '')
    
    # Condición 3: N_Sisben_Actual es DIFERENTE a SUB_SISBEN_IV (ADRES desactualizado)
    # Rellenamos los vacíos de SUB_SISBEN_IV con un valor de control (ej: 'VacioADRES') para que la comparación funcione
    mask_sisben_desactualizado = (
        registros_activos['N_Sisben_Actual'] != registros_activos['SUB_SISBEN_IV'].fillna('VacioADRES')
    )
    
    # Combinar todas las máscaras
    mask_caso_sisben = mask_regimen_sisben & mask_sisben_local_valido & mask_sisben_desactualizado
    
    registros_caso_sisben = registros_activos[mask_caso_sisben].copy().reset_index(drop=True)
    total_caso_sisben = len(registros_caso_sisben)
    
    print(f"   📊 Registros con Sisbén desactualizado a corregir: {total_caso_sisben:,}")
    
    if total_caso_sisben == 0:
        print(f"\n⚠️ No se encontraron registros con Sisbén desactualizado para generar N21")
        return df_ns
    
    # =========================================================================
    # PASO 2: CONSTRUIR NOVEDADES N21 - SISBÉN DESACTUALIZADO
    # =========================================================================
    print(f"\n🔧 CONSTRUYENDO NOVEDADES N21 - SISBÉN DESACTUALIZADO")
    
    # Función auxiliar para determinar COD_2_NOVEDAD
    def determinar_cod2_sisben(sisben_actual):
        if pd.isna(sisben_actual) or sisben_actual == '':
            return None
        
        grupo = sisben_actual[0].upper()
        
        # Grupos A y B asignan '1'
        if grupo in ('A', 'B'):
            return '1'
        # Grupo C asigna '2'
        elif grupo == 'C':
            return '2'
        else:
            return None

    # Aplicar la lógica para COD_2_NOVEDAD
    registros_caso_sisben['COD_2_NOVEDAD_CALC'] = registros_caso_sisben['N_Sisben_Actual'].apply(determinar_cod2_sisben)
    
    # Verificar columnas necesarias
    columnas_requeridas = [
        'ENT_ID', 'TPS_IDN_ID', 'HST_IDN_NUMERO_IDENTIFICACION',
        'AFL_PRIMER_APELLIDO', 'AFL_SEGUNDO_APELLIDO', 
        'AFL_PRIMER_NOMBRE', 'AFL_SEGUNDO_NOMBRE',
        'AFL_FECHA_NACIMIENTO', 'DPR_ID', 'MNC_ID', 
        'TPS_EST_AFL_ID'
    ]
    
    columnas_faltantes = [col for col in columnas_requeridas if col not in registros_caso_sisben.columns]
    if columnas_faltantes:
        print(f"❌ Columnas faltantes para construir novedad N21: {columnas_faltantes}")
        return df_ns
    
    try:
        # Construir DataFrame de novedades
        nuevas_novedades_sisben = pd.DataFrame({
            'ENT_ID': registros_caso_sisben['ENT_ID'],
            'TPS_IDN_ID': registros_caso_sisben['TPS_IDN_ID'],
            'HST_IDN_NUMERO_IDENTIFICACION': registros_caso_sisben['HST_IDN_NUMERO_IDENTIFICACION'],
            'AFL_PRIMER_APELLIDO': registros_caso_sisben['AFL_PRIMER_APELLIDO'],
            'AFL_SEGUNDO_APELLIDO': registros_caso_sisben['AFL_SEGUNDO_APELLIDO'],
            'AFL_PRIMER_NOMBRE': registros_caso_sisben['AFL_PRIMER_NOMBRE'],
            'AFL_SEGUNDO_NOMBRE': registros_caso_sisben['AFL_SEGUNDO_NOMBRE'],
            'AFL_FECHA_NACIMIENTO': registros_caso_sisben['AFL_FECHA_NACIMIENTO'],
            'DPR_ID': registros_caso_sisben['DPR_ID'],
            'MNS_ID': registros_caso_sisben['MNC_ID'],  # MNC_ID -> MNS_ID
            'NOVEDAD': 'N21',
            'FECHA_NOVEDAD': fecha_novedad_str,
            'COD_1_NOVEDAD': '2',                                            # Código fijo para Sisbén
            'COD_2_NOVEDAD': registros_caso_sisben['N_Sisben_Actual'],        # Valor del Sisbén Local                                            # Subcategoría Sisbén IV
            'COD_3_NOVEDAD': '5',
            'COD_4_NOVEDAD': registros_caso_sisben['COD_2_NOVEDAD_CALC'],     # Categoria 1 (A/B) o 2 (C)
            'COD_5_NOVEDAD': '06',                                           # Código de etnia por defecto
            'ENT_ID_ADRES': registros_caso_sisben['ENT_ID'],
            'TPS_EST_AFL_ID_from_adres': registros_caso_sisben['TPS_EST_AFL_ID'],
            'Where': 'Novedad Masiva Sisben'
        })
        
        # Crear ID_User e ID_Register
        nuevas_novedades_sisben['ID_User'] = (
            nuevas_novedades_sisben['TPS_IDN_ID'].astype(str) + 
            nuevas_novedades_sisben['HST_IDN_NUMERO_IDENTIFICACION'].astype(str)
        )
        
        nuevas_novedades_sisben['ID_Register'] = (
            nuevas_novedades_sisben['TPS_IDN_ID'].astype(str) + 
            nuevas_novedades_sisben['HST_IDN_NUMERO_IDENTIFICACION'].astype(str) + 
            nuevas_novedades_sisben['NOVEDAD'].astype(str)
        )
        
        # Rellenar columnas faltantes del DF de destino (df_ns)
        columnas_faltantes_df = [col for col in df_ns.columns if col not in nuevas_novedades_sisben.columns]
        for col in columnas_faltantes_df:
            nuevas_novedades_sisben[col] = None
        
        # Ordenar columnas
        nuevas_novedades_sisben = nuevas_novedades_sisben[df_ns.columns]
        
        print(f"   ✅ Novedades N21 Sisbén construidas: {len(nuevas_novedades_sisben):,}")
        
        # Mostrar ejemplos
        print(f"\n   📋 Ejemplos de novedades Sisbén:")
        ejemplos_columnas = [
            'TPS_IDN_ID', 'HST_IDN_NUMERO_IDENTIFICACION', 'AFL_PRIMER_APELLIDO',
            'COD_1_NOVEDAD', 'COD_2_NOVEDAD', 'COD_3_NOVEDAD', 
            'COD_4_NOVEDAD', 'N_Sisben_Actual', 'SUB_SISBEN_IV' # Columnas de contexto
        ]
        ejemplos_disponibles = [col for col in ejemplos_columnas if col in nuevas_novedades_sisben.columns]
        
        # Añadir las columnas de Sisben del maestro para contexto
        columnas_contexto = ['N_Sisben_Actual', 'SUB_SISBEN_IV']
        
        # Unir las nuevas novedades con las columnas de Sisbén del maestro para el ejemplo
        ejemplos_mostrar = nuevas_novedades_sisben.merge(
            registros_caso_sisben[['HST_IDN_NUMERO_IDENTIFICACION'] + columnas_contexto],
            on='HST_IDN_NUMERO_IDENTIFICACION',
            how='left'
        )[ejemplos_columnas].drop_duplicates(subset=['HST_IDN_NUMERO_IDENTIFICACION']).head(10)
        
        print(ejemplos_mostrar.to_string(index=False))

        # =========================================================================
        # PASO 3: AGREGAR AL DATAFRAME PRINCIPAL
        # =========================================================================
        print(f"\n🔄 PASO 3: AGREGANDO NOVEDADES A df_NS")
        
        registros_antes = len(df_ns)
        df_ns = pd.concat([df_ns, nuevas_novedades_sisben], ignore_index=True)
        registros_despues = len(df_ns)
        
        print(f"   📊 Total de novedades Sisbén agregadas: {len(nuevas_novedades_sisben):,}")
        print(f"   📊 Registros agregados: {registros_despues - registros_antes:,}")
        
    except Exception as e:
        print(f"❌ Error al construir o agregar novedades Sisbén: {e}")
        return df_ns
    
    # =========================================================================
    # PASO 4: ESTADÍSTICAS FINALES
    # =========================================================================
    print(f"\n📈 ESTADÍSTICAS DE NOVEDADES N21 SISBÉN AGREGADAS:")
    
    novedades_n21_masivas = df_ns[
        (df_ns['Where'] == 'Novedad Masiva Sisben') & 
        (df_ns['NOVEDAD'] == 'N21')
    ]
    
    if len(novedades_n21_masivas) > 0:
        print(f"   📊 Total novedades N21 Sisbén masivas: {len(novedades_n21_masivas):,}")
        
        # Distribución por Sisbén (COD_3_NOVEDAD)
        print(f"\n   📊 Por Sisbén (COD_3_NOVEDAD):")
        distribucion_cod3 = novedades_n21_masivas['COD_3_NOVEDAD'].value_counts().head(10)
        for cod3, count in distribucion_cod3.items():
            print(f"     Sisbén {cod3}: {count:,} registros")
        
        # Distribución por Categoría (COD_4_NOVEDAD)
        print(f"\n   📊 Por Categoría (COD_4_NOVEDAD):")
        distribucion_cod4 = novedades_n21_masivas['COD_4_NOVEDAD'].value_counts()
        for cod4, count in distribucion_cod4.items():
            print(f"     Categoría {cod4}: {count:,} registros")

    else:
        print(f"   ⚠️ No se encontraron novedades N21 Sisbén masivas en el DataFrame final")

    print(f"\n✅ ACTUALIZACIONES MASIVAS SISBÉN N21 APLICADAS EXITOSAMENTE")
    
    return df_ns

# Aplicar actualizaciones masivas N21 Sisbén
df_NS = aplicar_actualizaciones_sisben_N21(df_NS, maestro_ADRES, fecha_reporte)

## 5.11. N25 Actualización de IPS Primaria

In [ ]:
def validar_novedades_N25(df, df_no_envio, df_ips):
    """
    Valida registros con NOVEDAD = 'N25' (Actualización de IPS Primaria).
    
    Proceso:
    1. Filtrar solo registros del departamento de Casanare (DPR_ID = 85)
    2. Traer código de IPS válido desde df_IPS según municipio (MNS_ID)
    3. Asignar el código de IPS a COD_1_NOVEDAD y COD_2_NOVEDAD
    4. Vaciar columnas COD_4_NOVEDAD hasta COD_7_NOVEDAD
    5. Mover registros de otros departamentos a DF_No_Envio
    """
    print("🔍 INICIANDO VALIDACIÓN DE NOVEDADES N25")
    print("="*60)
    
    # Filtrar solo registros N25
    mask_n25 = df['NOVEDAD'] == 'N25'
    registros_n25 = df[mask_n25].copy()
    total_n25_inicial = len(registros_n25)
    
    print(f"📊 Total de registros N25 iniciales: {total_n25_inicial}")
    
    if total_n25_inicial == 0:
        print("⚠️ No se encontraron registros N25")
        return df, df_no_envio
    
    # =========================================================================
    # PASO 1: FILTRAR POR DEPARTAMENTO DE CASANARE (DPR_ID = 85)
    # =========================================================================
    print(f"\n🔍 PASO 1: Validando departamento")
    
    # Normalizar DPR_ID
    registros_n25['DPR_ID_NORM'] = registros_n25['DPR_ID'].astype(str).str.strip()
    
    # Identificar registros de Casanare vs otros departamentos
    mask_casanare = registros_n25['DPR_ID_NORM'] == '85'
    registros_casanare = registros_n25[mask_casanare].copy()
    registros_otros_dpto = registros_n25[~mask_casanare].copy()
    
    total_casanare = len(registros_casanare)
    total_otros_dpto = len(registros_otros_dpto)
    
    print(f"   📊 Registros de Casanare (DPR_ID = 85): {total_casanare}")
    print(f"   📊 Registros de otros departamentos: {total_otros_dpto}")
    
    # Mostrar distribución de departamentos si hay otros
    if total_otros_dpto > 0:
        print(f"\n   📊 Distribución de otros departamentos:")
        distribucion_dpto = registros_otros_dpto['DPR_ID_NORM'].value_counts()
        for dpto, count in distribucion_dpto.items():
            print(f"     DPR_ID {dpto}: {count} registros")
        
        # Mostrar ejemplos
        print(f"\n   📋 Ejemplos de registros de otros departamentos:")
        ejemplos_otros = registros_otros_dpto[['NUM_SOLICITUD_NOVEDAD', 'HST_IDN_NUMERO_IDENTIFICACION', 
                                              'DPR_ID_NORM', 'MNS_ID']].head(5)
        print(ejemplos_otros.to_string(index=False))
        
        # Mover a DF_No_Envio
        registros_otros_clean = registros_otros_dpto.drop(columns=['DPR_ID_NORM'])
        registros_otros_clean['motivo'] = 'Registro de otro departamento'
        
        df_no_envio = pd.concat([df_no_envio, registros_otros_clean], ignore_index=True)
        
        # Eliminar del DataFrame principal
        indices_otros_dpto = df.loc[mask_n25][~mask_casanare].index
        df = df.drop(indices_otros_dpto).reset_index(drop=True)
        
        print(f"   ✅ Movidos {total_otros_dpto} registros a DF_No_Envio")
    
    # =========================================================================
    # PASO 2: PROCESAR df_IPS PARA CREAR MAPEO
    # =========================================================================
    if total_casanare == 0:
        print(f"\n⚠️ No quedan registros N25 de Casanare para procesar")
        return df, df_no_envio
    
    print(f"\n🔍 PASO 2: Procesando datos de IPS")
    
    # Verificar estructura de df_IPS
    print(f"   📊 df_IPS shape: {df_ips.shape}")
    print(f"   📋 Primeras filas de df_IPS:")
    print(df_ips.head().to_string(index=False, header=False))
    
    # Asignar nombres a las columnas (asumiendo que columna 0 = municipio, columna 1 = código IPS)
    df_ips_procesado = df_ips.copy()
    df_ips_procesado.columns = ['MNS_ID', 'COD_IPS']
    
    # Normalizar códigos de municipio (asegurar formato de 3 dígitos)
    df_ips_procesado['MNS_ID_NORM'] = df_ips_procesado['MNS_ID'].astype(str).str.strip().str.zfill(3)
    df_ips_procesado['COD_IPS_NORM'] = df_ips_procesado['COD_IPS'].astype(str).str.strip()
    
    # Mostrar mapeo de IPS
    print(f"\n   📊 Municipios con IPS válidas: {df_ips_procesado['MNS_ID_NORM'].nunique()}")
    print(f"   📋 Ejemplos de mapeo:")
    ejemplos_ips = df_ips_procesado[['MNS_ID_NORM', 'COD_IPS_NORM']].head(10)
    print(ejemplos_ips.to_string(index=False))
    
    # =========================================================================
    # PASO 3: TRAER CÓDIGOS DE IPS PARA REGISTROS DE CASANARE
    # =========================================================================
    print(f"\n🔍 PASO 3: Asignando códigos de IPS")
    
    # Normalizar MNS_ID en registros de Casanare
    registros_casanare['MNS_ID_NORM'] = registros_casanare['MNS_ID'].astype(str).str.strip().str.zfill(3)
    
    # Hacer merge para traer códigos de IPS
    registros_con_ips = registros_casanare.merge(
        df_ips_procesado[['MNS_ID_NORM', 'COD_IPS_NORM']],
        on='MNS_ID_NORM',
        how='left'
    )
    
    # Verificar match
    registros_con_match = registros_con_ips['COD_IPS_NORM'].notna().sum()
    registros_sin_match = registros_con_ips['COD_IPS_NORM'].isna().sum()
    
    print(f"   ✅ Registros con IPS válida encontrada: {registros_con_match}")
    print(f"   ⚠️ Registros sin IPS válida: {registros_sin_match}")
    
    if registros_sin_match > 0:
        print(f"\n   📋 Municipios sin IPS válida:")
        municipios_sin_ips = registros_con_ips[registros_con_ips['COD_IPS_NORM'].isna()]['MNS_ID_NORM'].unique()
        print(f"      {municipios_sin_ips}")
        
        # Mostrar ejemplos
        ejemplos_sin_ips = registros_con_ips[registros_con_ips['COD_IPS_NORM'].isna()][
            ['NUM_SOLICITUD_NOVEDAD', 'HST_IDN_NUMERO_IDENTIFICACION', 'MNS_ID_NORM']
        ].head(5)
        print(f"\n   📋 Ejemplos de registros sin IPS:")
        print(ejemplos_sin_ips.to_string(index=False))
    
    # =========================================================================
    # PASO 4: APLICAR CAMBIOS AL DATAFRAME PRINCIPAL
    # =========================================================================
    print(f"\n🔄 PASO 4: Aplicando cambios al DataFrame principal")
    
    # Actualizar registros en df_NS que son N25 y de Casanare
    mask_n25_casanare = (df['NOVEDAD'] == 'N25') & (df['DPR_ID'].astype(str).str.strip() == '85')
    
    if mask_n25_casanare.any():
        # Crear mapeo temporal para actualización
        mapeo_ips = dict(zip(registros_con_ips['ID_User'], registros_con_ips['COD_IPS_NORM']))
        
        # Contadores de actualizaciones
        actualizaciones_cod1 = 0
        actualizaciones_cod2 = 0
        vaciado_cod4_7 = 0
        
        for idx in df[mask_n25_casanare].index:
            id_user = df.loc[idx, 'ID_User']
            
            if id_user in mapeo_ips and pd.notna(mapeo_ips[id_user]):
                # Asignar código de IPS a COD_1_NOVEDAD y COD_2_NOVEDAD
                codigo_ips = mapeo_ips[id_user]
                
                if str(df.loc[idx, 'COD_1_NOVEDAD']).strip() != codigo_ips:
                    actualizaciones_cod1 += 1
                if str(df.loc[idx, 'COD_2_NOVEDAD']).strip() != codigo_ips:
                    actualizaciones_cod2 += 1
                
                df.loc[idx, 'COD_1_NOVEDAD'] = codigo_ips
                df.loc[idx, 'COD_2_NOVEDAD'] = codigo_ips
            
            # Vaciar columnas COD_4_NOVEDAD hasta COD_7_NOVEDAD
            columnas_a_vaciar = ['COD_4_NOVEDAD', 'COD_5_NOVEDAD', 'COD_6_NOVEDAD', 'COD_7_NOVEDAD']
            for columna in columnas_a_vaciar:
                if pd.notna(df.loc[idx, columna]) and str(df.loc[idx, columna]).strip() != '':
                    vaciado_cod4_7 += 1
                df.loc[idx, columna] = ''
        
        print(f"   ✅ COD_1_NOVEDAD actualizados: {actualizaciones_cod1}")
        print(f"   ✅ COD_2_NOVEDAD actualizados: {actualizaciones_cod2}")
        print(f"   ✅ Columnas COD_4-COD_7 vaciadas: {vaciado_cod4_7}")
    
    # =========================================================================
    # PASO 5: ESTADÍSTICAS FINALES
    # =========================================================================
    registros_n25_final = len(df[df['NOVEDAD'] == 'N25'])
    
    print(f"\n📈 RESUMEN FINAL:")
    print(f"   Registros N25 iniciales: {total_n25_inicial}")
    print(f"   Registros movidos (otros departamentos): {total_otros_dpto}")
    print(f"   Registros N25 restantes (Casanare): {registros_n25_final}")
    print(f"   Total en DF_No_Envio actual: {len(df_no_envio)}")
    
    # Mostrar ejemplos de registros procesados
    if registros_n25_final > 0:
        print(f"\n📋 EJEMPLOS DE REGISTROS N25 PROCESADOS:")
        ejemplos_finales = df[df['NOVEDAD'] == 'N25'][[
            'NUM_SOLICITUD_NOVEDAD', 'HST_IDN_NUMERO_IDENTIFICACION',
            'DPR_ID', 'MNS_ID', 'COD_1_NOVEDAD', 'COD_2_NOVEDAD'
        ]].head(10)
        print(ejemplos_finales.to_string(index=False))
        
        # Verificar que COD_4-COD_7 están vacías
        registros_n25_actuales = df[df['NOVEDAD'] == 'N25']
        columnas_vacias = ['COD_4_NOVEDAD', 'COD_5_NOVEDAD', 'COD_6_NOVEDAD', 'COD_7_NOVEDAD']
        
        print(f"\n🔍 VERIFICACIÓN DE COLUMNAS VACÍAS:")
        for columna in columnas_vacias:
            vacios = (registros_n25_actuales[columna].isna() | 
                     (registros_n25_actuales[columna] == '')).sum()
            total = len(registros_n25_actuales)
            print(f"   {columna}: {vacios}/{total} vacías")
        
        # Mostrar distribución de códigos IPS asignados
        print(f"\n📊 CÓDIGOS IPS ASIGNADOS:")
        distribucion_ips = df[df['NOVEDAD'] == 'N25']['COD_1_NOVEDAD'].value_counts().head(10)
        for codigo, count in distribucion_ips.items():
            print(f"   {codigo}: {count} registros")
    
    print(f"\n✅ VALIDACIÓN N25 COMPLETADA")
    
    return df, df_no_envio

# Aplicar validación N25
df_NS, DF_No_Envio = validar_novedades_N25(df_NS, DF_No_Envio, df_IPS)

## 5.11. N32 Corrección de parentesco. - Discapacidad

In [ ]:
def validar_novedades_N32(df, df_no_envio, maestro_adres):
    """
    Valida registros con NOVEDAD = 'N32' (Corrección de parentesco - Discapacidad).
    
    Proceso:
    1. Estandarización:
       - COD_3_NOVEDAD: Se fuerza a "B".
       - COD_5_NOVEDAD: Si tiene valor, se fuerza a "D". Si está vacío, se deja vacío.
    2. Validación de formato:
       - COD_4_NOVEDAD: Debe ser uno de [1, 2, 3, 4, 5, 8, 9, 10, 11].
    3. Validación de No-Cambio (ADRES):
       - Se compara COD_1_NOVEDAD (Tipo Doc CF) y COD_2_NOVEDAD (Num Doc CF)
         con TPS_IDN_ID_CF y HST_IDN_NUMERO_IDENTIFICACION_CF del maestro.
       - Si son iguales, se mueve a DF_No_Envio.
    """
    print("🔍 INICIANDO VALIDACIÓN DE NOVEDADES N32")
    print("="*60)
    
    # Filtrar solo registros N32
    mask_n32 = df['NOVEDAD'] == 'N32'
    registros_n32 = df[mask_n32].copy()
    total_n32_inicial = len(registros_n32)
    
    print(f"📊 Total de registros N32 iniciales: {total_n32_inicial}")
    
    if total_n32_inicial == 0:
        print("⚠️ No se encontraron registros N32")
        return df, df_no_envio

    # =========================================================================
    # PASO 1: ESTANDARIZACIÓN (COD_3 y COD_5)
    # =========================================================================
    print(f"\n🔧 PASO 1: Estandarizando COD_3 (Parentesco) y COD_5 (Discapacidad)")
    
    # 1.1 COD_3_NOVEDAD siempre debe ser "B"
    correcciones_cod3 = (registros_n32['COD_3_NOVEDAD'] != 'B').sum()
    registros_n32['COD_3_NOVEDAD'] = 'B'
    print(f"   ✅ COD_3_NOVEDAD forzado a 'B' en {correcciones_cod3} registros")

    # 1.2 COD_5_NOVEDAD: Si tiene valor -> "D", si no -> vacío
    def corregir_discapacidad(valor):
        if pd.isna(valor) or str(valor).strip() == '':
            return ''
        return 'D'

    registros_n32['COD_5_NOVEDAD'] = registros_n32['COD_5_NOVEDAD'].apply(corregir_discapacidad)
    print(f"   ✅ COD_5_NOVEDAD estandarizado (D o vacío)")

    # Aplicar correcciones al DataFrame principal (para los registros que pasen la validación)
    df.loc[mask_n32, 'COD_3_NOVEDAD'] = registros_n32['COD_3_NOVEDAD']
    df.loc[mask_n32, 'COD_5_NOVEDAD'] = registros_n32['COD_5_NOVEDAD']

    # =========================================================================
    # PASO 2: VALIDACIÓN DE FORMATO COD_4 (CÓDIGO PARENTESCO)
    # =========================================================================
    print(f"\n🔧 PASO 2: Validando COD_4_NOVEDAD (Código Parentesco)")
    
    valores_validos_cod4 = ['1', '2', '3', '4', '5', '8', '9', '10', '11']
    
    # Normalizar para comparar
    registros_n32['COD_4_NORM'] = registros_n32['COD_4_NOVEDAD'].astype(str).str.strip()
    
    mask_cod4_invalido = ~registros_n32['COD_4_NORM'].isin(valores_validos_cod4)
    registros_cod4_invalido = registros_n32[mask_cod4_invalido].copy()
    total_cod4_invalido = len(registros_cod4_invalido)
    
    if total_cod4_invalido > 0:
        print(f"   ⚠️ Se encontraron {total_cod4_invalido} registros con COD_4 inválido")
        print(f"      Valores permitidos: {valores_validos_cod4}")
        
        # Mostrar ejemplos
        print(f"      Ejemplos inválidos: {registros_cod4_invalido['COD_4_NOVEDAD'].unique()}")
        
        # Mover a DF_No_Envio
        registros_cod4_invalido_clean = registros_cod4_invalido.drop(columns=['COD_4_NORM'])
        registros_cod4_invalido_clean['motivo'] = "Código de parentesco (COD_4) inválido"
        
        df_no_envio = pd.concat([df_no_envio, registros_cod4_invalido_clean], ignore_index=True)
        
        # Eliminar del set de trabajo actual
        registros_n32 = registros_n32[~mask_cod4_invalido].copy()
        
        # Eliminar del DF principal
        ids_eliminar = registros_cod4_invalido['ID_User'].unique()
        mask_eliminar = (df['NOVEDAD'] == 'N32') & (df['ID_User'].isin(ids_eliminar))
        df = df[~mask_eliminar].reset_index(drop=True)
        
        print(f"   ✅ Movidos {total_cod4_invalido} registros a DF_No_Envio")
    else:
        print(f"   ✅ Todos los registros tienen COD_4 válido")

    # =========================================================================
    # PASO 3: VALIDACIÓN DE NO-CAMBIO CON MAESTRO ADRES
    # =========================================================================
    print(f"\n🔍 PASO 3: Verificando si el Cabeza de Familia ya está actualizado en ADRES")
    
    if len(registros_n32) == 0:
        print("   ⚠️ No quedan registros para validar contra ADRES")
        return df, df_no_envio

    # Verificar si existen las columnas de Cabeza de Familia en el maestro
    cols_cf = ['TPS_IDN_ID_CF', 'HST_IDN_NUMERO_IDENTIFICACION_CF']
    cols_existentes = [c for c in cols_cf if c in maestro_adres.columns]
    
    if len(cols_existentes) < 2:
        print(f"   ❌ Faltan columnas de Cabeza de Familia en maestro_ADRES: {cols_cf}")
        print("   ⚠️ Saltando validación de cruce ADRES")
        return df, df_no_envio

    # Preparar subset del maestro
    maestro_cf = maestro_adres[['ID_User'] + cols_cf].drop_duplicates(subset=['ID_User'])
    
    # Merge
    registros_n32_con_adres = registros_n32.merge(maestro_cf, on='ID_User', how='left')
    
    # Normalizar para comparación
    registros_n32_con_adres['COD_1_NORM'] = registros_n32_con_adres['COD_1_NOVEDAD'].astype(str).str.strip().str.upper()
    registros_n32_con_adres['COD_2_NORM'] = registros_n32_con_adres['COD_2_NOVEDAD'].astype(str).str.strip()
    
    registros_n32_con_adres['CF_TIPO_ADRES'] = registros_n32_con_adres['TPS_IDN_ID_CF'].astype(str).str.strip().str.upper()
    registros_n32_con_adres['CF_NUM_ADRES'] = registros_n32_con_adres['HST_IDN_NUMERO_IDENTIFICACION_CF'].astype(str).str.strip()
    
    # Lógica: Si Tipo Y Número son iguales a lo que hay en ADRES -> Rechazar
    mask_sin_cambios = (
        (registros_n32_con_adres['COD_1_NORM'] == registros_n32_con_adres['CF_TIPO_ADRES']) &
        (registros_n32_con_adres['COD_2_NORM'] == registros_n32_con_adres['CF_NUM_ADRES']) &
        (registros_n32_con_adres['TPS_IDN_ID_CF'].notna()) # Solo si existe dato en ADRES
    )
    
    registros_sin_cambios = registros_n32_con_adres[mask_sin_cambios].copy()
    total_sin_cambios = len(registros_sin_cambios)
    
    print(f"   Registros con Cabeza de Familia idéntico en ADRES: {total_sin_cambios}")
    print(f"   Registros con cambio válido: {len(registros_n32) - total_sin_cambios}")
    
    if total_sin_cambios > 0:
        print(f"\n   📋 Ejemplos sin cambios (Rechazados):")
        ejemplos = registros_sin_cambios[[
            'NUM_SOLICITUD_NOVEDAD', 'HST_IDN_NUMERO_IDENTIFICACION',
            'COD_1_NOVEDAD', 'COD_2_NOVEDAD', 'CF_TIPO_ADRES', 'CF_NUM_ADRES'
        ]].head(5)
        print(ejemplos.to_string(index=False))
        
        # Limpiar y mover a DF_No_Envio
        cols_temp = ['COD_4_NORM', 'COD_1_NORM', 'COD_2_NORM', 'CF_TIPO_ADRES', 'CF_NUM_ADRES', 'TPS_IDN_ID_CF', 'HST_IDN_NUMERO_IDENTIFICACION_CF']
        registros_sin_cambios_clean = registros_sin_cambios.drop(columns=[c for c in cols_temp if c in registros_sin_cambios.columns])
        registros_sin_cambios_clean['motivo'] = "Cabeza de familia ya actualizado en ADRES"
        
        df_no_envio = pd.concat([df_no_envio, registros_sin_cambios_clean], ignore_index=True)
        
        # Eliminar del DataFrame principal
        ids_eliminar = registros_sin_cambios['ID_User'].unique()
        mask_eliminar = (df['NOVEDAD'] == 'N32') & (df['ID_User'].isin(ids_eliminar))
        df = df[~mask_eliminar].reset_index(drop=True)
        
        print(f"   ✅ Movidos {total_sin_cambios} registros a DF_No_Envio")

    # =========================================================================
    # RESUMEN FINAL
    # =========================================================================
    registros_n32_final = len(df[df['NOVEDAD'] == 'N32'])
    
    print(f"\n📈 RESUMEN FINAL N32:")
    print(f"   Registros iniciales: {total_n32_inicial}")
    print(f"   Rechazados por formato COD_4: {total_cod4_invalido}")
    print(f"   Rechazados por sin cambios (ADRES): {total_sin_cambios}")
    print(f"   Registros válidos finales: {registros_n32_final}")
    print(f"   Total en DF_No_Envio: {len(df_no_envio)}")
    
    if registros_n32_final > 0:
        print(f"\n📋 EJEMPLOS FINALES VÁLIDOS:")
        print(df[df['NOVEDAD'] == 'N32'][[
            'HST_IDN_NUMERO_IDENTIFICACION', 'COD_1_NOVEDAD', 'COD_2_NOVEDAD', 
            'COD_3_NOVEDAD', 'COD_4_NOVEDAD', 'COD_5_NOVEDAD'
        ]].head().to_string(index=False))

    return df, df_no_envio

# Aplicar validación N32
df_NS, DF_No_Envio = validar_novedades_N32(df_NS, DF_No_Envio, maestro_ADRES)

## 5.12.N43 Municipio de nacimiento o recidencia

### 5.12.1. Validación de Novedades N43 Existentes

In [ ]:
def validar_novedades_N43(df, df_no_envio, maestro_adres):
    """
    Valida registros con NOVEDAD = 'N43' (Municipio de nacimiento o residencia).
    
    Proceso:
    1. Validar formato de COD_1_NOVEDAD (debe ser de 5 dígitos).
    2. Comparar con el valor actual en ADRES (AFL_MUNICIPIO_NACIMIENTO).
    3. Si son iguales y el afiliado es Activo y Subsidiado (EPS025), mover a DF_No_Envio.
    """
    print("🔍 INICIANDO VALIDACIÓN DE NOVEDADES N43")
    print("="*60)
    
    # Filtrar solo registros N43
    mask_n43 = df['NOVEDAD'] == 'N43'
    registros_n43 = df[mask_n43].copy()
    total_n43_inicial = len(registros_n43)
    
    print(f"📊 Total de registros N43 iniciales: {total_n43_inicial}")
    
    if total_n43_inicial == 0:
        print("⚠️ No se encontraron registros N43")
        return df, df_no_envio

    # =========================================================================
    # PASO 1: VALIDAR FORMATO (5 DÍGITOS)
    # =========================================================================
    print(f"\n🔧 PASO 1: Validando formato de COD_1_NOVEDAD (5 dígitos)")
    
    def validar_formato_municipio(valor):
        if pd.isna(valor) or str(valor).strip() == '':
            return False
        valor_str = str(valor).strip()
        return len(valor_str) == 5 and valor_str.isdigit()

    registros_n43['FORMATO_VALIDO'] = registros_n43['COD_1_NOVEDAD'].apply(validar_formato_municipio)
    invalidos_formato = (~registros_n43['FORMATO_VALIDO']).sum()
    
    if invalidos_formato > 0:
        print(f"   ⚠️ Se encontraron {invalidos_formato} registros con formato inválido")
        # Aquí podrías decidir moverlos a No_Envio si quisieras ser estricto, 
        # por ahora solo advertimos.

    # =========================================================================
    # PASO 2: TRAER DATOS DE MAESTRO ADRES
    # =========================================================================
    print(f"\n🔍 PASO 2: Comparando con datos actuales en ADRES")
    
    # Columnas necesarias del maestro
    cols_maestro = ['ID_User', 'AFL_MUNICIPIO_NACIMIENTO', 'ENT_ID', 'TPS_EST_AFL_ID']
    
    # Asegurar que existan en el maestro (AFL_MUNICIPIO_NACIMIENTO a veces tiene otros nombres)
    if 'AFL_MUNICIPIO_NACIMIENTO' not in maestro_adres.columns:
        print("❌ Columna 'AFL_MUNICIPIO_NACIMIENTO' no encontrada en maestro_ADRES. Saltando validación de igualdad.")
        return df, df_no_envio

    maestro_subset = maestro_adres[cols_maestro].drop_duplicates(subset=['ID_User'])
    
    # Merge
    registros_n43_con_maestro = registros_n43.merge(
        maestro_subset,
        on='ID_User',
        how='left'
    )
    
    # =========================================================================
    # PASO 3: IDENTIFICAR REGISTROS SIN CAMBIOS (IGUALES A ADRES)
    # =========================================================================
    
    # Normalizar para comparación
    registros_n43_con_maestro['COD_1_NORM'] = registros_n43_con_maestro['COD_1_NOVEDAD'].astype(str).str.strip()
    registros_n43_con_maestro['MUNI_ADRES_NORM'] = registros_n43_con_maestro['AFL_MUNICIPIO_NACIMIENTO'].astype(str).str.strip()
    
    # Lógica de rechazo:
    # 1. El código nuevo es IGUAL al actual.
    # 2. El afiliado es EPS025 (Subsidiado).
    # 3. El afiliado está Activo (AC).
    mask_sin_cambios = (
        (registros_n43_con_maestro['COD_1_NORM'] == registros_n43_con_maestro['MUNI_ADRES_NORM']) &
        (registros_n43_con_maestro['ENT_ID_y'] == 'EPS025') & # _y viene del maestro
        (registros_n43_con_maestro['TPS_EST_AFL_ID'] == 'AC') &
        (registros_n43_con_maestro['MUNI_ADRES_NORM'] != '') & 
        (registros_n43_con_maestro['MUNI_ADRES_NORM'] != 'nan')
    )
    
    registros_rechazados = registros_n43_con_maestro[mask_sin_cambios].copy()
    total_rechazados = len(registros_rechazados)
    
    print(f"   Registros N43 totales: {total_n43_inicial}")
    print(f"   Registros rechazados (Dato ya actualizado en ADRES): {total_rechazados}")
    print(f"   Registros válidos: {total_n43_inicial - total_rechazados}")
    
    if total_rechazados > 0:
        # Mostrar ejemplos
        print(f"\n📋 EJEMPLOS DE REGISTROS RECHAZADOS (YA ACTUALIZADOS):")
        ejemplos = registros_rechazados[[
            'NUM_SOLICITUD_NOVEDAD', 'HST_IDN_NUMERO_IDENTIFICACION', 
            'COD_1_NORM', 'MUNI_ADRES_NORM'
        ]].head(5)
        print(ejemplos.to_string(index=False))
        
        # Preparar para mover a DF_No_Envio
        # Limpiar columnas temporales del merge
        cols_temp = ['FORMATO_VALIDO', 'AFL_MUNICIPIO_NACIMIENTO', 'ENT_ID_y', 'TPS_EST_AFL_ID', 'COD_1_NORM', 'MUNI_ADRES_NORM']
        registros_rechazados_clean = registros_rechazados.drop(columns=[c for c in cols_temp if c in registros_rechazados.columns])
        
        # Restaurar nombre original de ENT_ID si se renombró en el merge (ENT_ID_x)
        if 'ENT_ID_x' in registros_rechazados_clean.columns:
            registros_rechazados_clean = registros_rechazados_clean.rename(columns={'ENT_ID_x': 'ENT_ID'})
            
        registros_rechazados_clean['motivo'] = "Dato de municipio ya actualizado en ADRES"
        
        # Concatenar
        df_no_envio = pd.concat([df_no_envio, registros_rechazados_clean], ignore_index=True)
        
        # Eliminar del DataFrame principal usando ID_User
        ids_rechazados = registros_rechazados['ID_User'].unique()
        mask_eliminar = (df['NOVEDAD'] == 'N43') & (df['ID_User'].isin(ids_rechazados))
        df = df[~mask_eliminar].reset_index(drop=True)
        
        print(f"✅ Movidos {total_rechazados} registros a DF_No_Envio")
    else:
        print(f"✅ Todos los registros N43 son cambios válidos o necesarios")

    return df, df_no_envio

# Aplicar validación N43
df_NS, DF_No_Envio = validar_novedades_N43(df_NS, DF_No_Envio, maestro_ADRES)

### 5.12.2. Construcción Masiva de N43

In [ ]:
def aplicar_actualizaciones_masivas_N43(df_ns, maestro_adres, fecha_reporte):
    """
    Aplica actualizaciones masivas N43 (Municipio de nacimiento/residencia) para afiliados
    del régimen subsidiado (EPS025) activos que tienen este dato vacío en ADRES.
    
    Lógica:
    1. Filtrar EPS025 + Activos (AC).
    2. Filtrar quienes tengan AFL_MUNICIPIO_NACIMIENTO vacío.
    3. Excluir quienes ya tengan una N43 reportada en df_ns.
    4. Construir el código (COD_1_NOVEDAD) usando DPR_ID + MNC_ID (residencia actual).
    """
    print("🔍 INICIANDO APLICACIÓN DE ACTUALIZACIONES MASIVAS N43")
    print("="*80)
    
    fecha_novedad_str = fecha_reporte.strftime('%d/%m/%Y')
    print(f"📅 Fecha de novedad: {fecha_novedad_str}")
    
    # Verificar existencia de columna en maestro
    col_muni_nac = 'AFL_MUNICIPIO_NACIMIENTO'
    if col_muni_nac not in maestro_adres.columns:
        print(f"❌ Columna '{col_muni_nac}' no encontrada en maestro_ADRES. No se puede procesar.")
        return df_ns

    # =========================================================================
    # PASO 1: IDENTIFICAR CANDIDATOS (EPS025 + AC + MUNICIPIO VACÍO)
    # =========================================================================
    print(f"\n🔍 PASO 1: Identificando candidatos sin municipio de nacimiento")
    
    # Normalizar columna para verificar vacíos
    maestro_adres[col_muni_nac] = maestro_adres[col_muni_nac].fillna('').astype(str).str.strip()
    
    mask_candidatos = (
        (maestro_adres['ENT_ID'] == 'EPS025') &
        (maestro_adres['TPS_EST_AFL_ID'] == 'AC') &
        ((maestro_adres[col_muni_nac] == '') | (maestro_adres[col_muni_nac] == 'nan')) &
        (maestro_adres['DPR_ID'].notna()) & # Deben tener datos de residencia para construir
        (maestro_adres['MNC_ID'].notna())
    )
    
    registros_candidatos = maestro_adres[mask_candidatos].copy().reset_index(drop=True)
    total_candidatos = len(registros_candidatos)
    
    print(f"   📊 Registros activos EPS025 sin municipio nacimiento: {total_candidatos:,}")
    
    if total_candidatos == 0:
        print("⚠️ No hay registros para procesar")
        return df_ns

    # =========================================================================
    # PASO 2: EXCLUIR LOS QUE YA ESTÁN EN df_NS (COMO N43)
    # =========================================================================
    print(f"\n🔍 PASO 2: Excluyendo afiliados que ya tienen N43 en proceso")
    
    ids_con_n43 = df_ns[df_ns['NOVEDAD'] == 'N43']['ID_User'].unique()
    
    registros_finales = registros_candidatos[~registros_candidatos['ID_User'].isin(ids_con_n43)].copy()
    total_finales = len(registros_finales)
    
    print(f"   Excluidos (ya en df_NS): {total_candidatos - total_finales}")
    print(f"   Total a generar: {total_finales}")
    
    if total_finales == 0:
        return df_ns

    # =========================================================================
    # PASO 3: CONSTRUIR CÓDIGO DE MUNICIPIO (DPR + MNC)
    # =========================================================================
    print(f"\n🔧 PASO 3: Construyendo código de municipio (5 dígitos)")
    
    def limpiar_codigo(valor, relleno):
        """Limpia y formatea códigos numéricos eliminando decimales si existen"""
        try:
            # Convertir a float primero para manejar strings como "5.0" y luego a int
            val_int = int(float(str(valor).strip()))
            return str(val_int).zfill(relleno)
        except:
            return str(valor).strip().zfill(relleno)

    # Asegurar formato: DPR (2 dígitos) + MNC (3 dígitos)
    registros_finales['DPR_STR'] = registros_finales['DPR_ID'].apply(lambda x: limpiar_codigo(x, 2))
    registros_finales['MNC_STR'] = registros_finales['MNC_ID'].apply(lambda x: limpiar_codigo(x, 3))
    
    registros_finales['COD_MUNICIPIO_CALC'] = registros_finales['DPR_STR'] + registros_finales['MNC_STR']
    
    # Validar que haya quedado de 5 dígitos
    mask_validos = registros_finales['COD_MUNICIPIO_CALC'].str.len() == 5
    registros_finales = registros_finales[mask_validos].copy()
    
    print(f"   Registros con código válido construido: {len(registros_finales)}")

    # =========================================================================
    # PASO 4: CREAR DATAFRAME DE NOVEDADES
    # =========================================================================
    print(f"\n🔧 PASO 4: Construyendo estructura N43")
    
    try:
        nuevas_novedades = pd.DataFrame({
            'ENT_ID': registros_finales['ENT_ID'],
            'TPS_IDN_ID': registros_finales['TPS_IDN_ID'],
            'HST_IDN_NUMERO_IDENTIFICACION': registros_finales['HST_IDN_NUMERO_IDENTIFICACION'],
            'AFL_PRIMER_APELLIDO': registros_finales['AFL_PRIMER_APELLIDO'],
            'AFL_SEGUNDO_APELLIDO': registros_finales['AFL_SEGUNDO_APELLIDO'],
            'AFL_PRIMER_NOMBRE': registros_finales['AFL_PRIMER_NOMBRE'],
            'AFL_SEGUNDO_NOMBRE': registros_finales['AFL_SEGUNDO_NOMBRE'],
            'AFL_FECHA_NACIMIENTO': registros_finales['AFL_FECHA_NACIMIENTO'],
            'DPR_ID': registros_finales['DPR_ID'],
            'MNS_ID': registros_finales['MNC_ID'], # MNC_ID -> MNS_ID
            'NOVEDAD': 'N43',
            'FECHA_NOVEDAD': fecha_novedad_str,
            'COD_1_NOVEDAD': registros_finales['COD_MUNICIPIO_CALC'], # Código construido
            'ENT_ID_ADRES': registros_finales['ENT_ID'],
            'TPS_EST_AFL_ID_from_adres': registros_finales['TPS_EST_AFL_ID'],
            'Where': 'Novedad Masiva N43'
        })
        
        # Crear IDs
        nuevas_novedades['ID_User'] = (
            nuevas_novedades['TPS_IDN_ID'].astype(str) + 
            nuevas_novedades['HST_IDN_NUMERO_IDENTIFICACION'].astype(str)
        )
        
        nuevas_novedades['ID_Register'] = (
            nuevas_novedades['TPS_IDN_ID'].astype(str) + 
            nuevas_novedades['HST_IDN_NUMERO_IDENTIFICACION'].astype(str) + 
            nuevas_novedades['NOVEDAD'].astype(str)
        )
        
        # Rellenar columnas faltantes
        for col in df_ns.columns:
            if col not in nuevas_novedades.columns:
                nuevas_novedades[col] = None
                
        # Ordenar columnas
        nuevas_novedades = nuevas_novedades[df_ns.columns]
        
        print(f"   ✅ Novedades N43 construidas: {len(nuevas_novedades):,}")
        
        # Mostrar ejemplos
        print(f"\n   📋 Ejemplos de N43 Masivas:")
        print(nuevas_novedades[['HST_IDN_NUMERO_IDENTIFICACION', 'COD_1_NOVEDAD', 'DPR_ID', 'MNS_ID']].head().to_string(index=False))

        # Agregar al DF principal
        registros_antes = len(df_ns)
        df_ns = pd.concat([df_ns, nuevas_novedades], ignore_index=True)
        print(f"\n   📊 Registros agregados a df_NS: {len(df_ns) - registros_antes:,}")

    except Exception as e:
        print(f"❌ Error al construir novedades N43: {e}")
        
    print(f"\n✅ ACTUALIZACIONES MASIVAS N43 APLICADAS EXITOSAMENTE")
    return df_ns

# Aplicar actualizaciones masivas N43
df_NS = aplicar_actualizaciones_masivas_N43(df_NS, maestro_ADRES, fecha_reporte)

## 🚩 N46 Reporte de datos de contacto 

In [ ]:
import pandas as pd # Se agrega import de pandas

def validar_novedades_N46(df, df_no_envio):
    """
    Valida registros con NOVEDAD = 'N46' (Reporte de datos de contacto).
    
    Proceso:
    1. COD_1_NOVEDAD: Si no inicia con "BRR ", se corrige anteponiéndolo. (MODIFICADO)
    2. COD_2_NOVEDAD: Debe tener 5 dígitos (DDMMM), si no tiene formato correcto se construye con DPR_ID + MNS_ID
    3. COD_3_NOVEDAD: Debe ser un teléfono válido colombiano (inicia con 3, 10 dígitos), si no cumple se vacía
    4. COD_4_NOVEDAD: Debe ser un email válido, si no cumple se vacía
    
    Args:
        df: DataFrame con novedades
        df_no_envio: DataFrame con registros rechazados
    
    Returns:
        df actualizado, df_no_envio actualizado
    """
    print("🔍 INICIANDO VALIDACIÓN DE NOVEDADES N46")
    print("="*60)
    
    # Filtrar solo registros N46
    mask_n46 = df['NOVEDAD'] == 'N46'
    registros_n46 = df[mask_n46].copy()
    total_n46_inicial = len(registros_n46)
    
    print(f"📊 Total de registros N46 iniciales: {total_n46_inicial}")
    
    if total_n46_inicial == 0:
        print("⚠️ No se encontraron registros N46")
        return df, df_no_envio
    
    # Inicializar contadores de corrección
    correcciones_cod1 = 0
    total_invalidos_cod1 = 0 # Se mantiene en 0 porque ya no se rechazan en este punto
    
    # =========================================================================
    # CORRECCIÓN 1: COD_1_NOVEDAD DEBE INICIAR CON "BRR " (Lógica de corrección)
    # =========================================================================
    print(f"\n🔧 CORRECCIÓN 1: Estandarizando COD_1_NOVEDAD para que inicie con 'BRR '")
    
    # Normalizar y verificar
    registros_n46['COD_1_NOVEDAD_NORM'] = registros_n46['COD_1_NOVEDAD'].astype(str).str.strip()
    mask_valido_cod1 = registros_n46['COD_1_NOVEDAD_NORM'].str.upper().str.startswith('BRR ')
    
    mask_a_corregir = ~mask_valido_cod1
    
    if mask_a_corregir.any():
        
        # Guardar solo la parte que sigue a 'BRR ' (el contenido original)
        contenido_original = registros_n46.loc[mask_a_corregir, 'COD_1_NOVEDAD_NORM']
        
        # Aplicar la corrección: anteponer 'BRR ' al contenido original
        nuevo_valor_cod1 = 'BRR ' + contenido_original
        
        # Contar las correcciones
        correcciones_cod1 = mask_a_corregir.sum()
        
        # 1. Actualizar el DataFrame principal (df) con los valores corregidos
        indices_a_corregir_df = df.loc[mask_n46][mask_a_corregir].index
        df.loc[indices_a_corregir_df, 'COD_1_NOVEDAD'] = nuevo_valor_cod1.values
        
        # 2. Actualizar el DataFrame temporal (registros_n46)
        registros_n46.loc[mask_a_corregir, 'COD_1_NOVEDAD'] = nuevo_valor_cod1
        
        print(f"   🔧 Registros corregidos (se les antepuso 'BRR '): {correcciones_cod1}")
        
        if correcciones_cod1 > 0:
            print(f"\n   📋 Ejemplo de COD_1_NOVEDAD corregido:")
            ejemplo_corregido = df.loc[indices_a_corregir_df, ['NUM_SOLICITUD_NOVEDAD', 'HST_IDN_NUMERO_IDENTIFICACION', 'COD_1_NOVEDAD']].head(1)
            print(ejemplo_corregido.to_string(index=False))

    else:
        print("   ✅ Todos los registros N46 ya iniciaban con 'BRR '.")
    
    # Actualizar registros_n46 (ya se hizo en el paso 2, solo quitamos la columna temporal y re-filtramos)
    registros_n46 = df[df['NOVEDAD'] == 'N46'].copy() 
    
    # =========================================================================
    # VALIDACIÓN 2: COD_2_NOVEDAD DEBE TENER 5 DÍGITOS (DDMMM)
    # =========================================================================
    print(f"\n🔧 VALIDACIÓN 2: Validando y corrigiendo COD_2_NOVEDAD (código de municipio)")
    
    def validar_codigo_municipio(valor):
        """Valida si es un código de 5 dígitos"""
        if pd.isna(valor) or str(valor).strip() == '':
            return False
        
        valor_str = str(valor).strip()
        
        # Debe tener exactamente 5 dígitos
        if len(valor_str) != 5 or not valor_str.isdigit():
            return False
        
        return True
    
    # Aplicar validación
    registros_n46['COD_2_VALIDO'] = registros_n46['COD_2_NOVEDAD'].apply(validar_codigo_municipio)
    invalidos_cod2 = (~registros_n46['COD_2_VALIDO']).sum()
    
    print(f"   ✅ Registros con COD_2_NOVEDAD válido (5 dígitos): {registros_n46['COD_2_VALIDO'].sum()}")
    print(f"   🔧 Registros que requieren corrección: {invalidos_cod2}")
    
    # Corregir los inválidos construyendo con DPR_ID + MNS_ID
    correcciones_cod2 = 0
    
    for idx in registros_n46[~registros_n46['COD_2_VALIDO']].index:
        # Obtener DPR_ID (2 dígitos) y MNS_ID (3 dígitos)
        # Importante: se usa df.loc[idx] para obtener DPR_ID/MNS_ID del dataframe principal.
        dpr_id = str(df.loc[idx, 'DPR_ID']).strip().zfill(2)
        mns_id = str(df.loc[idx, 'MNS_ID']).strip().zfill(3)
        
        # Validar que sean numéricos
        if dpr_id.isdigit() and mns_id.isdigit():
            codigo_municipio = dpr_id + mns_id
            df.loc[idx, 'COD_2_NOVEDAD'] = codigo_municipio
            correcciones_cod2 += 1
    
    print(f"   ✅ COD_2_NOVEDAD corregidos: {correcciones_cod2}")
    
    # Mostrar ejemplos de correcciones
    if correcciones_cod2 > 0:
        print(f"\n   📋 Ejemplos de COD_2_NOVEDAD corregidos:")
        registros_actualizados = df[df['NOVEDAD'] == 'N46'][[
            'NUM_SOLICITUD_NOVEDAD', 'HST_IDN_NUMERO_IDENTIFICACION', 
            'DPR_ID', 'MNS_ID', 'COD_2_NOVEDAD'
        ]].tail(5) # Usamos tail para mostrar correcciones recientes
        print(registros_actualizados.to_string(index=False))
    
    # =========================================================================
    # VALIDACIÓN 3: COD_3_NOVEDAD DEBE SER UN TELÉFONO VÁLIDO
    # =========================================================================
    print(f"\n🔧 VALIDACIÓN 3: Validando y limpiando COD_3_NOVEDAD (teléfono)")
    
    def validar_telefono_colombia(valor):
        """
        Valida si es un teléfono válido colombiano:
        - Inicia con 3
        - Tiene exactamente 10 dígitos
        - Solo números, sin espacios
        """
        if pd.isna(valor) or str(valor).strip() == '':
            return None
        
        # Limpiar: solo números
        valor_limpio = ''.join(filter(str.isdigit, str(valor)))
        
        # Validar formato
        if len(valor_limpio) == 10 and valor_limpio.startswith('3'):
            return valor_limpio
        else:
            return None
    
    # Aplicar validación y corrección
    telefonos_validos_antes = df[df['NOVEDAD'] == 'N46']['COD_3_NOVEDAD'].notna().sum()
    
    for idx in df[df['NOVEDAD'] == 'N46'].index:
        valor_actual = df.loc[idx, 'COD_3_NOVEDAD']
        telefono_validado = validar_telefono_colombia(valor_actual)
        
        df.loc[idx, 'COD_3_NOVEDAD'] = telefono_validado if telefono_validado else ''
    
    telefonos_validos_despues = (df[df['NOVEDAD'] == 'N46']['COD_3_NOVEDAD'] != '').sum()
    telefonos_limpiados = telefonos_validos_antes - telefonos_validos_despues
    
    print(f"   ✅ Teléfonos válidos: {telefonos_validos_despues}")
    print(f"   🧹 Teléfonos inválidos limpiados: {telefonos_limpiados}")
    
    # Mostrar ejemplos
    registros_con_telefono = df[(df['NOVEDAD'] == 'N46') & (df['COD_3_NOVEDAD'] != '')]
    if len(registros_con_telefono) > 0:
        print(f"\n   📋 Ejemplos de teléfonos válidos:")
        ejemplos_telefonos = registros_con_telefono[[
            'NUM_SOLICITUD_NOVEDAD', 'HST_IDN_NUMERO_IDENTIFICACION', 'COD_3_NOVEDAD'
        ]].head(5)
        print(ejemplos_telefonos.to_string(index=False))
    
    # =========================================================================
    # VALIDACIÓN 4: COD_4_NOVEDAD DEBE SER UN EMAIL VÁLIDO
    # =========================================================================
    print(f"\n🔧 VALIDACIÓN 4: Validando y limpiando COD_4_NOVEDAD (email)")
    
    
    def validar_email(valor):
        """
        Valida si es un email válido usando expresión regular
        """
        if pd.isna(valor) or str(valor).strip() == '':
            return None
        
        valor_limpio = str(valor).strip()
        
        # Patrón básico de email
        patron_email = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
        
        if re.match(patron_email, valor_limpio):
            return valor_limpio
        else:
            return None
    
    # Aplicar validación y corrección
    emails_validos_antes = df[df['NOVEDAD'] == 'N46']['COD_4_NOVEDAD'].notna().sum()
    
    for idx in df[df['NOVEDAD'] == 'N46'].index:
        valor_actual = df.loc[idx, 'COD_4_NOVEDAD']
        email_validado = validar_email(valor_actual)
        
        df.loc[idx, 'COD_4_NOVEDAD'] = email_validado if email_validado else ''
    
    emails_validos_despues = (df[df['NOVEDAD'] == 'N46']['COD_4_NOVEDAD'] != '').sum()
    emails_limpiados = emails_validos_antes - emails_validos_despues
    
    print(f"   ✅ Emails válidos: {emails_validos_despues}")
    print(f"   🧹 Emails inválidos limpiados: {emails_limpiados}")
    
    # Mostrar ejemplos
    registros_con_email = df[(df['NOVEDAD'] == 'N46') & (df['COD_4_NOVEDAD'] != '')]
    if len(registros_con_email) > 0:
        print(f"\n   📋 Ejemplos de emails válidos:")
        ejemplos_emails = registros_con_email[[
            'NUM_SOLICITUD_NOVEDAD', 'HST_IDN_NUMERO_IDENTIFICACION', 'COD_4_NOVEDAD'
        ]].head(5)
        print(ejemplos_emails.to_string(index=False))
    
    # =========================================================================
    # RESUMEN FINAL
    # =========================================================================
    registros_n46_final = len(df[df['NOVEDAD'] == 'N46'])
    
    print(f"\n📈 RESUMEN FINAL:")
    print(f"   Registros N46 iniciales: {total_n46_inicial}")
    # Nota: total_invalidos_cod1 es 0 porque ya no se rechazan por el COD_1, se corrigen.
    print(f"   Registros rechazados (COD_1 inválido): {total_invalidos_cod1} (fueron corregidos)") 
    print(f"   Registros N46 restantes: {registros_n46_final}")
    print(f"   Total en DF_No_Envio actual: {len(df_no_envio)}")
    
    print(f"\n📋 CORRECCIONES APLICADAS:")
    print(f"   COD_1_NOVEDAD estandarizados (anteponiendo 'BRR '): {correcciones_cod1}")
    print(f"   COD_2_NOVEDAD corregidos (construidos con DPR_ID + MNS_ID): {correcciones_cod2}")
    print(f"   COD_3_NOVEDAD limpiados (teléfonos inválidos): {telefonos_limpiados}")
    print(f"   COD_4_NOVEDAD limpiados (emails inválidos): {emails_limpiados}")
    
    # Mostrar estadísticas finales
    if registros_n46_final > 0:
        registros_finales = df[df['NOVEDAD'] == 'N46']
        
        print(f"\n📊 ESTADÍSTICAS DE DATOS DE CONTACTO:")
        print(f"   Registros con teléfono válido: {(registros_finales['COD_3_NOVEDAD'] != '').sum()}")
        print(f"   Registros con email válido: {(registros_finales['COD_4_NOVEDAD'] != '').sum()}")
        print(f"   Registros con ambos datos: {((registros_finales['COD_3_NOVEDAD'] != '') & (registros_finales['COD_4_NOVEDAD'] != '')).sum()}")
        print(f"   Registros sin ningún dato de contacto: {((registros_finales['COD_3_NOVEDAD'] == '') & (registros_finales['COD_4_NOVEDAD'] == '')).sum()}")
        
        print(f"\n📋 EJEMPLOS DE REGISTROS N46 FINALES:")
        ejemplos_finales = registros_finales[[
            'NUM_SOLICITUD_NOVEDAD', 'HST_IDN_NUMERO_IDENTIFICACION',
            'COD_1_NOVEDAD', 'COD_2_NOVEDAD', 'COD_3_NOVEDAD', 'COD_4_NOVEDAD'
        ]].head(10)
        print(ejemplos_finales.to_string(index=False))
    
    print(f"\n✅ VALIDACIÓN N46 COMPLETADA")
    
    return df, df_no_envio

# Aplicar validación N46
df_NS, DF_No_Envio = validar_novedades_N46(df_NS, DF_No_Envio)

# Limpiar nan

In [ ]:
#Limpiar valores NaN en df_NS reemplazándolos por cadenas vacías
print("🧹 LIMPIANDO VALORES NaN EN df_NS")
print("="*50)

# Mostrar información antes de la limpieza
total_registros = len(df_NS)
total_celdas = df_NS.size
nan_antes = df_NS.isna().sum().sum()

print(f"📊 Registros totales: {total_registros:,}")
print(f"📊 Celdas totales: {total_celdas:,}")
print(f"📊 Valores NaN antes: {nan_antes:,}")

# Reemplazar todos los valores NaN por cadenas vacías
df_NS = df_NS.fillna('')

# Verificar después de la limpieza
nan_despues = df_NS.isna().sum().sum()

print(f"📊 Valores NaN después: {nan_despues:,}")
print(f"✅ Se limpiaron {nan_antes:,} valores NaN")

# Mostrar ejemplo de primeras filas después de la limpieza
print(f"\n📋 Ejemplo de datos después de la limpieza:")
print(f"Shape: {df_NS.shape}")
print("\nPrimeras 3 filas y primeras 10 columnas:")
print(df_NS.iloc[:3, :10].to_string())

print(f"\n✅ LIMPIEZA DE NaN COMPLETADA")

# 🚩🚩Estandarizar N21 SIE 4 y 06
## N32 Parentesco SIE
14535,EPS025,RC,1115870139,SILVA,NIÑO,LISBETH,SAMARA,07/07/2022,85,250,N32,10/11/2025,CC,1115864791,*A*,2,,,,GE0030
14551,EPS025,CC,1067031533,BENAVIDES,MORA,YINETH,CAROLINA,20/10/2005,85,125,N32,10/11/2025,CC,1066086055,B,*9,4*,,,GE0204
## N36
14566,EPS025,RC,1115869036,VALOR,SOLANO,ALAN,,23/09/2020,85,250,N36,11/11/2025,F,4,B01,5,1,06,,GE0200
14567,EPS025,TI,1157963502,ENDES,FANDIÑO,AUDRY,STHEFANY,24/02/2010,85,001,N36,12/11/2025,F,5,,9,N,06,,GE0201;GE0202
14568,EPS025,CC,1006450245,BARRETO,CALDERON,ANA,VICTORIA,04/01/1991,85,001,N36,11/11/2025,1,4,C06,5,2,06,,GE0060;GE0200
14569,EPS025,TI,1118704349,NARANJO,VELANDIA,KELLY,JOHANNA,24/12/2007,85,250,N36,11/11/2025,F,4,A02,5,1,06,,GE0200
14570,EPS025,CC,1115862059,SIBO,CORREDOR,PAULA,ANDREA,02/09/1995,85,250,N36,13/11/2025,1,,,9,,06,,GE0060;GE0201;GE0202;GE0200
14571,EPS025,TI,1118650729,SEGUA,ALBARRACIN,CLARIS,ZAMANTHA,03/05/2016,85,125,N36,07/11/2025,1,4,A03,5,1,06,,GE0060;GE0200
14572,EPS025,TI,1029658524,ALBARRACIN,ROMERO,VALERIN,YELENYS,20/10/2011,85,125,N36,07/11/2025,1,4,A03,5,1,06,,GE0060;GE0200
14573,EPS025,CC,1006636098,ALBARRACIN,ROMERO,MILDRED,ROCIO,26/08/1991,85,125,N36,07/11/2025,1,4,A03,5,1,06,,GE0060;GE0200
14574,EPS025,RC,1118651668,SEGUA,ALBARRACIN,ASLY,ADARA,05/08/2022,85,125,N36,07/11/2025,1,4,A03,5,1,06,,GE0060;GE0200

## N39
14660,EPS025,CC,1193519273,GONZALEZ,BELLO,ESNEYDER,ALBERTO,22/09/1998,85,125,N39,30/07/2026,0,85,001,,,,,GE0079
14661,EPS025,CC,1116544784,SALAMANCA,CORREA,YEISON,HERNEY,11/10/2003,85,010,N39,14/02/2026,0,85,001,,,,,GE0079
## N41
14680,EPS025,CC,33646632,VEGA,SANABRIA,JENNY,LISEDT,21/02/1983,85,010,N41,11/11/2025,5,,,,,,,GE0305
14681,EPS025,TI,1115917869,PAEZ,NUÑEZ,PAMELA,DENISE,18/10/2015,85,410,N41,08/11/2025,3,,,,,,,GE0305
14682,EPS025,CC,51958092,DOMINGUEZ,VERA,SONIA,ISABEL,22/05/1965,85,010,N41,10/11/2025,4,,,,,,,GE0305
14683,EPS025,CC,1006783781,MORENO,BOHORQUEZ,MIGUEL,ESTEBAN,10/07/1996,85,162,N41,10/11/2025,5,,,,,,,GE0305
## N46
15008,EPS025,RC,1116044816,PIDIACHE,DEDIOS,XAVIER  ,SNEIDER,21/05/2024,85,263,N46,11/11/2025,BRR CARRERA 9  03 50,85263,3223764941,,,,,GE0085
15870,EPS025,CC,1029644056,BELTRAN,ALVAREZ,JUAN,SEBASTIAN,28/04/2006,85,001,N46,12/11/2025,BRR N,85001,3000000000,actualizar@actualizar.com,,,,GE0314
16530,EPS025,CC,17555227,RODRIGUEZ,GUTIERREZ,OLIVERIO,,22/07/1957,85,410,N46,10/11/2025,BRR Actualizar,85410,3000000000,actualizar@actualizar.com,,,,GE0314
16557,EPS025,CC,4184360,GUINA,GUANARO,ALFONSO,,02/01/1963,85,225,N46,09/11/2025,BRR Actualizar,85225,3000000000,actualizar@actualizar.com,,,,GE0314


In [ ]:
df_NS.columns


In [ ]:
maestro_ADRES.columns

# GUARDAR 

In [ ]:
print("Valores únicos en la columna NOVEDAD:") 
print(df_NS['NOVEDAD'].unique())

In [ ]:
maestro_ADRES

In [ ]:
# Guardar con nombres de hojas más descriptivos
output_file = os.path.join(Ruta_Salida, f"Datos_Procesados_{fecha_archivo}.xlsx")

print(f"💾 GUARDANDO DATAFRAMES PROCESADOS")
print(f"📁 Archivo: {output_file}")
print("="*60)

try:
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        # Guardar maestro_ADRES
        maestro_ADRES.to_excel(writer, sheet_name='Maestro_ADRES_Limpio', index=False)
        print(f"✅ Maestro ADRES guardado:")
        print(f"   📊 Filas: {len(maestro_ADRES):,}")
        print(f"   📊 Columnas: {len(maestro_ADRES.columns)}")
        
        # Guardar df_NS
        df_NS.to_excel(writer, sheet_name='NS_Validado', index=False)
        print(f"✅ DataFrame NS guardado:")
        print(f"   📊 Filas: {len(df_NS):,}")
        print(f"   📊 Columnas: {len(df_NS.columns)}")
        
        # Guardar DF_No_Envio
        DF_No_Envio.to_excel(writer, sheet_name='NS_No_Envio', index=False)
        print(f"✅ DataFrame No Envío guardado:")
        print(f"   📊 Filas: {len(DF_No_Envio):,}")
        print(f"   📊 Columnas: {len(DF_No_Envio.columns)}")
        
        # Guardar df_IPS
        df_IPS.to_excel(writer, sheet_name='IPS', index=False)
        print(f"✅ DataFrame IPS guardado:")
        print(f"   📊 Filas: {len(df_IPS):,}")
        print(f"   📊 Columnas: {len(df_IPS.columns)}")
        
        # Resumen de novedades en df_NS
        if 'NOVEDAD' in df_NS.columns:
            print(f"   📋 Tipos de novedad válidas: {df_NS['NOVEDAD'].value_counts().to_dict()}")
        
        # Resumen de motivos de rechazo en DF_No_Envio
        if 'motivo' in DF_No_Envio.columns and len(DF_No_Envio) > 0:
            print(f"   📋 Motivos de rechazo: {DF_No_Envio['motivo'].value_counts().to_dict()}")
    
    print(f"\n🎉 GUARDADO EXITOSO")
    print(f"📂 Ubicación: {output_file}")
    print(f"📋 Hojas creadas: 'Maestro_ADRES_Limpio', 'NS_Validado', 'NS_No_Envio'")
    
    # Mostrar estadísticas finales
    print(f"\n📊 ESTADÍSTICAS FINALES:")
    print(f"   Total registros procesados: {len(df_NS) + len(DF_No_Envio):,}")
    print(f"   Registros válidos para envío: {len(df_NS):,}")
    print(f"   Registros rechazados: {len(DF_No_Envio):,}")
    if len(df_NS) + len(DF_No_Envio) > 0:
        porcentaje_validos = (len(df_NS) / (len(df_NS) + len(DF_No_Envio))) * 100
        print(f"   Porcentaje de validez: {porcentaje_validos:.2f}%")
    
except Exception as e:
    print(f"❌ Error al guardar: {e}")